# Import libraries & data

Article: [Temporal evolution of the relationship between political partisanship, underlying social vulnerabilities, and COVID-19 mortality across U.S. counties](https://deliverypdf.ssrn.com/delivery.php?ID=140021013065022109105064093124000120105018010061023037119085116064086000000026017000035020062104054111107000093000109110026026037007090023044116120031078127095119070054065073120025098112098068013072098094092019029127003096101011071108122077110028014066&EXT=pdf&INDEX=TRUE)

In [ ]:
# upgrade excel package to load specific files
%pip install -U xlrd
# county choropleth graph
%pip install -U geopandas
%pip install -U pyshp
%pip install -U shapely
%pip install -U plotly-geo
%pip install -U xgboost
%pip install -U lightgbm
%pip install -U scikit-learn
%pip install -U tslearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm

If working on Google Colab with Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
if (os.getcwd() != "/content/drive/MyDrive/MIT Internship/American Communities") and (os.getcwd() != "/content/drive/My Drive/MIT Internship/American Communities"):
  os.chdir("drive/MyDrive/MIT Internship/American Communities")

Execute the cells below to directly import the datasets (instead of calculate everything)

In [ ]:
type_data = "COVID-19"  # default type: All Causes, COVID-19, Excess Mortality

county_databases = {}

# Load all causes dataset
county_database = pd.read_csv("county_database_all_causes.csv")
county_database.index = county_database.FIPS
county_database2 = pd.read_csv("county_database2_all_causes.csv")
county_database2.index = county_database2.FIPS
county_database2_imputed = pd.read_csv("county_database2_imputed_all_causes.csv")
county_database2_imputed.index = county_database2_imputed.FIPS

county_database.drop(columns=["FIPS.1"], inplace=True)
county_database2.drop(columns=["FIPS.1"], inplace=True)
county_database2_imputed.drop(columns=["FIPS.1"], inplace=True)

county_databases["All Causes"] = {}
county_databases["All Causes"]["county_database"] = county_database
county_databases["All Causes"]["county_database2"] = county_database2
county_databases["All Causes"]["county_database2_imputed"] = county_database2_imputed

# Load COVID-19 dataset
county_database = pd.read_csv("county_database_covid19.csv")
county_database.index = county_database.FIPS
county_database2 = pd.read_csv("county_database2_covid19.csv")
county_database2.index = county_database2.FIPS
county_database2_imputed = pd.read_csv("county_database2_imputed_covid19.csv")
county_database2_imputed.index = county_database2_imputed.FIPS

county_database.drop(columns=["FIPS.1"], inplace=True)
county_database2.drop(columns=["FIPS.1"], inplace=True)
county_database2_imputed.drop(columns=["FIPS.1"], inplace=True)

county_databases["COVID-19"] = {}
county_databases["COVID-19"]["county_database"] = county_database
county_databases["COVID-19"]["county_database2"] = county_database2
county_databases["COVID-19"]["county_database2_imputed"] = county_database2_imputed

# Load Excess Mortality dataset
county_database = pd.read_csv("county_database_excess_mortality.csv")
county_database.index = county_database.FIPS
county_database2 = pd.read_csv("county_database2_excess_mortality.csv")
county_database2.index = county_database2.FIPS
county_database2_imputed = pd.read_csv("county_database2_imputed_excess_mortality.csv")
county_database2_imputed.index = county_database2_imputed.FIPS

county_database.drop(columns=["FIPS.1"], inplace=True)
county_database2.drop(columns=["FIPS.1"], inplace=True)
county_database2_imputed.drop(columns=["FIPS.1"], inplace=True)

county_databases["Excess Mortality"] = {}
county_databases["Excess Mortality"]["county_database"] = county_database
county_databases["Excess Mortality"]["county_database2"] = county_database2
county_databases["Excess Mortality"]["county_database2_imputed"] = county_database2_imputed

with open("feature_selection", "rb") as fp:  # load feature selection
  features = pickle.load(fp)
  selected_columns_list = features[0]
  X_selected_columns_list = features[1]

# load default dataset
county_database = county_databases[type_data]["county_database"]
county_database2 = county_databases[type_data]["county_database2"]
county_database2_imputed = county_databases[type_data]["county_database2_imputed"]

# Create datasets

## Load & compile datasets

### Response Variable (1 dataset for each response variable)

#### John Hopkins University
[COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19)

In [ ]:
if not os.path.exists("COVID-19_JHU_data"):
  os.mkdir("COVID-19_JHU_data")

In [ ]:
# COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University
"""
!unzip csse_covid_19_daily_reports.zip -d COVID-19_JHU_data
"""

In [ ]:
import glob
import math
df_to_concatenate = []
for filename in glob.glob(r"COVID-19_JHU_data/csse_covid_19_daily_reports/*.csv"):
  df_date = pd.read_csv(filename)
  if "FIPS" in df_date.columns:  # county-level data available
    df_date = df_date[df_date["Country_Region"] == "US"]
    date = filename.split("\\")[-1][:-4]
    date = date[6:] + "-" + date[:2] + "-" + date[3:5]
    df_date.index = df_date["FIPS"].apply(lambda x: str(int(x)) if not(math.isnan(x)) else np.nan)
    df_date = df_date[["FIPS", "Deaths"]].dropna()  # remove county with no FIPS/no data
    df_date = df_date[["Deaths"]]
    df_date.columns = [date]
    df_date = df_date.T

    # Merge duplicated counties
    if df_date.columns.duplicated().any():
      df_date = df_date.sum(axis=1, level=0, skipna=True)

    df_to_concatenate.append(df_date)
  
# Merge all the dates
df = pd.concat(df_to_concatenate)
# Sort the dataframe by chronological order
df.sort_index(inplace=True)
df = df.fillna(method="ffill")  # fill na forward
df = df.cummax()  # cumulative max to deal with sudden drop of deaths
df = df.fillna(0)  # then, nan=no data before (so replace with 0)
df

In [ ]:
# applied a 7-day average to smooth the daily death counts to account for noise
# in the data at the daily level

###########
# Floor ??
###########

df = df.rolling(window=7).mean()
df

In [ ]:
df.to_csv("COVID-19_timeseries.csv")

[US County Boundaries](https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?flg=fr&disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name) and the custom python script us_county_boundaries_to_coordinates.py

In [ ]:
jhu_data = pd.read_csv("COVID-19_JHU_data/csse_covid_19_daily_reports/03-06-2022.csv")
jhu_data = jhu_data[jhu_data["Country_Region"] == "US"]
jhu_data["FIPS"] = jhu_data["FIPS"].apply(lambda x: str(x).replace(".0", ""))
jhu_data = jhu_data.drop_duplicates(subset="FIPS", keep="first")  # drop duplicated counties
jhu_data = jhu_data[["FIPS", "Province_State"]]

us_county_coordinates = pd.read_csv("us_county_coordinates.csv")
lat_list = []
long_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    lat = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Lat"].iloc[0]
  except Exception as e:
    lat = np.nan
  try:
    long = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Long_"].iloc[0]
  except Exception as e:
    long = np.nan
  lat_list.append(lat)
  long_list.append(long)

jhu_data["Lat"] = lat_list
jhu_data["Long_"] = long_list
jhu_data = jhu_data[~jhu_data["FIPS"].isin(["nan"])]
jhu_data

#### Provisional Multiple Cause of Death Data [CDC Wonder](https://wonder.cdc.gov/mcd.html)

Group by: Residence county, Year, Month

In [ ]:
cdc2020_1 = pd.read_csv("CDC Wonder/CDC_2020_1.csv")  # Jan-Jun 2020
cdc2020_1 = cdc2020_1[~cdc2020_1["Month Code"].isna()]
cdc2020_2 = pd.read_csv("CDC Wonder/CDC_2020_2.csv")  # Jul-Dec 2020
cdc2020_2 = cdc2020_2[~cdc2020_2["Month Code"].isna()]
cdc2021_1 = pd.read_csv("CDC Wonder/CDC_2021_1.csv")  # Jan-Jun 2021
cdc2021_1 = cdc2021_1[~cdc2021_1["Month Code"].isna()]
cdc2021_2 = pd.read_csv("CDC Wonder/CDC_2021_2.csv")  # Jul-Dec 2021
cdc2021_2 = cdc2021_2[~cdc2021_2["Month Code"].isna()]
cdc2022_1 = pd.read_csv("CDC Wonder/CDC_2022_1.csv")  # Jan-Feb 2022
cdc2022_1 = cdc2022_1[~cdc2022_1["Month Code"].isna()]

df1 = pd.concat([cdc2020_1, cdc2020_2, cdc2021_1, cdc2021_2, cdc2022_1])
df1["Month Code"] = pd.to_datetime(df1["Month Code"], format="%Y/%m")
df1["Residence County Code"] = df1["Residence County Code"].apply(lambda x: str(int(x)))  # remove trailing zeros
df1

In [ ]:
fips = np.unique(df1["Residence County Code"])[0]
df_fips = df1[df1["Residence County Code"] == fips][["Deaths", "Month Code"]]
df_fips.index = df_fips["Month Code"]
df_fips = df_fips[["Deaths"]]
df_fips.columns = [fips]
df = df_fips.copy()
months = [f"0{i}" for i in range(1, 10)] + ["10", "11", "12"]
complete_index_list = [pd.to_datetime(f"{y}-{m}-01") for y in [2020, 2021] for m in months] + [pd.to_datetime("2022-01-01"), pd.to_datetime("2022-02-01")]
df = df.reindex(complete_index_list, fill_value=np.nan)
for fips in np.unique(df1["Residence County Code"])[1:]:
  df_fips = df1[df1["Residence County Code"] == fips][["Deaths", "Month Code"]]
  df_fips.index = df_fips["Month Code"]
  df_fips = df_fips[["Deaths"]]
  df_fips.columns = [fips]
  df = df.join(df_fips)
######
# df = df.rolling(window=7).mean()
######
df = df.cumsum()  # cumulative sum
df = df.fillna(method="ffill")  # fill na forward
df = df.fillna(0)  # then, nan=no data before (so replace with 0)
df

[US County Boundaries](https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?flg=fr&disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name) and the custom python script us_county_boundaries_to_coordinates.py

In [ ]:
state_id = pd.read_csv("state_id.csv", encoding="latin")
state_id = {y: x for x, y in state_id.values}
state_id

In [ ]:
jhu_data = df1[["Residence County Code", "Residence County"]]
jhu_data.columns = ["FIPS", "Province_State"]
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: x.split(", ")[1])
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: state_id[x] if x in state_id else x)
jhu_data["FIPS"] = jhu_data["FIPS"].apply(lambda x: str(int(x)))
jhu_data = jhu_data.drop_duplicates(subset="FIPS", keep="first")  # drop duplicated counties

us_county_coordinates = pd.read_csv("us_county_coordinates.csv")
lat_list = []
long_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    lat = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Lat"].iloc[0]
  except Exception as e:
    lat = np.nan
  try:
    long = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Long_"].iloc[0]
  except Exception as e:
    long = np.nan
  lat_list.append(lat)
  long_list.append(long)

jhu_data["Lat"] = lat_list
jhu_data["Long_"] = long_list
jhu_data = jhu_data[~jhu_data["FIPS"].isin(["nan"])]
jhu_data

#### [CDC Wonder](https://wonder.cdc.gov/mcd.html) - COVID-19 (not reliable)

Group by: Residence county, Year, Month

Select multiple cause of death: U*07.1 (COVID-19)

In [ ]:
cdc2020_1 = pd.read_csv("CDC Wonder/CDC_covid_2020_1.csv")  # Jan-Jun 2020
cdc2020_1 = cdc2020_1[~cdc2020_1["Month Code"].isnull()]
cdc2020_2 = pd.read_csv("CDC Wonder/CDC_covid_2020_2.csv")  # Jul-Dec 2020
cdc2020_2 = cdc2020_2[~cdc2020_2["Month Code"].isnull()]
cdc2021_1 = pd.read_csv("CDC Wonder/CDC_covid_2021_1.csv")  # Jan-Jun 2021
cdc2021_1 = cdc2021_1[~cdc2021_1["Month Code"].isnull()]
cdc2021_2 = pd.read_csv("CDC Wonder/CDC_covid_2021_2.csv")  # Jul-Dec 2021
cdc2021_2 = cdc2021_2[~cdc2021_2["Month Code"].isnull()]
cdc2022_1 = pd.read_csv("CDC Wonder/CDC_covid_2022_1.csv")  # Jan-Feb 2022
cdc2022_1 = cdc2022_1[~cdc2022_1["Month Code"].isnull()]

df1 = pd.concat([cdc2020_1, cdc2020_2, cdc2021_1, cdc2021_2, cdc2022_1])
df1["Month Code"] = pd.to_datetime(df1["Month Code"], format="%Y/%m")
df1["Residence County Code"] = df1["Residence County Code"].apply(lambda x: str(int(x)))  # remove trailing zeros
df1

In [ ]:
fips = np.unique(df1["Residence County Code"])[0]
df_fips = df1[df1["Residence County Code"] == fips][["Deaths", "Month Code"]]
df_fips.index = df_fips["Month Code"]
df_fips = df_fips[["Deaths"]]
df_fips.columns = [fips]
df = df_fips.copy()
months = [f"0{i}" for i in range(1, 10)] + ["10", "11", "12"]
complete_index_list = [pd.to_datetime(f"{y}-{m}-01") for y in [2020, 2021] for m in months] + [pd.to_datetime("2022-01-01"), pd.to_datetime("2022-02-01")]
df = df.reindex(complete_index_list, fill_value=np.nan)
for fips in np.unique(df1["Residence County Code"])[1:]:
  df_fips = df1[df1["Residence County Code"] == fips][["Deaths", "Month Code"]]
  df_fips.index = df_fips["Month Code"]
  df_fips = df_fips[["Deaths"]]
  df_fips.columns = [fips]
  df = df.join(df_fips)
######
# df = df.rolling(window=7).mean()
######
df = df.cumsum()  # cumulative sum
df = df.fillna(method="ffill")  # fill na forward
df = df.fillna(0)  # then, nan=no data before (so replace with 0)
df

[US County Boundaries](https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?flg=fr&disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name) and the custom python script us_county_boundaries_to_coordinates.py

In [ ]:
state_id = pd.read_csv("state_id.csv", encoding="latin")
state_id = {y: x for x, y in state_id.values}

In [ ]:
jhu_data = df1[["Residence County Code", "Residence County"]]
jhu_data.columns = ["FIPS", "Province_State"]
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: x.split(", ")[1])
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: state_id[x] if x in state_id else x)
jhu_data["FIPS"] = jhu_data["FIPS"].apply(lambda x: str(int(x)))
jhu_data = jhu_data.drop_duplicates(subset="FIPS", keep="first")  # drop duplicated counties

us_county_coordinates = pd.read_csv("us_county_coordinates.csv")
lat_list = []
long_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    lat = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Lat"].iloc[0]
  except Exception as e:
    lat = np.nan
  try:
    long = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Long_"].iloc[0]
  except Exception as e:
    long = np.nan
  lat_list.append(lat)
  long_list.append(long)

jhu_data["Lat"] = lat_list
jhu_data["Long_"] = long_list
jhu_data = jhu_data[~jhu_data["FIPS"].isin(["nan"])]
jhu_data

#### Excess Mortality - [CDC Wonder](https://wonder.cdc.gov/mcd.html) Monthly

Three different custom python scripts have benn executed to obtain the excess mortality data:

*  process_cdc_wonder_data.py to convert CDC Wonder text files to csv files
*  cdc_to_excessmort_format.py to merge CDC death datasets, add population data from [US Census Bureau](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html), more precisely [here](https://www2.census.gov/programs-surveys/popest/datasets/)
*  excessmort_cdc.R to apply the excessmort R package to our death data

Excess mortality data are calculated using the following formula:

$\mu_{t}=N_{t} exp(\alpha(t)+s(t))$

with $Nt$ the population at time $t$, $\alpha(t)$ a slow trend to account for the increase in life expectancy we have seen in the last few decades, a seasonal trend $s(t)$ to account for more deaths during the winter.

In [ ]:
excess_mort_data = pd.read_csv("CDC Wonder/excess_mort_data.csv")
excess_mort_data["date"] = pd.to_datetime(excess_mort_data["date"])
excess_mort_data

In [ ]:
fips = np.unique(excess_mort_data["FIPS"])[0]
df_fips = excess_mort_data[excess_mort_data["FIPS"] == fips][["excess_mortality", "date"]]
df_fips.index = df_fips["date"]
df_fips = df_fips[["excess_mortality"]]
df_fips.columns = [fips]
df = df_fips.copy()
months = [f"0{i}" for i in range(1, 10)] + ["10", "11", "12"]
complete_index_list = [pd.to_datetime(f"{y}-{m}-01") for y in [2020, 2021] for m in months] + [pd.to_datetime("2022-01-01"), pd.to_datetime("2022-02-01")]
df = df.reindex(complete_index_list, fill_value=np.nan)
for fips in np.unique(excess_mort_data["FIPS"])[1:]:
  df_fips = excess_mort_data[excess_mort_data["FIPS"] == fips][["excess_mortality", "date"]]
  df_fips.index = df_fips["date"]
  df_fips = df_fips[["excess_mortality"]]
  df_fips.columns = [fips]
  df = df.join(df_fips)
######
# df = df.rolling(window=7).mean()
######
df = df.cumsum()  # cumulative sum
df = df.fillna(method="ffill")  # fill na forward
df = df.fillna(0)  # then, nan=no data before (so replace with 0)
df

Drop outlier

In [ ]:
df[48153]

In [ ]:
df.drop(columns=[48153], inplace=True)

In [ ]:
df.to_csv("excess_mortality_timeseries.csv")

[US County Boundaries](https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?flg=fr&disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name) and the custom python script us_county_boundaries_to_coordinates.py

In [ ]:
state_id = pd.read_csv("state_id.csv", encoding="latin")
state_id = {y: x for x, y in state_id.values}
state_id

In [ ]:
jhu_data = excess_mort_data[["FIPS", "county"]]
jhu_data.columns = ["FIPS", "Province_State"]
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: x.split(", ")[1])
jhu_data["Province_State"] = jhu_data["Province_State"].apply(lambda x: state_id[x] if x in state_id else x)
jhu_data["FIPS"] = jhu_data["FIPS"].apply(lambda x: str(int(x)))
jhu_data = jhu_data.drop_duplicates(subset="FIPS", keep="first")  # drop duplicated counties

us_county_coordinates = pd.read_csv("us_county_coordinates.csv")
lat_list = []
long_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    lat = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Lat"].iloc[0]
  except Exception as e:
    lat = np.nan
  try:
    long = us_county_coordinates[us_county_coordinates["GEOID"] == int(fips)]["Long_"].iloc[0]
  except Exception as e:
    long = np.nan
  lat_list.append(lat)
  long_list.append(long)

jhu_data["Lat"] = lat_list
jhu_data["Long_"] = long_list
jhu_data = jhu_data[~jhu_data["FIPS"].isin(["nan"])]
jhu_data

### [Airports in the United States of America](https://data.humdata.org/dataset/ourairports-usa)

In [ ]:
airports_df_loc = pd.read_csv("us-airports.csv")
airports_df_loc

### US Department of Transportation
[International_Report_Passengers](https://data.transportation.gov/Aviation/International_Report_Passengers/xgub-n9bw)

In [ ]:
IRP = pd.read_csv("International_Report_Passengers.csv")
IRP["data_dte"] = pd.to_datetime(IRP["data_dte"], format="%m/%d/%Y")
# Filter flights before the pandemic
IRP = IRP[IRP["data_dte"] >= pd.to_datetime("2020-01-01")]
IRP = IRP[IRP["data_dte"] <= pd.to_datetime("2020-03-01")]
airports_foreign_passengers = IRP.groupby(by="usg_apt")["Total"].sum()
airports_foreign_passengers.sort_values(ascending=False, inplace=True)
airports_foreign_passengers

In [ ]:
plt.hist(airports_foreign_passengers)

In [ ]:
df_airports_foreign_passengers = airports_foreign_passengers.to_frame()
airport_list = list(df_airports_foreign_passengers.index)
latitude = []
longitude = []
for airport in airport_list:
  try:
    latitude.append(airports_df_loc[airports_df_loc["ident"] == "K{}".format(airport)]["latitude_deg"].iloc[0])
    longitude.append(airports_df_loc[airports_df_loc["ident"] == "K{}".format(airport)]["longitude_deg"].iloc[0])
  except Exception as e:  # small airport an an island
    latitude.append(np.nan)
    longitude.append(np.nan)
df_airports_foreign_passengers["latitude"] = latitude
df_airports_foreign_passengers["longitude"] = longitude
df_afp = df_airports_foreign_passengers  # shorter name
df_afp.reset_index(drop=False, inplace=True)
df_afp

Bubble Map Airport Frequentation early 2020 before the pandemic

In [ ]:
import plotly.graph_objects as go

text_list = []
for i in range(len(df_afp)):
  text_list.append("{}<br>Passengers {}".format(df_afp["usg_apt"].iloc[i], "{:,}".format(df_afp["Total"].iloc[i])))
df_afp["text"] = text_list
limits = [(0, 5), (5, 10), (10, 50), (50, 100), (100, len(df_afp))]
colors = ["royalblue", "crimson", "lightseagreen", "orange", "lightgrey"]
cities = []
scale = 2000

fig = go.Figure()
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_afp[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = "USA-states",
        lon = df_sub["longitude"],
        lat = df_sub["latitude"],
        text = df_sub["text"],
        marker = dict(
            size = df_sub["Total"] / scale,
            color = colors[i],
            line_color="rgb(40, 40, 40)",
            line_width=0.5,
            sizemode = "area"
        ),
        name = "{0} - {1} top airports (ito passengers)".format(lim[0]+1, lim[1])))

fig.update_layout(
        title_text = "2020 January-March total passengers per airport<br>(Click legend to toggle traces)",
        showlegend = True,
        geo = dict(
            scope = "usa",
            landcolor = "rgb(217, 217, 217)",
        )
    )
fig.show()

Compute distance

In [ ]:
# Lets' keep the top top_airport_number=50 airports (less than 25% of all airports in the US (except islands))
top_airport_number = 50
top_airport = df_afp[:top_airport_number]

# calculate for each county the minimum distance to a top airport
# don't put a distance = 0 if the airport is in the county. Compute the distance instead

dist_list = []  # store the minimum distance to a top airport for each county
closest_airport_list = []  # name of the closest top airport
for i in range(len(jhu_data)):
  lat_c = float(jhu_data["Lat"].iloc[i])  # latitude county i
  long_c = float(jhu_data["Long_"].iloc[i])  # longitude county i
  county_coord = np.array([lat_c, long_c])
  
  # initialize variables
  min_dist = np.inf
  closest_airport = np.nan
  # find min distance
  for j in range(top_airport_number):
    lat_a = float(df_afp["latitude"].iloc[j])  # latitude airport j
    long_a = float(df_afp["longitude"].iloc[j])  # longitude airport j
    airport_coord = np.array([lat_a, long_a])
    distance = np.linalg.norm(county_coord - airport_coord)  # L2 norm: euclidean distance
    if distance < min_dist:
      min_dist = distance
      closest_airport = df_afp["usg_apt"].iloc[j]
  dist_list.append(min_dist)
  closest_airport_list.append(closest_airport)

# Update John Hopkins dataset
jhu_data["min_distance_top_airport"] = dist_list
jhu_data["closest_airport"] = closest_airport_list
jhu_data

### HHS Regions
[HHS Region Table](https://hrbrmstr.github.io/cdcfluview/reference/hhs_regions.html) and a custom R script to transform the R dataset into a csv

In [ ]:
hhs = pd.read_csv("hhs_regions.csv")
hhs.drop(columns=["Unnamed: 0"], inplace=True)

hhs_regions = []
for i in range(len(jhu_data)):
  try:
    state = jhu_data["Province_State"].iloc[i]
    region = hhs[hhs["state_or_territory"] == state]["region_number"].iloc[0]
  except Exception as e:
    region = np.nan
  hhs_regions.append(region)
jhu_data["HHS Region"] = hhs_regions
jhu_data["HHS Region"]

In [ ]:
hhs_region_dic = {1: "Region 1 - Boston",
                  2: "Region 2 - New York",
                  3: "Region 3 - Philadelphia",
                  4: "Region 4 - Atlanta",
                  5: "Region 5 - Chicago",
                  6: "Region 6 - Dallas",
                  7: "Region 7 - Kansas City",
                  8: "Region 8 - Denver",
                  9: "Region 9 - San Francisco",
                  10: "Region 10 - Seattle"}

### Oxford COVID-19 Government Response Tracker
[COVID-19 GOVERNMENT RESPONSE TRACKER](https://github.com/OxCGRT/covid-policy-tracker/blob/master/data/OxCGRT_latest.csv)

In [ ]:
oxcgrt = pd.read_csv("OxCGRT_latest.csv")
oxcgrt = oxcgrt[oxcgrt["CountryName"] == "United States"]  # keep US data only
oxcgrt["Date"] = pd.to_datetime(oxcgrt["Date"], format="%Y%m%d")

# Date filter, between 2020/01/01 and max_date

max_date_list = ["2020-05-31", "2020-10-31", "2021-02-28", "2021-07-31",
                 "2021-10-31", "2022-02-28"]
min_date_list = ["2020-02-01", "2020-05-31", "2020-10-31", "2021-02-28",
                 "2021-07-31", "2021-10-31"]
max_date = pd.to_datetime("2021-02-01")

si_lists = []  # for plots after
std_lists = []  # for plots after

for per in range(6):
  min_date = pd.to_datetime(min_date_list[per])
  max_date = pd.to_datetime(max_date_list[per])
  oxcgrt2 = oxcgrt[oxcgrt["Date"] <= max_date]
  oxcgrt2 = oxcgrt2[oxcgrt2["Date"] > min_date]

  # group by state
  oxcgrt2 = oxcgrt2.groupby(by="RegionName")["StringencyIndex"]
  # extract features and back to dataframe format
  oxcgrt2_mean = oxcgrt2.mean().to_frame()  # calculate the mean of the Stringency Index over the study period
  oxcgrt2_median = oxcgrt2.median().to_frame()
  oxcgrt2_min = oxcgrt2.min().to_frame()
  oxcgrt2_max = oxcgrt2.max().to_frame()
  std_df = oxcgrt2.std().to_frame()

  # Update county database
  """
  # Not stringency index for these states. NaN value instead
  American Samoa
  Diamond Princess
  District of Columbia
  Grand Princess
  Guam
  Northern Mariana Islands
  Puerto Rico
  Virgin Islands
  """

  stringency_index_list = []  # mean
  si_median_list = []
  si_min_list = []
  si_max_list = []
  std_list = []
  for i in range(len(jhu_data)):
    try:
      state = jhu_data["Province_State"].iloc[i]
      si_mean = oxcgrt2_mean.loc[state].iloc[0]
      si_median = oxcgrt2_median.loc[state].iloc[0]
      si_min = oxcgrt2_min.loc[state].iloc[0]
      si_max = oxcgrt2_max.loc[state].iloc[0]
      std = std_df.loc[state].iloc[0]
    except Exception as e:
      si_mean = np.nan
      si_median = np.nan
      si_min = np.nan
      si_max = np.nan
      std = np.nan
    stringency_index_list.append(si_mean)
    si_median_list.append(si_median)
    si_min_list.append(si_min)
    si_max_list.append(si_max)
    std_list.append(std)
  
  si_lists.append(stringency_index_list)
  std_lists.append(std_list)

  jhu_data[f"StringencyIndex{per+1}_mean"] = stringency_index_list
  jhu_data[f"StringencyIndex{per+1}_median"] = si_median_list
  jhu_data[f"StringencyIndex{per+1}_min"] = si_min_list
  jhu_data[f"StringencyIndex{per+1}_max"] = si_max_list
  jhu_data[f"StringencyIndex{per+1}_std"] = std_list
jhu_data

Check the standard deviations'distributions to check if it's ok to take the mean of the stringency index over a period

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=3)

for per in range(6):
  row = (per // 3) + 1
  col = (per % 3) + 1
  meanSI = np.nanmean(std_lists[per])
  fig.add_trace(
      go.Histogram(x=std_lists[per],
                   name=f"Period {per+1}: mean={round(meanSI, 2)}"),
      row=row, col=col
  )

fig.update_layout(height=row*300, width=900,
                  title_text=f"Distribution of Stringency Index's standard deviation for each county for all the periods",
                  xaxis_title=f"Stringency index standard deviation",
                  yaxis_title="Count",
                  legend_title="Period")
fig.show()

Check the mean

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=3)

for per in range(6):
  row = (per // 3) + 1
  col = (per % 3) + 1
  meanSI = np.nanmean(si_lists[per])
  fig.add_trace(
      go.Histogram(x=si_lists[per],
                   name=f"Period {per+1}: mean={round(meanSI, 2)}"),
      row=row, col=col
  )

fig.update_layout(height=row*300, width=900,
                  title_text=f"Distribution of Stringency Index's mean for each county for all the periods",
                  xaxis_title=f"Average stringency index",
                  yaxis_title="Count",
                  legend_title="Period")
fig.show()

### Election Results
[Presidential precinct data for the 2020 general election](https://github.com/TheUpshot/presidential-precinct-map-2020)

In [ ]:
# Load the preprocessed csv (original data from precincts-with-results.geojson.gz)
# custom preprocessing script process_election_data.py
# total population of each county extracted from ACS
political_data = pd.read_csv("political_leaning.csv")
political_data.index = political_data["Unnamed: 0"]
political_data.drop(columns=["Unnamed: 0"], inplace=True)

# change columns name to remove 0s from the beginning of the FIPS
political_data.columns = [str(int(x)) for x in list(political_data.columns)]
political_data

Update county information dataframe

In [ ]:
# Update county database
political_leaning_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    political_leaning = political_data.loc["political_leaning"][fips]
  except Exception as e:
    political_leaning = np.nan
  political_leaning_list.append(political_leaning)
jhu_data["political_leaning"] = political_leaning_list
jhu_data

### American Communities Project
[American Communities Project](https://www.americancommunities.org/#map)

In [ ]:
# American Communities Project (ACP) typology by county
county_df = pd.read_excel("County-Type-Share.xlsx")

county_info = county_df[["Key", "New Names"]].dropna()
county_info["Key"] = county_info["Key"].astype(int)
county_df = county_df[["County", "Type Number", "FIPS"]]
description_county = {"Exurbs": "Wealthy, well-educated, largely white. Low crime rates & longer commutes.",
                      "Graying America": "Large senior population. Middle-income, low diversity, avg. education.",
                      "African American South": "Median roughly 40% African American. Low income, high unemployment.",
                      "Evangelical Hubs": "Many evangelical adherents. Fewer college grads & healthcare providers.",
                      "Working Class Country": "Highly educated, low diversity. Young with high turnover in population.",
                      "Military Posts": "Middle-income, diverse communities around military bases.",
                      "Urban Suburbs": "Educated and densely populated. Racially and economically diverse.",
                      "Hispanic Centers": "Heavily Hispanic & rural. Young, lower incomes, limited healthcare access.",
                      "Native American Lands": "Large Native American population. Young, low income, many uninsured.",
                      "Rural Middle America": "Largely rural and white. Middle-income, avg. college grad numbers.",
                      "College Towns": "Highly educated, low diversity. Young with high turnover in population.",
                      "LDS Enclaves": "Large youth population. Middle-income, low diversity, low crime.",
                      "Aging Farmlands": "Agricultural with many seniors. Little diversity, low unemployment.",
                      "Big Cities": "Dense & diverse. High incomes & high poverty. High crime rates.",
                      "Middle Suburbs": "Middle-income, low diversity, and below average for college education."}
county_info["County Description"] = county_info["New Names"].apply(lambda x: description_county[x])

In [ ]:
county_info.head()

In [ ]:
county_info_dic = {}
for i in range(len(county_info)):
  county_info_dic[county_info["Key"].iloc[i]] = county_info["New Names"].iloc[i]
county_info_dic

In [ ]:
county_df.head()

Update county database

In [ ]:
acp_list = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    acp = county_df[county_df["FIPS"] == int(fips)]["Type Number"].iloc[0]
  except Exception as e:
    acp = np.nan
  acp_list.append(acp)
jhu_data["acp"] = acp_list
jhu_data

In [ ]:
# Features with name
import math
jhu_data["acp_name"] = jhu_data["acp"].apply(lambda x: county_info_dic[x] if not(math.isnan(x)) else "Unknown")
jhu_data["acp_name_with_number"] = jhu_data["acp"].apply(lambda x: str(int(x)) + " " + str(county_info_dic[x]) if not(math.isnan(x)) else "Unknown")

### American Community Survey
[American Community Survey 2014-2018 5-Year Estimates Now Available](https://www.census.gov/newsroom/press-releases/2019/acs-5-year.html), more precisely [here](https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_comparison_profiles/county/) and for the areas [here](https://www.census.gov/library/publications/2011/compendia/usa-counties-2011.html#LND)

~7min runtime

In [ ]:
# race/ethnic composition
pct_white_l = []
pct_black_l = []
pct_asian_l = []
pct_hispanic_l = []
pct_other_l = []
# education
education_l = []
# age
under_19_l = []
over_65_l = []
# income: median household income
income_l = []
# pop densisty
total_pop_l = []
county_area_l = []
# household crowding: more occupants than rooms
hc_l = []

# load the files
cp05 = pd.read_csv("American_Community_Survey_-_United_States_Census/cp05.csv", encoding="latin")
cp05["county_id"] = cp05["GEOID"].apply(lambda x: str(int(x.split("US")[1])))
cp04 = pd.read_csv("American_Community_Survey_-_United_States_Census/cp04.csv", encoding="latin")
cp04["county_id"] = cp04["GEOID"].apply(lambda x: str(int(x.split("US")[1])))
cp03 = pd.read_csv("American_Community_Survey_-_United_States_Census/cp03.csv", encoding="latin")
cp03["county_id"] = cp03["GEOID"].apply(lambda x: str(int(x.split("US")[1])))
cp02 = pd.read_csv("American_Community_Survey_-_United_States_Census/cp02.csv", encoding="latin")
cp02["county_id"] = cp02["GEOID"].apply(lambda x: str(int(x.split("US")[1])))
# county land area data: process it
land_df = pd.read_excel("American_Community_Survey_-_United_States_Census/LND01.xls")
column_names = list(land_df.columns)
metadata_land = pd.read_excel("American_Community_Survey_-_United_States_Census/Mastdata.xls")
for i, v in enumerate(column_names):
  sub_df = metadata_land[metadata_land["Item_Id"] == v]
  if not(sub_df.empty):  # change the name of the column to a more comprehensible one
    column_names[i] = sub_df["Item_Description"].iloc[0]
land_df.columns = column_names  # update columns names
land_df["STCOU"] = land_df["STCOU"].astype(str)

# useful function to fetch data
def _fetch_data(df, title):
  result = df[df["TITLE"] == title]["EST_1418"].iloc[0]
  if "(X)" in result:
    result = np.nan
  else:
    result = float(result.replace(",", ""))
  return result

# retrieve the information
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]

  # land area
  county_area_df = land_df[land_df["STCOU"] == fips]
  if county_area_df.empty:
    county_area = np.nan
  else:
    county_area = float(county_area_df["Land area in square miles 2000"].iloc[0])

  # other info
  cp05_county = cp05[cp05["county_id"] == fips]
  if cp05_county.empty:
    pct_white = np.nan
    pct_black = np.nan
    pct_asian = np.nan
    pct_hispanic = np.nan
    pct_other = np.nan
    total_pop = np.nan
    under_19 = np.nan
    over_65 = np.nan
  else:
    pct_white = _fetch_data(cp05_county, "White")
    pct_black = _fetch_data(cp05_county, "Black or African American")
    pct_asian = _fetch_data(cp05_county, "Asian")
    pct_hispanic = _fetch_data(cp05_county, "Hispanic or Latino (of any race)")
    pct_other = _fetch_data(cp05_county, "Some other race")
    total_pop = _fetch_data(cp05_county, "Total population")
    under_19 = _fetch_data(cp05_county, "Under 18 years")
    over_65 = _fetch_data(cp05_county, "65 years and over")

  cp04_county = cp04[cp04["county_id"] == fips]
  if cp04_county.empty:
    hc = np.nan
  else:
    hc = _fetch_data(cp04_county, "1.00 or less")
    if not(np.isnan(hc)):
      hc = 100 - hc

  cp03_county = cp03[cp03["county_id"] == fips]
  if cp03_county.empty:
    income = np.nan
  else:
    income = _fetch_data(cp03_county, "Median household income (dollars)")
  
  cp02_county = cp02[cp02["county_id"] == fips]
  if cp02_county.empty:
    education = np.nan
  else:
    education = _fetch_data(cp02_county, "Percent high school graduate or higher")

  pct_white_l.append(pct_white)
  pct_black_l.append(pct_black)
  pct_asian_l.append(pct_asian)
  pct_hispanic_l.append(pct_hispanic)
  pct_other_l.append(pct_other)
  education_l.append(education)
  under_19_l.append(under_19)
  over_65_l.append(over_65)
  income_l.append(income)
  total_pop_l.append(total_pop)
  county_area_l.append(county_area)
  hc_l.append(hc)

jhu_data["pch_white"] = pct_white_l
jhu_data["pct_black"] = pct_black_l
jhu_data["pct_asian"] = pct_asian_l
jhu_data["pct_hispanic"] = pct_hispanic_l
jhu_data["pct_other"] = pct_other_l
jhu_data["education"] = education_l
jhu_data["under_19"] = under_19_l
jhu_data["over_65"] = over_65_l
jhu_data["income"] = income_l
jhu_data["total_pop"] = total_pop_l
jhu_data["county_area"] = county_area_l
jhu_data["hc"] = hc_l
jhu_data["pop_density"] = jhu_data["total_pop"] / jhu_data["county_area"]
# jhu_data.drop(columns=["total_pop", "county_area"], inplace=True)  # calculus artifacts
jhu_data

### VERA Institute
[Incarceration Trends Dataset](https://github.com/vera-institute/incarceration-trends)

Total jail population is defined as the average daily number of people held in jail through December 31 of a given year.

In [ ]:
vera_data = pd.read_csv("incarceration_trends.csv")
vera_data["fips"] = vera_data["fips"].astype(str)
vera_data = vera_data[vera_data["year"] == 2018]  # filter only last year of data (2018)
vera_data

Add VERA jail data to database

In [ ]:
tot_jail_pop_l = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  vera_subdf = vera_data[vera_data["fips"] == fips]
  if vera_subdf.empty:
    tot_jail_pop = np.nan
  else:
    tot_jail_pop = float(vera_subdf["total_jail_pop"].iloc[0])
  tot_jail_pop_l.append(tot_jail_pop)
jhu_data["total_jail_pop"] = tot_jail_pop_l
jhu_data["total_jail_pop"]

### Center for Medicare and Medicaid Services
[COVID-19 Nursing Home Data](https://data.cms.gov/covid-19/covid-19-nursing-home-data)

In [ ]:
"""
!unzip "Center_for_Medicare_and_Medicaid_Services/faclevel_2020.zip" -d "Center_for_Medicare_and_Medicaid_Services"
"""

In [ ]:
nursing_data = pd.read_csv("Center_for_Medicare_and_Medicaid_Services/faclevel_2020.csv")
# Calculate average number of occupied bed per county over the year 2020
nursing_data = nursing_data.groupby(by=["County", "Provider Name"])["Total Number of Occupied Beds"].mean().groupby(by="County").sum()
nursing_data

In [ ]:
pairs_fips_countyName = vera_data[["fips", "county_name"]]
pairs_fips_countyName["county_name"] = pairs_fips_countyName["county_name"].apply(lambda x: x.replace(" County", ""))
nursing_pop_l = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    cty_name = pairs_fips_countyName[pairs_fips_countyName["fips"] == fips]["county_name"].iloc[0]
    nursing_pop = nursing_data.loc[cty_name]
  except Exception as e:
    nursing_pop = np.nan
  nursing_pop_l.append(nursing_pop)
jhu_data["nursing_population"] = nursing_pop_l
jhu_data["nursing_population"]

### PLACES dataset
[PLACES: Local Data for Better Health, County Data 2021 release500 Cities & Places](https://chronicdata.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/swc5-untb) & [2020](https://chronicdata.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/dv4u-3x3q)

In [ ]:
places = pd.read_csv("PLACES/PLACES__Local_Data_for_Better_Health__County_Data_2021_release.csv")  # 2021 release contains year 2019 data
places = places[places["Measure"] == "Obesity among adults aged >=18 years"]
places = places[places["Year"] == 2019]
places

In [ ]:
obesity_data_l = []
county_name_l = []
for i in range(len(jhu_data)):
  fips = jhu_data["FIPS"].iloc[i]
  try:
    cty_name = pairs_fips_countyName[pairs_fips_countyName["fips"] == fips]["county_name"].iloc[0]
    obesity_data = float(places[places["LocationName"] == cty_name]["Data_Value"].iloc[0])
  except Exception as e:
    cty_name = np.nan
    obesity_data = np.nan
  obesity_data_l.append(obesity_data)
  county_name_l.append(cty_name)
jhu_data["obesity"] = obesity_data_l
jhu_data["county_name"] = county_name_l
jhu_data["obesity"]

### Nationwide Wastewater Monitoring Network

[Data repository for Biobot Analytics Nationwide Wastewater Monitoring Network](https://github.com/biobotanalytics/covid19-wastewater-data)

In [ ]:
biobot = pd.read_csv("wastewater_by_county.csv")
biobot["sampling_week"] = pd.to_datetime(biobot["sampling_week"])
biobot.drop(columns=["Unnamed: 0"], inplace=True)
biobot

In [ ]:
# Date filter, between 2020/01/01 and max_date

max_date_list = ["2020-05-31", "2020-10-31", "2021-02-28", "2021-07-31",
                 "2021-10-31", "2022-02-28"]
min_date_list = ["2020-02-01", "2020-05-31", "2020-10-31", "2021-02-28",
                 "2021-07-31", "2021-10-31"]
max_date = pd.to_datetime("2021-02-01")

# ecra = effective_concentration_rolling_average
ecra_lists = []  # for plots after
std_lists = []  # for plots after

for per in range(6):
  min_date = pd.to_datetime(min_date_list[per])
  max_date = pd.to_datetime(max_date_list[per])
  biobot2 = biobot[biobot["sampling_week"] <= max_date]
  biobot2 = biobot2[biobot2["sampling_week"] > min_date]

  # group by county
  biobot2 = biobot2.groupby(by="fipscode")["effective_concentration_rolling_average"]
  # extract features and back to dataframe format
  biobot2_mean = biobot2.mean().to_frame()  # calculate the mean of the effective_concentration_rolling_average over the study period
  biobot2_median = biobot2.median().to_frame()
  biobot2_min = biobot2.min().to_frame()
  biobot2_max = biobot2.max().to_frame()
  std_df = biobot2.std().to_frame()

  # Update county database

  ecra_list = []  # mean
  ecra_median_list = []
  ecra_min_list = []
  ecra_max_list = []
  std_list = []
  for i in range(len(jhu_data)):
    try:
      fips = jhu_data["FIPS"].iloc[i]
      fips = int(fips)  # convert to int type (type of biobot fips code)
      ecra_mean = biobot2_mean.loc[fips].iloc[0]
      ecra_median = biobot2_median.loc[fips].iloc[0]
      ecra_min = biobot2_min.loc[fips].iloc[0]
      ecra_max = biobot2_max.loc[fips].iloc[0]
      std = std_df.loc[fips].iloc[0]
    except Exception as e:
      ecra_mean = np.nan
      ecra_median = np.nan
      ecra_min = np.nan
      ecra_max = np.nan
      std = np.nan
    ecra_list.append(ecra_mean)
    ecra_median_list.append(ecra_median)
    ecra_min_list.append(ecra_min)
    ecra_max_list.append(ecra_max)
    std_list.append(std)
  
  ecra_lists.append(ecra_list)
  std_lists.append(std_list)

  jhu_data[f"ecra{per+1}_mean"] = ecra_list
  jhu_data[f"ecra{per+1}_median"] = ecra_median_list
  jhu_data[f"ecra{per+1}_min"] = ecra_min_list
  jhu_data[f"ecra{per+1}_max"] = ecra_max_list
  jhu_data[f"ecra{per+1}_std"] = std_list
jhu_data

## Optional import (not implemented)

### Delphi Epidata
[a]()

### Google Mobility
[a]()

### The COVID Tracking Project
[a]()

## Create county database

In [ ]:
# Rename our county-level database and remove the county with a nan FIPS
county_database = jhu_data[jhu_data["FIPS"] != "nan"]

# Add some log scale columns & percentage columns
county_database["log_pop_density"] = np.log(county_database["pop_density"])
county_database["log_crowding"] = np.log(county_database["hc"])
county_database["log_crowding"] = county_database["log_crowding"].replace(-np.inf, -10)  # -inf crowding replaced by -10 for fitting models
county_database["pct_nursing"] = county_database["nursing_population"] / county_database["total_pop"]
county_database["pct_jail"] = county_database["total_jail_pop"] / county_database["total_pop"]

# Change index
county_database.index = county_database.FIPS

# Replace inf values by nan
county_database = county_database.replace(np.inf, np.nan)
county_database = county_database.replace(-np.inf, np.nan)

## Load additional data

### [Risk factors for increased COVID-19 case-fatality in the United States: A county-level analysis during the first wave](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0258308)

GitHub [link](https://github.com/jmillar201/COVID19_CFR)

Features are imported a csv file created with the custom script rds_file_to_csv.R (drop the column geometry that contains lists and convert the dataframe from an rds file to a csv file)

In [ ]:
df_alt = pd.read_csv("Alt_data/county_indicators_2020-06-14.csv", sep=",", engine="python")
df_alt = df_alt[~np.isnan(df_alt["FIPS"])]  # drop counties without fips
df_alt["FIPS"] = df_alt["FIPS"].apply(lambda x: str(int(x)))  # remove trailing zeros on fips
df_alt

In [ ]:
# Add data: SVI (social vulnerability index), healthcare accessibility, comorbities

# npi_keystone too many nan values
columns_to_keep = ["FIPS", "sv_groupquarterpop",
                   'sv_pdisability', 'sv_singleparent',
                   'sv_penglish',
                   'sv_pmultiunit', 'sv_pmobilehome', 'ses_ppoverty',
                   'ses_punemployed',
                   'hc_hospitals_per1000', 'hc_icubeds_per1000',
                   'hc_icubeds_per60more1000', 'hc_icubeds_per65more1000',
                   'hc_pnotinsured_acs',
                   'hc_primarycare_per1000',
                   'como_medicareheartdizprev', 'hc_medicaid',
                   'como_pdiabetes', 'como_htn_hosp', 'como_htn_mort',
                   'como_cvd_hosp',
                   'como_cvd_mort', 'como_allheartdis_hosp', 'allheartdis_mort',
                   'como_stroke_hosp', 'como_stroke_mort', 'como_smoking',
                   'como_COPD', 'como_asthma', 'como_cancer5yr']
df_alt = df_alt[columns_to_keep]
df_alt.index = df_alt.FIPS

## Clustering (to determine periods)

### Load timeseries

In [ ]:
df = pd.read_csv("COVID-19_timeseries.csv")
df.index = df["Unnamed: 0"]
col = list(df.columns)
col[0] = "FIPS"
df.columns = col
df.drop(columns=["FIPS"], inplace=True)

### February 2020 - February 2021: 3 periods clustering (old)

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
A = df[df.index <= pd.to_datetime("2021-02-28")]  # not transposed like in the article!
A = A.fillna(0)  # let's suppose that no data = no death
A_zscore = (A - A.mean(axis=0)) / A.std(axis=0)
A_zscore

In [ ]:
# Drop county with no values (only nan and or 0 death)
county_to_drop = []
for fips in list(A_zscore.columns):
  if np.isnan(A_zscore[fips]).sum() == len(A_zscore):
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
A_zscore.drop(columns=county_to_drop, inplace=True)
A_zscore = A_zscore.T  # transpose the matrix: for clustering
A_zscore

Elbow plot (WCSS: sum of the distances between each county and its assigned cluster)

In [ ]:
from sklearn.cluster import KMeans

wcss_list = []
for k in range(1, 20):
  kmeans = KMeans(n_clusters=k, random_state=42).fit(A_zscore)
  wcss_list.append(kmeans.inertia_)
plt.plot([i for i in range(1, 20)], wcss_list, "o-", color="blue")
plt.xlabel("Number of clusters")
plt.ylabel("Distance")
plt.title("Elbow plot for k-means clustering")

Choose k=3

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(A_zscore)
labels = kmeans.labels_
labels_dic = {}
for i in range(len(labels)):
  labels_dic[A_zscore.index[i]] = labels[i]

In [ ]:
cluster_list = []
for i in range(len(county_database)):
  fips = county_database["FIPS"].iloc[i]
  try:
    cluster = labels_dic[fips]
  except Exception as e:
    cluster = np.nan
  cluster_list.append(cluster)
county_database["cluster"] = cluster_list
county_database["cluster"]

In [ ]:
color_continuous_scale = "agsunset"
label_col = "cluster"
label_display = "Counties by Cluster"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

Representative Cluster COVID-19 Dynamics (deaths)

In [ ]:
dates = list(A_zscore.columns)
color_list = [(79/255, 41/255, 146/255),
              (235/255, 83/255, 134/255),
              (237/255, 217/255, 163/255)]
for i in range(kmeans.n_clusters):
  plt.plot(dates, kmeans.cluster_centers_[i], c=color_list[i])

plt.xlabel("Date")
plt.ylabel("Deaths (z-score)")
plt.title("Representative Cluster COVID-19 Dynamics (deaths)")

### March 2021 - February 2022

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
A = df[(df.index > pd.to_datetime("2021-02-28")) & (df.index <= pd.to_datetime("2022-02-28"))]  # not transposed like in the article!
A = A.fillna(0)  # let's suppose that no data = no death
A_zscore = (A - A.mean(axis=0)) / A.std(axis=0)
A_zscore

In [ ]:
# Drop county with no values (only nan and or 0 death)
county_to_drop = []
for fips in list(A_zscore.columns):
  if np.isnan(A_zscore[fips]).sum() == len(A_zscore):
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
A_zscore.drop(columns=county_to_drop, inplace=True)
A_zscore = A_zscore.T  # transpose the matrix: for clustering
A_zscore

In [ ]:
from sklearn.cluster import KMeans

wcss_list = []
for k in range(1, 20):
  kmeans = KMeans(n_clusters=k, random_state=42).fit(A_zscore)
  wcss_list.append(kmeans.inertia_)
plt.plot([i for i in range(1, 20)], wcss_list, "o-", color="blue")
plt.xlabel("Number of clusters")
plt.ylabel("Distance")
plt.title("Elbow plot for k-means clustering")

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(A_zscore)
labels = kmeans.labels_
labels_dic = {}
for i in range(len(labels)):
  labels_dic[A_zscore.index[i]] = labels[i]

In [ ]:
dates = list(A_zscore.columns)
color_list = [(79/255, 41/255, 146/255),
              (235/255, 83/255, 134/255),
              (237/255, 217/255, 163/255)]
for i in range(kmeans.n_clusters):
  plt.plot(dates, kmeans.cluster_centers_[i], c=color_list[i])

plt.xlabel("Date")
plt.ylabel("Deaths (z-score)")
plt.title("Representative Cluster COVID-19 Dynamics (deaths)")

### February 2020 - February 2022

In [ ]:
df.index = pd.to_datetime(df.index)
A = df[df.index <= pd.to_datetime("2022-02-28")]  # not transposed like in the article!
A = A.fillna(0)  # let's suppose that no data = no death
A_zscore = (A - A.mean(axis=0)) / A.std(axis=0)
A_zscore

In [ ]:
# Drop county with no values (only nan and or 0 death)
county_to_drop = []
for fips in list(A_zscore.columns):
  if np.isnan(A_zscore[fips]).sum() == len(A_zscore):
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
A_zscore.drop(columns=county_to_drop, inplace=True)
A_zscore = A_zscore.T  # transpose the matrix: for clustering
A_zscore

In [ ]:
from sklearn.cluster import KMeans

wcss_list = []
for k in range(1, 20):
  kmeans = KMeans(n_clusters=k, random_state=42).fit(A_zscore)
  wcss_list.append(kmeans.inertia_)
plt.plot([i for i in range(1, 20)], wcss_list, "o-", color="blue")
plt.xlabel("Number of clusters")
plt.ylabel("Distance")
plt.title("Elbow plot for k-means clustering")

Let's choose k=3

In [ ]:
n_clusters = 3

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(A_zscore)
labels = kmeans.labels_
labels_dic = {}
for i in range(len(labels)):
  labels_dic[int(A_zscore.index[i])] = labels[i]

In [ ]:
cluster_list = []
for i in range(len(county_database)):
  fips = county_database["FIPS"].iloc[i]
  try:
    cluster = labels_dic[int(fips)]
  except Exception as e:
    cluster = np.nan
  cluster_list.append(cluster)
county_database["cluster"] = cluster_list
plt.hist(county_database["cluster"])
plt.title("Histogram of the number of counties per cluster")
plt.xlabel("Cluster")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# create_custom_choropleth is defined in the section Plot, subsection Initialization

color_continuous_scale = "agsunset"
label_col = "cluster"
label_display = "Counties by Cluster"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

In [ ]:
dates = list(A_zscore.columns)
nb_clusters = kmeans.n_clusters
color_list = [(((nb_clusters - 1 - k) * 79 + k * 237)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 41 + k * 217)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 146 + k * 163)/((nb_clusters - 1) * 255)) for k in range(nb_clusters)]
fig = plt.figure(figsize=(24, 10))
for i in range(nb_clusters):
  plt.plot(dates, kmeans.cluster_centers_[i], c=color_list[i])

plt.xlabel("Date")
plt.ylabel("Deaths (z-score)")
plt.title("Representative Cluster COVID-19 Dynamics (deaths)")

### February 2020 - February 2022 (DBSCAN)

In [ ]:
df.index = pd.to_datetime(df.index)
A = df[df.index <= pd.to_datetime("2022-02-28")]  # not transposed like in the article!
A = A.fillna(0)  # let's suppose that no data = no death
A_zscore = (A - A.mean(axis=0)) / A.std(axis=0)

In [ ]:
# Drop county with no values (only nan and or 0 death)
county_to_drop = []
for fips in list(A_zscore.columns):
  if np.isnan(A_zscore[fips]).sum() == len(A_zscore):
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
A_zscore.drop(columns=county_to_drop, inplace=True)
A_zscore = A_zscore.T  # transpose the matrix: for clustering

In [ ]:
from sklearn.cluster import DBSCAN

nb_tot_clusters = []
not_clustered = []
for eps in np.linspace(0.0001, 5, 3):
  dbscan = DBSCAN(eps=eps, min_samples=10, n_jobs=-1).fit(A_zscore)
  nb_tot_clusters.append(len(np.unique(dbscan.labels_)) - 1)
  not_clustered.append(100 * np.count_nonzero(dbscan.labels_==-1) / len(dbscan.labels_))

# plot
fig, axs = plt.subplots(2)
fig.suptitle("Hyperparameters tuning plot for DBSCAN clustering")
axs[0].plot(np.linspace(0.0001, 5, 100), nb_tot_clusters, "o-", color="blue")
axs[0].set(xlabel="Epsilon", ylabel="Number of clusters")
axs[1].plot(np.linspace(0.0001, 5, 100), not_clustered, "o-", color="blue")
axs[1].set(xlabel="Epsilon", ylabel="Points not clustered (%)")

# Zoom for eps between 3 and 4
nb_tot_clusters = []
not_clustered = []
for eps in np.linspace(3, 4, 100):
  dbscan = DBSCAN(eps=eps, min_samples=10, n_jobs=-1).fit(A_zscore)
  nb_tot_clusters.append(len(np.unique(dbscan.labels_)) - 1)
  not_clustered.append(100 * np.count_nonzero(dbscan.labels_==-1) / len(dbscan.labels_))

# plot
fig, axs = plt.subplots(2)
fig.suptitle("Hyperparameters tuning plot for DBSCAN clustering")
axs[0].plot(np.linspace(3, 4, 100), nb_tot_clusters, "o-", color="blue")
axs[0].set(xlabel="Epsilon", ylabel="Number of clusters")
axs[1].plot(np.linspace(3, 4, 100), not_clustered, "o-", color="blue")
axs[1].set(xlabel="Epsilon", ylabel="Points not clustered (%)")

Let's choose $\epsilon = 3.3$ to obtain 3 clusters with less than 15% of unlabeled counties

In [ ]:
from sklearn.cluster import DBSCAN

eps = 3.3
dbscan = DBSCAN(eps=eps, min_samples=10, n_jobs=-1).fit(A_zscore)
labels = dbscan.labels_
labels_dic = {}
for i in range(len(labels)):
  labels_dic[int(A_zscore.index[i])] = labels[i]

In [ ]:
cluster_list = []
for i in range(len(county_database)):
  fips = county_database["FIPS"].iloc[i]
  try:
    cluster = labels_dic[fips]
  except Exception as e:
    cluster = np.nan
  cluster_list.append(cluster)
county_database["cluster_dbscan"] = cluster_list
plt.hist(county_database["cluster_dbscan"])
plt.title("Histogram of the number of counties per cluster")
plt.xlabel("Cluster")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
color_continuous_scale = "agsunset"
label_col = "cluster_dbscan"
label_display = "Counties by Cluster"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

In [ ]:
dates = list(A_zscore.columns)
nb_clusters = dbscan.n_clusters
color_list = [(((nb_clusters - 1 - k) * 79 + k * 237)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 41 + k * 217)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 146 + k * 163)/((nb_clusters - 1) * 255)) for k in range(nb_clusters)]
fig = plt.figure(figsize=(24, 10))
for i in range(nb_clusters):
  plt.plot(dates, kmeans.cluster_centers_[i], c=color_list[i])

plt.xlabel("Date")
plt.ylabel("Deaths (z-score)")
plt.title("Representative Cluster COVID-19 Dynamics (deaths)")

### February 2020 - February 2022 (DTW)

In [ ]:
df.index = pd.to_datetime(df.index)
A = df[df.index <= pd.to_datetime("2022-02-28")]  # not transposed like in the article!
A = A.fillna(0)  # let's suppose that no data = no death
A_zscore = (A - A.mean(axis=0)) / A.std(axis=0)

# Drop county with no values (only nan and or 0 death)
county_to_drop = []
for fips in list(A_zscore.columns):
  if np.isnan(A_zscore[fips]).sum() == len(A_zscore):
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
A_zscore.drop(columns=county_to_drop, inplace=True)
A_zscore = A_zscore.T  # transpose the matrix: for clustering

In [ ]:
A_zscore_save = A_zscore.copy()
A_zscore = A_zscore[:10]

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

wcss_list = []
for k in range(1, 20):
  kmeans = TimeSeriesKMeans(n_clusters=3, metric="dtw",
                            max_iter=10, random_state=42).fit(A_zscore)
  wcss_list.append(kmeans.inertia_)
plt.plot([i for i in range(1, 20)], wcss_list, "o-", color="blue")
plt.xlabel("Number of clusters")
plt.ylabel("Distance")
plt.title("Elbow plot for timeseries k-means clustering with DTW")

In [ ]:
n_clusters = 3

from tslearn.clustering import TimeSeriesKMeans
kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw",
                          max_iter=10, random_state=42).fit(A_zscore)
labels = kmeans.labels_
labels_dic = {}
for i in range(len(labels)):
  labels_dic[int(A_zscore.index[i])] = labels[i]

In [ ]:
cluster_list = []
for i in range(len(county_database)):
  fips = county_database["FIPS"].iloc[i]
  try:
    cluster = labels_dic[fips]
  except Exception as e:
    cluster = np.nan
  cluster_list.append(cluster)
county_database["cluster_dtw"] = cluster_list
plt.hist(county_database["cluster_dtw"])
plt.title("Histogram of the number of counties per cluster")
plt.xlabel("Cluster")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
color_continuous_scale = "agsunset"
label_col = "cluster_dtw"
label_display = "Counties by Cluster"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

In [ ]:
dates = list(A_zscore.columns)
nb_clusters = kmeans.n_clusters
color_list = [(((nb_clusters - 1 - k) * 79 + k * 237)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 41 + k * 217)/((nb_clusters - 1) * 255),
               ((nb_clusters - 1 - k) * 146 + k * 163)/((nb_clusters - 1) * 255)) for k in range(nb_clusters)]
fig = plt.figure(figsize=(24, 10))
for i in range(nb_clusters):
  plt.plot(dates, kmeans.cluster_centers_[i], c=color_list[i])

plt.xlabel("Date")
plt.ylabel("Deaths (z-score)")
plt.title("Representative Cluster COVID-19 Dynamics (deaths)")

## Add death counts and crude death rates per period (per 100k residents)

In [ ]:
df.index = pd.to_datetime(df.index)
period1 = df[df.index <= pd.to_datetime("2020-05-31")]
period2 = df[df.index <= pd.to_datetime("2020-10-31")]
period2 = period2[period2.index > pd.to_datetime("2020-05-31")]
period3 = df[df.index > pd.to_datetime("2020-10-31")]
period3 = period3[period3.index <= pd.to_datetime("2021-02-28")]
period4 = df[df.index > pd.to_datetime("2021-02-28")]
period4 = period4[period4.index <= pd.to_datetime("2021-07-31")]
period5 = df[df.index > pd.to_datetime("2021-07-31")]
period5 = period5[period5.index <= pd.to_datetime("2021-10-31")]
period6 = df[df.index > pd.to_datetime("2021-10-31")]
period6 = period6[period6.index <= pd.to_datetime("2022-02-28")]

# death rates (covid-19 related)
deathCount1_l = []
deathCount2_l = []
deathCount3_l = []
deathCount4_l = []
deathCount5_l = []
deathCount6_l = []
for i in range(len(county_database)):
  fips = county_database["FIPS"].iloc[i]
  deathCount1_l.append(period1[fips].iloc[-1])
  deathCount2_l.append(period2[fips].iloc[-1])
  deathCount3_l.append(period3[fips].iloc[-1])
  deathCount4_l.append(period4[fips].iloc[-1])
  deathCount5_l.append(period5[fips].iloc[-1])
  deathCount6_l.append(period6[fips].iloc[-1])
factor_increase = 100_000  # per 100k residents
county_database["deathCount_period1"] = deathCount1_l
county_database["deathRate_period1"] = county_database["deathCount_period1"] * factor_increase / county_database["total_pop"]
county_database["deathCount_period2"] = deathCount2_l
county_database["deathRate_period2"] = county_database["deathCount_period2"] * factor_increase / county_database["total_pop"]
county_database["deathCount_period3"] = deathCount3_l
county_database["deathRate_period3"] = county_database["deathCount_period3"] * factor_increase / county_database["total_pop"]
county_database["deathCount_period4"] = deathCount4_l
county_database["deathRate_period4"] = county_database["deathCount_period4"] * factor_increase / county_database["total_pop"]
county_database["deathCount_period5"] = deathCount5_l
county_database["deathRate_period5"] = county_database["deathCount_period5"] * factor_increase / county_database["total_pop"]
county_database["deathCount_period6"] = deathCount6_l
county_database["deathRate_period6"] = county_database["deathCount_period6"] * factor_increase / county_database["total_pop"]

## Create county database 2 (with the additional data) and the imputed version at the state level version of county database 2

File Data-Cleaning_2020-06-14.html useful & website [socialexplorer](https://www.socialexplorer.com/data/ACS2015/metadata/?ds=ACS15&var=B25024010)

In [ ]:
county_database["FIPS"] = county_database["FIPS"].apply(lambda x: int(x))
county_database.index = county_database["FIPS"]
df_alt["FIPS"] = df_alt["FIPS"].apply(lambda x: int(x))
df_alt.index = df_alt["FIPS"]

In [ ]:
county_database.index = county_database.FIPS
if "FIPS.1" in county_database.columns:
  county_database.drop(columns=["FIPS.1"], inplace=True)
county_database2 = county_database.join(df_alt.drop(columns=["FIPS"]))
# county_database2.describe().loc["count"]

In [ ]:
for c in ['como_medicareheartdizprev', 'hc_medicaid',
          'como_pdiabetes', 'como_htn_hosp', 'como_htn_mort',
          'como_cvd_hosp', 'como_cvd_mort', 'como_allheartdis_hosp',
          'allheartdis_mort', 'como_stroke_hosp', 'como_stroke_mort',
          'como_smoking', 'como_COPD', 'como_asthma', 'como_cancer5yr',
          "sv_groupquarterpop"]:
  county_database2[c] = county_database2[c] / county_database2["total_pop"]

### Impute data at the state level

In [ ]:
for c in county_database2.columns:
  county_database2[c] = county_database2[c].replace(np.inf, np.nan)
  county_database2[c] = county_database2[c].replace(-np.inf, np.nan)
if "FIPS.1" in county_database2.columns:
  county_database2.drop(columns=["FIPS.1"], inplace=True)
columns_to_fill = list(county_database2.columns)
for c in ["FIPS", "Province_State", "closest_airport", "county_name",
          "acp_name", "acp_name_with_number"]:
  columns_to_fill.remove(c)

In [ ]:
# fit an imputer per state
from sklearn.impute import SimpleImputer
county_database2_imputed = county_database2.copy()
for state in np.unique(county_database2_imputed["Province_State"]):
  statedf = county_database2_imputed[county_database2_imputed["Province_State"] == state]  # state sub dataframe
  columns_to_fill_state = []
  for x in columns_to_fill:
    if len(statedf[x].dropna()) != 0:  # at least one value in this column to fit an imputer
      columns_to_fill_state.append(x)
  statedf_to_fill = statedf[columns_to_fill_state]  # state sub dataframe with the columns to fill
  impute = SimpleImputer(strategy='mean')
  statedf_to_fill = pd.DataFrame(impute.fit_transform(statedf_to_fill),
                                 columns=statedf_to_fill.columns,
                                 index=statedf_to_fill.index)
  statedf[columns_to_fill_state] = statedf_to_fill  # replace the filled columns
  county_database2_imputed[county_database2_imputed["Province_State"] == state] = statedf  # update the database for this state
county_database2_imputed

## Save the datasets (for this response variable)

In [ ]:
# TO EDIT: in function of the chosen response variable
type_data_filename =  # covid19, all_causes, excess_mortality

In [ ]:
county_database.to_csv(f"county_database_{type_data_filename}.csv")
county_database2.to_csv(f"county_database2_{type_data_filename}.csv")
county_database2_imputed.to_csv(f"county_database2_imputed_{type_data_filename}.csv")

## Correlations & feature selection

We choose a correlation threshold equal to 0.7 (assumption that a correlation greater than 0.7 is too strong)

In [ ]:
import seaborn as sns


correlation_threshold = 0.7

def correlation_feature_selection(period, county_database2_imputed):

  numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
  dfcorr = county_database2_imputed.select_dtypes(include=numerics)
  # drop features that will be kept whether are not they are correlated to another one
  columns_to_drop = ["FIPS", "Lat", "Long_", "acp", "HHS Region", "total_pop",
                     "county_area"]
  # drop stringency index of period in the future
  stringency_columns = [f"StringencyIndex{i}_{s}" for i in range(period+1, 7) for s in ["mean", "std", "median", "min", "max"]]
  columns_to_drop = columns_to_drop + stringency_columns

  # drop wastewater of period in the future
  biobot_columns = [f"ecra{i}_{s}" for i in range(period+1, 7) for s in ["mean", "std", "median", "min", "max"]]
  columns_to_drop = columns_to_drop + biobot_columns

  if period != 1:  # drop distance to airport
    columns_to_drop = columns_to_drop + ["min_distance_top_airport"]

  for per in range(1, 7):
    columns_to_drop.append(f"deathCount_period{per}")
    columns_to_drop.append(f"deathRate_period{per}")
  dfcorr.drop(columns=columns_to_drop, inplace=True)
  dfcorr = dfcorr.corr()

  columns = np.full((dfcorr.shape[0],), True, dtype=bool)
  for i in range(dfcorr.shape[0]):
    if columns[i]:
      correlated_to_i = [i]
      for j in range(i+1, dfcorr.shape[0]):
          if columns[j] and (abs(dfcorr.iloc[i,j]) >= correlation_threshold):
            correlated_to_i.append(j)
      if len(correlated_to_i) > 1:  # at least one correlated variable
        bloc = []
        for k in correlated_to_i:
          corr = 0
          for l in correlated_to_i:
            corr += abs(dfcorr.iloc[k,l])
          corr = corr / len(correlated_to_i)
          bloc.append([k, corr])
        bloc.sort(key=lambda x: x[1])  # we select the feature which is on average, more correlated to the other features in the group
        feature_within_bloc_to_keep = bloc.pop()[0]
        for k, _ in bloc:
          columns[k] = False

  X_selected_columns = list(dfcorr.columns[columns])
  # Add geographical features
  X_selected_columns.append("acp")
  X_selected_columns.append("Lat")
  X_selected_columns.append("Long_")
  X_selected_columns.append("HHS Region")

  selected_columns = X_selected_columns[:]
  # Add total population (after taking the log it will be the sample weight)
  selected_columns.append("total_pop")
  # Add outcome variables
  for per in range(1, 7):
    selected_columns.append(f"deathCount_period{per}")
    selected_columns.append(f"deathRate_period{per}")

  return dfcorr, selected_columns, X_selected_columns

def correlation_feature_selection_all_periods(county_database2_imputed):

  numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
  dfcorr = county_database2_imputed.select_dtypes(include=numerics)
  # drop features that will be kept whether are not they are correlated to another one
  columns_to_drop = ["FIPS", "Lat", "Long_", "acp", "HHS Region", "total_pop",
                     "county_area"]
  
  # don't drop stringency index and min distance airport and biobot

  for per in range(1, 7):
    columns_to_drop.append(f"deathCount_period{per}")
    columns_to_drop.append(f"deathRate_period{per}")
  dfcorr.drop(columns=columns_to_drop, inplace=True)
  dfcorr = dfcorr.corr()

  columns = np.full((dfcorr.shape[0],), True, dtype=bool)
  for i in range(dfcorr.shape[0]):
    if columns[i]:
      correlated_to_i = [i]
      for j in range(i+1, dfcorr.shape[0]):
          if columns[j] and (abs(dfcorr.iloc[i,j]) >= correlation_threshold):
            correlated_to_i.append(j)
      if len(correlated_to_i) > 1:  # at least one correlated variable
        bloc = []
        for k in correlated_to_i:
          corr = 0
          for l in correlated_to_i:
            corr += abs(dfcorr.iloc[k,l])
          corr = corr / len(correlated_to_i)
          bloc.append([k, corr])
        bloc.sort(key=lambda x: x[1])  # we select the feature which is on average, more correlated to the other features in the group
        feature_within_bloc_to_keep = bloc.pop()[0]
        for k, _ in bloc:
          columns[k] = False

  X_selected_columns = list(dfcorr.columns[columns])
  # Add geographical features
  X_selected_columns.append("acp")
  X_selected_columns.append("Lat")
  X_selected_columns.append("Long_")
  X_selected_columns.append("HHS Region")

  selected_columns = X_selected_columns[:]
  # Add total population (after taking the log it will be the sample weight)
  selected_columns.append("total_pop")
  # Add outcome variables
  for per in range(1, 7):
    selected_columns.append(f"deathCount_period{per}")
    selected_columns.append(f"deathRate_period{per}")

  return dfcorr, selected_columns, X_selected_columns

dfcorr_list, selected_columns_list, X_selected_columns_list = [], [], []
for period in range(1, 7):
  dfcorr, selected_columns, X_selected_columns = correlation_feature_selection(period,
                                                                               county_database2_imputed)
  dfcorr_list.append(dfcorr)
  selected_columns_list.append(selected_columns)
  X_selected_columns_list.append(X_selected_columns)

dfcorr, selected_columns, X_selected_columns = correlation_feature_selection_all_periods(county_database2_imputed)
dfcorr_list.append(dfcorr)
selected_columns_list.append(selected_columns)
X_selected_columns_list.append(X_selected_columns)

with open("feature_selection", "wb") as fp:  # save feature selection
  features = [selected_columns_list, X_selected_columns_list]
  pickle.dump(features, fp)

# plot a correlation heatmap
period = 1
fig = plt.figure(figsize=(24, 24))
sns.heatmap(dfcorr_list[period-1], cmap="RdBu")

# Dataset analysis

## Pandas Profiling

In [ ]:
!pip install -U pandas-profiling
!pip install -U pandas-profiling[notebook]

In [ ]:
from pandas_profiling import ProfileReport
# minimal=True for large datasets (remove computational intensive calculations such as correlation matrix, check duplicate rows etc)
profile = ProfileReport(county_database.reset_index(drop=True, inplace=False),
                        minimal=True)

In [ ]:
if not(os.path.exists("pandas-profiling")):
  os.mkdir("pandas-profiling")
profile.to_file("pandas-profiling/report.html")

## Outliers & shapes

In [ ]:
list(county_database2.columns)

### Shapes of the different datasets

In [ ]:
county_database2.index = county_database2.FIPS
dt = county_database2.copy(deep=True)
# transform
dt["total_pop"] = np.log(dt["total_pop"])
print("Initial dataset shape: {}".format(dt.shape))

# drop nan values (we loose approximately 1/3 of the counties with missing data)
dt.dropna(inplace=True)
print("Dataset shape after nan values removed: {}".format(dt.shape))

county_database2_imputed.index = county_database2_imputed.FIPS
dt = county_database2_imputed.copy(deep=True)
# transform
dt["total_pop"] = np.log(dt["total_pop"])

print("Dataset shape after imputing data at the state level: {}".format(dt.shape))
dt.dropna(inplace=True)
print("Same dataset after nan values removed: {}".format(dt.shape))

dt = county_database2_imputed.copy(deep=True)
columns_to_drop = []
for c in dt.columns:
  if ("ecra" in c) or (c == "political_leaning"):
    columns_to_drop.append(c)
dt.drop(columns=columns_to_drop, inplace=True)
print("Dataset shape after imputing data at the state level (without biobot data & political leaning): {}".format(dt.shape))
dt.dropna(inplace=True)
print("Same dataset after nan values removed: {}".format(dt.shape))

dt = county_database2_imputed.copy(deep=True)
columns_to_drop = []
for c in dt.columns:
  if "ecra" in c:
    columns_to_drop.append(c)
dt.drop(columns=columns_to_drop, inplace=True)
print("Dataset shape after imputing data at the state level (without biobot data): {}".format(dt.shape))
dt.dropna(inplace=True)
print("Same dataset after nan values removed: {}".format(dt.shape))

y1 = (dt["deathCount_period1"] >= 5).astype(int)
y2 = (dt["deathCount_period2"] >= 5).astype(int)
for i, v in enumerate(y1):
  if v == 1:  # infected county period 1
    y2.iloc[i] = 0
y3 = (dt["deathCount_period3"] >= 5).astype(int)
for i, v in enumerate(y1):
  if v == 1:  # infected county period 1
    y3.iloc[i] = 0
for i, v in enumerate(y2):
  if v == 1:  # infected county period 2
    y3.iloc[i] = 0
dtNoIntroduced = dt[(dt["deathCount_period1"] < 5) & (dt["deathCount_period2"] < 5) & (dt["deathCount_period3"] < 5)]
print("\n#####################")
print("Introduction models")
print("#####################\n")
print("Dataset shape: {}".format(dt.shape))
print("Number of COVID-19 introductions in period 1: {} ({}%)".format(y1.sum(),
                                                                      round(100 * (y1.sum()) / len(dt), 2) ))
print("Number of COVID-19 introductions in period 2: {} ({}%)".format(y2.sum(),
                                                                      round(100 * (y2.sum()) / len(dt), 2) ))
print("Number of COVID-19 introductions in period 3: {} ({}%)".format(y3.sum(),
                                                                      round(100 * (y3.sum()) / len(dt), 2) ))
print("Number of counties with less than 5 COVID-19 related death during period 1,2,3: {} ({}%)".format(len(dtNoIntroduced),
                                                                                                        round(100 * len(dtNoIntroduced) / len(dt), 2)))
print("\n#####################")
print("Virus spread models")
print("#####################\n")
dt1 = dt[dt["deathCount_period1"] >= 5]
dt2 = dt[(dt["deathCount_period2"] >= 5) | (dt["deathCount_period1"] >= 5)]
dt3 = dt[((dt["deathCount_period2"] >= 5) | (dt["deathCount_period3"] >= 5)) & (dt["deathCount_period1"] < 5)]
nodt = dt[(dt["deathCount_period1"] < 5) & (dt["deathCount_period2"] < 5) & (dt["deathCount_period3"] < 5)]
print("Number of counties considered for period 1 models: {} ({}%)".format(len(dt1),
                                                                           round(100 * len(dt1) / len(dt), 2) ))
print("Number of counties considered for period 2 models: {} ({}%)".format(len(dt2),
                                                                           round(100 * len(dt2) / len(dt), 2) ))
print("Number of counties considered for period 3 models: {} ({}%)".format(len(dt3),
                                                                           round(100 * len(dt3) / len(dt), 2) ))
print("Number of counties never considered for virus spread models: {} ({}%)".format(len(nodt),
                                                                                     round(100 * len(nodt) / len(dt), 2) ))

### Outliers

In [ ]:
columns_to_check = ['min_distance_top_airport',
                    'political_leaning', 'pch_white', 'pct_black',
                    'pct_asian', 'pct_hispanic', 'pct_other', 'education',
                    'under_19', 'over_65', 'income', 'hc',
                    'total_jail_pop', 'nursing_population', 'obesity',
                    'log_pop_density', 'log_crowding', 'pct_nursing',
                    'pct_jail', 'sv_groupquarterpop', 'sv_pdisability',
                    'sv_singleparent', 'sv_penglish',
                    'sv_pmultiunit', 'sv_pmobilehome', 'ses_ppoverty',
                    'ses_punemployed',
                    'hc_hospitals_per1000', 'hc_icubeds_per1000',
                    'hc_icubeds_per60more1000', 'hc_icubeds_per65more1000',
                    'hc_pnotinsured_acs',
                    'hc_primarycare_per1000', 'como_medicareheartdizprev',
                    'hc_medicaid', 'como_pdiabetes',
                    'como_htn_hosp', 'como_htn_mort', 'como_cvd_hosp',
                    'como_cvd_mort', 'como_allheartdis_hosp',
                    'allheartdis_mort', 'como_stroke_hosp', 'como_stroke_mort',
                    'como_smoking', 'como_COPD', 'como_asthma']
for per in range(1, 7):
    for cat in ["min", "median", "max", "mean", "std"]:
        columns_to_check.append(f"StringencyIndex{per}_{cat}")
county_database2[columns_to_check].describe()

Outlier detection with 3 sigma. Doesn't make a lot of sense due to the high number of features being percentages.

In [ ]:
county_database2["isOutlier"] = len(county_database2) * [False]
outliers_pairs = []
for c in columns_to_check:
  upper_boundary = county_database2[c].mean() + 3 * county_database2[c].std()
  lower_boundary = county_database2[c].mean() - 3 * county_database2[c].std()
  for i, v in enumerate(county_database2[c]):
    if (v < lower_boundary) or (v > upper_boundary):
      outliers_pairs.append([i, c, v, lower_boundary, upper_boundary])
    county_database2["isOutlier"].iloc[i] = county_database2["isOutlier"].iloc[i] or (v < lower_boundary) or (v > upper_boundary)
print("Number of outliers: {} ({}%)".format(county_database2["isOutlier"].sum(),
                                            round(100 * (county_database2["isOutlier"].sum() / len(county_database2)), 2)))

## Fit exponentials first wave

In [ ]:
df = pd.read_csv("COVID-19_timeseries.csv")
df.index = df["Unnamed: 0"]
col = list(df.columns)
col[0] = "FIPS"
df.columns = col
df.drop(columns=["FIPS"], inplace=True)

In [ ]:
df.index = pd.to_datetime(df.index)

county_to_drop = []
for fips in list(df.columns):
  if (df[fips].isnull()).sum() == len(df):  # only nan
    county_to_drop.append(fips)
  if np.count_nonzero(df[fips] == 0) == len(df):  # only 0
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
df.drop(columns=county_to_drop, inplace=True)
df = df.fillna(0)  # let's suppose that no data = no death
df = df.T
df

In [ ]:
new = (df.T - df.T.shift(1))[1:]  # new death per day (not cumulative data)

# drop counties with no death in the first period
county_to_drop = []
sub_new = new[new.index <= pd.to_datetime("2020-06-30")]
for fips in list(sub_new.columns):
  if np.count_nonzero(sub_new[fips] == 0) == len(sub_new):  # no death
    county_to_drop.append(fips)
print("{} counties dropped".format(len(county_to_drop)))
new.drop(columns=county_to_drop, inplace=True)
new = new.T
new

Plot first 10 counties (before the end of june)

In [ ]:
new[:10].T[new[:10].T.index <= pd.to_datetime("2020-06-30")].plot()

Let's select the maximum number of daily death before the end of June as a threshold to truncate (for each county) and fit an exponential on the cumulative number of deaths

In [ ]:
max_id_list = []
for c in list(new.index):
  max_id = np.argmax(new.T[new.T.index <= pd.to_datetime("2020-06-30")][c])
  if max_id < 5:  # maximum peak of death too early, so let's say the first wave for this county is until the end of June
    max_id = len(new)
  max_id_list.append(max_id)

$t_{0}=\min_{t, x(t)\neq 0}t$

For all $t\geq t_{0}$,

$x(t+1)=x(t)e^{A+B x(t)}$

Equivalent to: $y(t) = A+B x(t)$ where $y(t)=ln\left (\frac{x(t+1)}{x(t)}\right )$

In [ ]:
from sklearn.linear_model import LinearRegression

counties_list = list(new.index)
fips_used_list = []
coefficients = []
reg_list = []
x_list = []
time_list = []
for i, fips in enumerate(counties_list):
  try:
    x = df.loc[fips].to_numpy()
    # truncate x
    max_id = max_id_list[i]
    min_id = np.argmin(x == 0)  # first day with a number of death != 0

    if (max_id + 1 - min_id) >= 10:  # at least 10 days with deaths to fit a model

      x = x[min_id:max_id+2]  # new is shifted by 1 so max_id too

      y = np.log2(x[1:]/x[:-1])
      x = x[:-1].reshape(-1, 1)

      reg = LinearRegression()
      reg.fit(x, y)
      x_list.append(x)
      time_list.append(list(df.columns)[min_id:max_id+1])
      fips_used_list.append(fips)
      reg_list.append(reg)
      coefficients.append([reg.intercept_, reg.coef_[0]])  # [A, B]
  except Exception as e:
    pass
coefficients = np.array(coefficients)

Plot some exponential regression

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 12))
for i in range(6):
  x = x_list[i]
  reg = reg_list[i]
  fips = fips_used_list[i]
  time_l = time_list[i]
  A, B = coefficients[i]
  y = reg.predict(x)
  x = x.reshape(len(x))
  y = x * np.exp(y)  # x(t+1) predictions
  axs[i//3, i%3].plot(time_l[1:], x[1:], color="red", label=f"True {fips}")
  axs[i//3, i%3].plot(time_l[1:], y[:-1], color="black", label=f"Exp fit {fips}", linestyle="dashed")
  axs[i//3, i%3].tick_params(labelrotation=45)
  axs[i//3, i%3].legend(loc="best")

In [ ]:
pol_list = []
for fips in fips_used_list:
  try:
    pol = county_database2_imputed[county_database2_imputed["FIPS"] == int(fips)]["political_leaning"].iloc[0]
  except Exception as e:
    pol = np.nan
  pol_list.append(pol)
plt.scatter(coefficients[:,0], coefficients[:,1], c=pol_list)

# Plots

### Initialization

In [ ]:
import geopandas
import shapely
import shapefile
import plotly
from plotly.figure_factory._county_choropleth import create_choropleth

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
fips = county_database["FIPS"].tolist()

selected_col_name = "StringencyIndex1_mean"
title = "USA by mean Stringency Index during period 1"
legend_title = "stringency index period 1"
min_value = county_database[selected_col_name].min()
max_value = county_database[selected_col_name].max()
endpts = list(np.linspace(min_value, max_value, len(colorscale) - 1))

values = county_database[selected_col_name].tolist()

fig = create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title=title,
    legend_title=legend_title
)

fig.layout.template = None
fig.show()

Create a function to easily display a feature on the U.S. map

In [ ]:
import plotly.express as px
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_json = json.load(response)
# duplicate the county with no 0 at the beginning of the FIPS
list_of_geo = counties_json["features"]
list_of_id = [v["id"] for v in counties_json["features"]]
for cty in counties_json["features"]:
  new_id = str(int(cty["id"]))
  if not(new_id in list_of_id):
    list_of_id.append(new_id)
    new_cty = cty.copy()
    new_cty["id"] = new_id
    list_of_geo.append(new_cty)
counties_json["features"] = list_of_geo

def create_custom_choropleth(county_database, counties_json, label_col,
                             label_display, color_continuous_scale=None,
                             range_color=None):
  if range_color is None:
    range_color_min = county_database[label_col].min()
    range_color_max = county_database[label_col].max()
  else:
    range_color_min = range_color[0]
    range_color_max = range_color[1]
  fig = px.choropleth(county_database, geojson=counties_json, locations="FIPS",
                      color=label_col,
                      color_continuous_scale=color_continuous_scale,
                      range_color=(range_color_min, range_color_max),
                      scope="usa",
                      labels={label_col:label_display},
                      title="USA by {}".format(label_display)
                      )
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.show()

### Stringency Index

In [ ]:
color_continuous_scale = "Viridis"
label_col = "StringencyIndex1_mean"
label_display = "Stringency Index (mean) Period 1"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### Political Leaning

In [ ]:
color_continuous_scale = "rdbu_r"  # _r to reverse the color scale
label_col = "political_leaning"
label_display = "Political Leaning"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### American Communities

In [ ]:
color_continuous_scale = "rainbow"
label_col = "acp"
label_display = "American Communities"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### Percent Hispanic

In [ ]:
color_continuous_scale = "purples"
label_col = "pct_hispanic"
label_display = "Percent Hispanic"
range_color = (0, 32)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale, range_color=range_color)

### Percent Black

In [ ]:
color_continuous_scale = "oranges"
label_col = "pct_black"
label_display = "Percent Black"
range_color = (0, 39)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Percent Obesity

In [ ]:
color_continuous_scale = "teal"
label_col = "obesity"
label_display = "Percent Obesity"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### Median Household Income

In [ ]:
color_continuous_scale = "blues"
label_col = "income"
label_display = "Median Household Income"
range_color = (20000, 75000)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Population Density (log scale)

In [ ]:
color_continuous_scale = "brwnyl"
label_col = "log_pop_density"
label_display = "Population Density (log scale)"
range_color = (-1.338, 2.835)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Crowding (log scale)

In [ ]:
color_continuous_scale = "greens"
label_col = "log_crowding"
label_display = "Crowding (log scale)"
range_color = (0, 3.453)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Minimum distance to a major airport

In [ ]:
color_continuous_scale = "thermal"
label_col = "min_distance_top_airport"
label_display = "Minimum distance to a major airport"
# use quantiles
range_color_min = np.quantile(county_database[label_col].dropna(), 0.2)
range_color_max = np.quantile(county_database[label_col].dropna(), 0.8)
range_color = (range_color_min, range_color_max)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Education (level greater of equal than highschool) %

In [ ]:
color_continuous_scale = "spectral"
label_col = "education"
label_display = "Education (%)"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### Percent Population >= 65

In [ ]:
color_continuous_scale = "tropic"
label_col = "over_65"
label_display = "Percent over 65y"
range_color_min = np.quantile(county_database[label_col].dropna(), 0.2)
range_color_max = np.quantile(county_database[label_col].dropna(), 0.8)
range_color = (range_color_min, range_color_max)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Percent Nursing Population

In [ ]:
color_continuous_scale = "picnic"
label_col = "pct_nursing"
label_display = "Percent Nursing Population"
range_color_min = np.quantile(county_database[label_col].dropna(), 0.2)
range_color_max = np.quantile(county_database[label_col].dropna(), 0.8)
range_color = (range_color_min, range_color_max)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### Percent Jail Population

In [ ]:
color_continuous_scale = "reds"
label_col = "pct_jail"
label_display = "Percent Jail Population"
range_color_min = np.quantile(county_database[label_col].dropna(), 0.2)
range_color_max = np.quantile(county_database[label_col].dropna(), 0.8)
range_color = (range_color_min, range_color_max)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

### HHS Regions

In [ ]:
color_continuous_scale = "rainbow"
label_col = "HHS Region"
label_display = "HHS Region"
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale)

### Crude death rate scatter plots (with trends)

#### Functions

Custom function to plot the scatter plots with trends

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

def custom_scatter_plot(feat_col="political_leaning",
                        feat_name="Political Leaning", *,
                        type_data="COVID-19",
                        cmid=None,
                        min_val=None, max_val=None,
                        filter=None, filter_threshold=None,
                        **kwargs):

  additional_text = ""  # for the title
  height = 600  # height of the plot

  county_db_to_use = county_databases[type_data]["county_database2_imputed"]

  if "filter_equality" in kwargs:
    county_db_to_use = county_db_to_use[county_db_to_use[kwargs.get("filter_equality")] == kwargs.get("filter_equality_value")]
    additional_text = additional_text + kwargs.get("additional_text", "")

  if filter is None:
    county_db = county_db_to_use
    county_db2 = county_db_to_use  # not used
    rows = 2
  else:
    county_db = county_db_to_use[county_db_to_use[filter] <= filter_threshold]
    county_db2 = county_db_to_use[county_db_to_use[filter] > filter_threshold]
    rows = 4

  if cmid is None:
    cmid = 0

  # shared_yaxes="all" or missing from kwargs
  shared_yaxes = kwargs.get("shared_yaxes", False)
  fig = make_subplots(rows=rows, cols=3,
                      subplot_titles=tuple(["Period {i}".format(i=i) for i in range(1, 7)]),
                      shared_yaxes=shared_yaxes)
  for per in range(6):
    fig.add_trace(
        go.Scatter(x=county_db[feat_col],
                   y=county_db[f"deathRate_period{per+1}"],
                   mode="markers",
                   marker_color=county_db[feat_col],
                   marker_colorscale="rdbu_r",
                   marker={"cmid": cmid},
                   name=f"Period {per+1}",
                   showlegend=False),
        row=(per//3 + 1), col=(per%3 + 1)
    )

    dt = county_db[[feat_col, f"deathRate_period{per+1}", "total_pop"]].dropna()
    lr = LinearRegression(fit_intercept=True)
    lr.fit(dt[feat_col].to_numpy().reshape(-1, 1),
          dt[f"deathRate_period{per+1}"].to_numpy(),
          np.log(dt["total_pop"]).to_numpy())  # log total pop sample weight
    
    slope = round(lr.coef_[0], 2)

    if min_val is None:
      min_val2 = county_db[feat_col].min()
    else:
      min_val2 = min_val
    if max_val is None:
      max_val2 = county_db[feat_col].max()
    else:
      max_val2 = max_val
    y_pred = lr.predict(np.linspace(min_val2, max_val2, 1000).reshape(-1, 1))
    fig.append_trace(go.Scatter(x=np.linspace(min_val2, max_val2, 1000), y=y_pred,
                                name=f"Regression Period {per+1}. Slope={slope}",
                                line=dict(color="black", width=4,
                                          dash="dash")
                                ), row=(per//3 + 1), col=(per%3 + 1))

    if not(filter is None):
      fig.add_trace(
        go.Scatter(x=county_db2[feat_col],
                   y=county_db2[f"deathRate_period{per+1}"],
                   mode="markers",
                   marker_color=county_db2[feat_col],
                   marker_colorscale="rdbu_r",
                   marker={"cmid": cmid},
                   name=f"Period {per+1}, {filter} above",
                   showlegend=False),
        row=(per//3 + 3), col=(per%3 + 1)
      )
      dt = county_db2[[feat_col, f"deathRate_period{per+1}", "total_pop"]].dropna()
      lr = LinearRegression(fit_intercept=True)
      lr.fit(dt[feat_col].to_numpy().reshape(-1, 1),
            dt[f"deathRate_period{per+1}"].to_numpy(),
            np.log(dt["total_pop"]).to_numpy())  # log total pop sample weight
      
      slope = round(lr.coef_[0], 2)

      if min_val is None:
        min_val2 = county_db2[feat_col].min()
      else:
        min_val2 = min_val
      if max_val is None:
        max_val2 = county_db2[feat_col].max()
      else:
        max_val2 = max_val
      y_pred = lr.predict(np.linspace(min_val2, max_val2, 1000).reshape(-1, 1))
      fig.append_trace(go.Scatter(x=np.linspace(min_val2, max_val2, 1000), y=y_pred,
                                  name=f"Regression Period {per+1}, {filter} above. Slope={slope}",
                                  line=dict(color="black", width=4,
                                            dash="dash")
                                  ), row=(per//3 + 3), col=(per%3 + 1))
      additional_text = additional_text + f"<br>{filter} <= {filter_threshold} compared to {filter} above"
      height = 1200  # higher image is mandatory as we add more rows
  fig.update_layout(height=height, width=800,
                    title_text=f"{type_data} Crude Death Rate and {feat_name} by period{additional_text}",
                    xaxis_title=f"{feat_name}",
                    yaxis_title=f"{type_data} crude death rate",
                    legend_title="Slopes")
  fig.show()

Function for the stringency index (as it is a function of the period)

In [ ]:
def custom_scatter_plot_stringency(*,
                                   min_val=None, max_val=None,
                                   type_data="COVID-19",
                                   filter=None, filter_threshold=None):
  feat_name = "Stringency Index"

  county_db_to_use = county_databases[type_data]["county_database2_imputed"]

  if filter is None:
    county_db = county_db_to_use
    county_db2 = county_db_to_use  # not used
    rows = 2
  else:
    county_db = county_db_to_use[county_db_to_use[filter] <= filter_threshold]
    county_db2 = county_db_to_use[county_db_to_use[filter] > filter_threshold]
    rows = 4

  fig = make_subplots(rows=rows, cols=3,
                      subplot_titles=tuple(["Period {i}".format(i=i) for i in range(1, 7)]),
                      shared_yaxes="all")
  for per in range(6):
    fig.add_trace(
        go.Scatter(x=county_db[f"StringencyIndex{per+1}_mean"],
                   y=county_db[f"deathRate_period{per+1}"],
                   mode="markers",
                   marker_color=county_db[f"StringencyIndex{per+1}_mean"],
                   marker_colorscale="rdbu_r",
                   marker={"cmid": 37},
                   name=f"Period {per+1}",
                   showlegend=False),
        row=(per//3 + 1), col=(per%3 + 1)
    )

    dt = county_db[[f"StringencyIndex{per+1}_mean", f"deathRate_period{per+1}", "total_pop"]].dropna()
    lr = LinearRegression(fit_intercept=True)
    lr.fit(dt[f"StringencyIndex{per+1}_mean"].to_numpy().reshape(-1, 1),
           dt[f"deathRate_period{per+1}"].to_numpy(),
           np.log(dt["total_pop"]).to_numpy())  # log total pop sample weight
    
    slope = round(lr.coef_[0], 2)

    if min_val is None:
      min_val2 = county_db[f"StringencyIndex{per+1}_mean"].min()
    else:
      min_val2 = min_val
    if max_val is None:
      max_val2 = county_db[f"StringencyIndex{per+1}_mean"].max()
    else:
      max_val2 = max_val

    y_pred = lr.predict(np.linspace(min_val2, max_val2, 1000).reshape(-1, 1))
    fig.append_trace(go.Scatter(x=np.linspace(min_val2, max_val2, 1000), y=y_pred,
                                name=f"Regression Period {per+1}. Slope={slope}",
                                line=dict(color="black", width=4,
                                          dash="dash")
                                ), row=(per//3 + 1), col=(per%3 + 1))
    additional_text = ""  # for the title
    height = 600  # height of the plot

    if not(filter is None):
      fig.add_trace(
        go.Scatter(x=county_db2[f"StringencyIndex{per+1}_mean"],
                   y=county_db2[f"deathRate_period{per+1}"],
                   mode="markers",
                   marker_color=county_db2[f"StringencyIndex{per+1}_mean"],
                   marker_colorscale="rdbu_r",
                   marker={"cmid": 37},
                   name=f"Period {per+1}, {filter} above",
                   showlegend=False),
        row=(per//3 + 3), col=(per%3 + 1)
      )
      dt = county_db2[[f"StringencyIndex{per+1}_mean", f"deathRate_period{per+1}", "total_pop"]].dropna()
      lr = LinearRegression(fit_intercept=True)
      lr.fit(dt[f"StringencyIndex{per+1}_mean"].to_numpy().reshape(-1, 1),
            dt[f"deathRate_period{per+1}"].to_numpy(),
            np.log(dt["total_pop"]).to_numpy())  # log total pop sample weight
      
      slope = round(lr.coef_[0], 2)

      if min_val is None:
        min_val2 = county_db2[f"StringencyIndex{per+1}_mean"].min()
      else:
        min_val2 = min_val
      if max_val is None:
        max_val2 = county_db2[f"StringencyIndex{per+1}_mean"].max()
      else:
        max_val2 = max_val
      y_pred = lr.predict(np.linspace(min_val2, max_val2, 1000).reshape(-1, 1))
      fig.append_trace(go.Scatter(x=np.linspace(min_val2, max_val2, 1000), y=y_pred,
                                  name=f"Regression Period {per+1}, {filter} above. Slope={slope}",
                                  line=dict(color="black", width=4,
                                            dash="dash")
                                  ), row=(per//3 + 3), col=(per%3 + 1))
      additional_text = f"<br>{filter} <= {filter_threshold} compared to {filter} above"
      height = 1200  # higher image is mandatory as we add more rows
  fig.update_layout(height=height, width=800,
                    title_text=f"{type_data} Crude Death Rate and {feat_name} by period{additional_text}",
                    xaxis_title=f"{feat_name}",
                    yaxis_title=f"{type_data} crude death rate",
                    legend_title="Slopes")
  fig.show()

#### Plots

In [ ]:
custom_scatter_plot(feat_col="political_leaning",
                    feat_name="Political Leaning",
                    type_data="COVID-19",
                    min_val=-0.5, max_val=0.5,
                    cmid=0)

In [ ]:
custom_scatter_plot(feat_col="political_leaning",
                    feat_name="Political Leaning",
                    type_data="All Causes",
                    min_val=-0.5, max_val=0.5,
                    cmid=0,
                    filter="log_pop_density",
                    filter_threshold=round(np.quantile(county_databases["All Causes"]["county_database2_imputed"]["log_pop_density"].dropna(), 0.9), 2))

In [ ]:
custom_scatter_plot(feat_col="political_leaning",
                    feat_name="Political Leaning",
                    type_data="All Causes",
                    min_val=-0.5, max_val=0.5,
                    cmid=0,
                    filter="log_pop_density",
                    filter_threshold=round(np.quantile(county_databases["All Causes"]["county_database2_imputed"]["log_pop_density"].dropna(), 0.9), 2))

In [ ]:
custom_scatter_plot(feat_col="political_leaning",
                    feat_name="Political Leaning",
                    type_data="Excess Mortality",
                    min_val=-0.5, max_val=0.5,
                    cmid=0)

In [ ]:
custom_scatter_plot(feat_col="StringencyIndex1",
                    feat_name="Stringency Index Period 1",
                    type_data="COVID-19",
                    cmid=37)

In [ ]:
custom_scatter_plot_stringency(type_data="COVID-19")

In [ ]:
custom_scatter_plot_stringency(type_data="All Causes")

In [ ]:
custom_scatter_plot_stringency(type_data="Excess Mortality")

In [ ]:
acp_nb = 5
acp_name = county_database[county_database["acp"] == acp_nb]["acp_name"].iloc[0]
kwargs = {"filter_equality": "acp",
          "filter_equality_value": acp_nb,
          "additional_text": f"<br>American Communities: {acp_name}"}
custom_scatter_plot(feat_col="political_leaning",
                    feat_name="Political Leaning",
                    type_data="Excess Mortality",
                    min_val=-0.5, max_val=0.5,
                    cmid=0, **kwargs)

In [ ]:
custom_scatter_plot(feat_col="log_pop_density",
                    feat_name="Log Pop density",
                    cmid=4.5)

In [ ]:
custom_scatter_plot(feat_col="over_65",
                    feat_name="Over 65 %",
                    cmid=30)

In [ ]:
custom_scatter_plot(feat_col="obesity",
                    feat_name="Obesity %",
                    type_data="COVID-19",
                    cmid=35)

In [ ]:
custom_scatter_plot(feat_col="obesity",
                    feat_name="Obesity %",
                    type_data="All Causes",
                    cmid=35)

In [ ]:
custom_scatter_plot(feat_col="obesity",
                    feat_name="Obesity %",
                    type_data="Excess Mortality",
                    cmid=35)

In [ ]:
custom_scatter_plot(feat_col="pct_hispanic",
                    feat_name="Hispanic population %",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["pct_hispanic"].median())

In [ ]:
custom_scatter_plot(feat_col="pct_jail",
                    feat_name="Jail population %",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["pct_jail"].median())

In [ ]:
custom_scatter_plot(feat_col="pct_nursing",
                    feat_name="Nursing population %",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["pct_nursing"].median())

In [ ]:
custom_scatter_plot(feat_col="income",
                    feat_name="Median Household Income",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["income"].median())

In [ ]:
custom_scatter_plot(feat_col="como_asthma",
                    feat_name="Comorbidities Asthma %",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["como_asthma"].median())

In [ ]:
custom_scatter_plot(feat_col="income",
                    feat_name="Median Household Income",
                    type_data="COVID-19",
                    cmid=county_databases["COVID-19"]["county_database2_imputed"]["income"].median())

In [ ]:
list(county_database2_imputed.columns)

### Crude Death Rate map plots

Plot the death count & death rate of each county per period

In [ ]:
color_continuous_scale = "matter"
label_col = "deathCount_period1"
label_display = "Period 1 Death Count"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathCount_period2"
label_display = "Period 2 Death Count"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathCount_period3"
label_display = "Period 3 Death Count"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period1"
label_display = "Period 1 Death Rates"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period2"
label_display = "Period 2 Death Rates"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period3"
label_display = "Period 3 Death Rates"
range_color = (0, 82.38)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period4"
label_display = "Period 4 Death Rates"
range_color = (0, 500)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period5"
label_display = "Period 5 Death Rates"
range_color = (0, 500)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

In [ ]:
color_continuous_scale = "matter"
label_col = "deathRate_period6"
label_display = "Period 6 Death Rates"
range_color = (0, 500)
create_custom_choropleth(county_database, counties_json, label_col,
                         label_display, color_continuous_scale,
                         range_color=range_color)

# Models

## Core functions

Import

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, GridSearchCV, RepeatedKFold, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_percentage_error
import seaborn as sns
from tqdm import tqdm

Virus introduction models

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, \
  balanced_accuracy_score, precision_score, recall_score, f1_score, \
  mean_absolute_error, r2_score, mean_squared_error, roc_auc_score, \
  precision_recall_curve, auc, mean_absolute_percentage_error, \
  average_precision_score, pairwise_distances
from sklearn.preprocessing import label_binarize


def pr_auc_score(y, y_pred, **kwargs):
  if len(y_pred.shape) == 1:  # binary classification
    precision, recall, _ = precision_recall_curve(y, y_pred, **kwargs)
  else:
    classes = list(range(y_pred.shape[1]))
    if len(classes) == 2:  # binary classification too
        precision, recall, _ = precision_recall_curve(y, y_pred[:,1],
                                                      **kwargs)
    else:  # multiclass
      Y = label_binarize(y, classes=classes)
      precision, recall, _ = precision_recall_curve(Y.ravel(), y_pred.ravel(),
                                                    **kwargs)
  return auc(recall, precision)

def avg_precision(y, y_pred, **kwargs):
  if len(y_pred.shape) == 1:  # binary classification
    return average_precision_score(y, y_pred, average="micro", **kwargs)
  classes = list(range(y_pred.shape[1]))
  if len(classes) == 2:  # binary classification too
    return average_precision_score(y, y_pred[:,1], average="micro", **kwargs)
  # multiclass
  Y = label_binarize(y, classes=classes)
  return average_precision_score(Y, y_pred, average="micro", **kwargs)

custom_scorer_clf = {'accuracy': make_scorer(accuracy_score,
                                             greater_is_better=True),
                     'balanced_accuracy': make_scorer(balanced_accuracy_score,
                                                      greater_is_better=True),
                     'precision': make_scorer(precision_score, average='macro'),
                     'recall': make_scorer(recall_score, average='macro'),
                     'f1': make_scorer(f1_score, average='macro',
                                       greater_is_better=True),
                     'auroc': make_scorer(roc_auc_score, multi_class="ovo",
                                          average="macro",
                                          needs_proba=True,
                                          greater_is_better=True),
                     'auprc': make_scorer(pr_auc_score, needs_proba=True,
                                          greater_is_better=True),
                     'avg_precision': make_scorer(avg_precision,
                                                  needs_proba=True,
                                                  greater_is_better=True)}

def virus_introduction_all(county_db, selected_columns_list,
                           X_selected_columns_list, *,
                           plot=False):
  
  def print2(x):
    if plot:
      print(x)

  county_db.index = county_db.FIPS
  dt = county_db[selected_columns_list[-1] + ["Province_State"]]  # total_pop, Province_State and deathcounts will be removed later

  dt["total_pop"] = np.log(dt["total_pop"])  # sample weights
  print2(dt.shape)

  # drop nan values
  dt.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt.shape))

  # extract groups (province states)
  groups = dt["Province_State"].to_numpy()

  # Dataset: Period 1, 2, 3 or after
  X = dt[X_selected_columns_list[-1]]
  y = np.array([0 for _ in range(len(dt))])
  for i in range(len(dt)):
    p1_count = (dt["deathCount_period1"].iloc[i] >= 5)
    p2_count = (dt["deathCount_period2"].iloc[i] >= 5)
    p3_count = (dt["deathCount_period3"].iloc[i] >= 5)
    if p1_count:
      y[i] = 0
    elif p2_count and not(p1_count):
      y[i] = 1
    elif p3_count and not(p1_count or p2_count):
      y[i] = 2
    else:
      y[i] = 3
  # weight the regression by the log of a county’s population
  sample_weight = dt["total_pop"].to_numpy()

  return dt, X, y, sample_weight, groups


def virus_introduction_periods(county_db, selected_columns_list,
                               X_selected_columns_list,
                               *, plot=False):
  def print2(x):
    if plot:
      print(x)
  
  county_db.index = county_db.FIPS
  dt = county_db[selected_columns_list[-1] + ["Province_State"]]  # total_pop and deathcounts will be removed later

  dt["total_pop"] = np.log(dt["total_pop"])  # sample weights
  print2(dt.shape)

  # drop nan values
  dt.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt.shape))

  # extract groups (province states)
  groups = dt["Province_State"].to_numpy()

  # Dataset: Period 1, 2, 3 or after
  X = dt[X_selected_columns_list[-1]]
  y1 = (dt["deathCount_period1"] >= 5).astype(int)

  # Second dataset: remove counties already infected by COVID-19
  y2 = (dt["deathCount_period2"] >= 5).astype(int)
  for i, v in enumerate(y1):
    if v == 1:  # infected county
      y2.iloc[i] = 0

  # Third dataset: remove counties already infected by COVID-19 in period 1 and 2
  y3 = (dt["deathCount_period3"] >= 5).astype(int)
  for i, v in enumerate(y1):
    if v == 1:  # infected county period 1
      y3.iloc[i] = 0
  for i, v in enumerate(y2):
    if v == 1:  # infected county period 2
      y3.iloc[i] = 0
  
  yother = (((dt["deathCount_period4"] >= 5) | (dt["deathCount_period5"] >= 5) | (dt["deathCount_period6"] >= 5)) & ((dt["deathCount_period1"] < 5) & (dt["deathCount_period2"] < 5) & (dt["deathCount_period3"] < 5))).astype(int)

  y_list = [y1, y2, y3, yother]

  # weight the regression by the log of a county’s population
  sample_weight = dt["total_pop"].to_numpy()

  return dt, X, y_list, sample_weight, groups


def lr_pipeline_period(X, y, sample_weight, period_number, *, groups=None, plot=True):

  model = make_pipeline(MinMaxScaler(), LogisticRegression(random_state=42, n_jobs=-1))
  fit_params = {"logisticregression__sample_weight": sample_weight}
  model.fit(X, y, **fit_params)

  coefs_p = pd.DataFrame(
   model[1].coef_[0],
   columns=["Coefficients"], index=X.columns
  )

  if plot:
    coefs_p.plot(kind="barh", figsize=(9, 7))
    plt.title(f"Coefficient: Virus Introduction Period {period_number} (Logistic Regression)")
    plt.axvline(x=0, color=".5")
    plt.subplots_adjust(left=.3)

  if groups is None:
    cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
  else:
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
    cv = sss.split(X, groups)
  cv_model = cross_validate(
  model, X, y, cv=cv,
  return_estimator=True, n_jobs=-1, fit_params=fit_params,
  scoring=custom_scorer_clf
  )
  coefs_all_p = pd.DataFrame(
    [model[1].coef_[0]
      for model in cv_model["estimator"]],
    columns=X.columns
  )

  if plot:
    plt.figure(figsize=(9, 7))
    sns.boxplot(data=coefs_all_p, orient="h", color="blue", saturation=0.5)
    plt.axvline(x=0, color='.5')
    plt.xlabel("Coefficient importance")
    plt.title(f"Coefficient importance and its variability - Virus Introduction Period {period_number} (Logistic Regression)")
    plt.subplots_adjust(left=.3)

  return coefs_p, coefs_all_p, model, cv_model

Virus spread models

In [ ]:
def virus_spread_dt(county_db, selected_columns_list, X_selected_columns_list,
                    type_data,
                    *, plot=False):
  def print2(x):
    if plot:
      print(x)
  
  county_db.index = county_db.FIPS

  # Datasets

  # Period 1

  dt1 = county_db[selected_columns_list[0] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt1["total_pop"] = np.log(dt1["total_pop"])  # sample weights
  print2(dt1.shape)
  # drop nan values
  dt1.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt1.shape))
  if type_data != "All Causes":
    dt1 = dt1[dt1["deathCount_period1"] >= 5]
  X1 = dt1[X_selected_columns_list[0]]
  y1 = dt1["deathRate_period1"]
  # weight the regression by the log of a county’s population
  sample_weight1 = dt1["total_pop"].to_numpy()
  # extract groups (province states)
  groups1 = dt1["Province_State"].to_numpy()
  print2("Dataset shape period 1: {}".format(dt1.shape))

  # Period 2

  dt2 = county_db[selected_columns_list[1] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt2["total_pop"] = np.log(dt2["total_pop"])  # sample weights
  print2(dt2.shape)
  # drop nan values
  dt2.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt2.shape))
  if type_data != "All Causes":
    dt2 = dt2[(dt2["deathCount_period2"] >= 5) | (dt2["deathCount_period1"] >= 5)]
  X2 = dt2[X_selected_columns_list[1]]
  y2 = dt2["deathRate_period2"]
  # weight the regression by the log of a county’s population
  sample_weight2 = dt2["total_pop"].to_numpy()
  # extract groups (province states)
  groups2 = dt2["Province_State"].to_numpy()
  print2("Dataset shape period 2: {}".format(dt2.shape))

  # Dataset 3

  dt3 = county_db[selected_columns_list[2] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt3["total_pop"] = np.log(dt3["total_pop"])  # sample weights
  print2(dt3.shape)
  # drop nan values
  dt3.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt3.shape))
  if type_data != "All Causes":
    dt3 = dt3[((dt3["deathCount_period2"] >= 5) | (dt3["deathCount_period3"] >= 5)) & (dt3["deathCount_period1"] < 5)]
  X3 = dt3[X_selected_columns_list[2]]
  y3 = dt3["deathRate_period3"]
  # weight the regression by the log of a county’s population
  sample_weight3 = dt3["total_pop"].to_numpy()
  # extract groups (province states)
  groups3 = dt3["Province_State"].to_numpy()
  print2("Dataset shape period 3: {}".format(dt3.shape))

  # Period 4

  dt4 = county_db[selected_columns_list[3] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt4["total_pop"] = np.log(dt4["total_pop"])  # sample weights
  print2(dt4.shape)
  # drop nan values
  dt4.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt4.shape))
  if type_data != "All Causes":
    dt4 = dt4[dt4["deathCount_period4"] >= 5]
  X4 = dt4[X_selected_columns_list[3]]
  y4 = dt4["deathRate_period4"]
  # weight the regression by the log of a county’s population
  sample_weight4 = dt4["total_pop"].to_numpy()
  # extract groups (province states)
  groups4 = dt4["Province_State"].to_numpy()
  print2("Dataset shape period 4: {}".format(dt4.shape))

  # Period 5

  dt5 = county_db[selected_columns_list[4] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt5["total_pop"] = np.log(dt5["total_pop"])  # sample weights
  print2(dt5.shape)
  # drop nan values
  dt5.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt5.shape))
  if type_data != "All Causes":
    dt5 = dt5[dt5["deathCount_period5"] >= 5]
  X5 = dt5[X_selected_columns_list[4]]
  y5 = dt5["deathRate_period5"]
  # weight the regression by the log of a county’s population
  sample_weight5 = dt5["total_pop"].to_numpy()
  # extract groups (province states)
  groups5 = dt5["Province_State"].to_numpy()
  print2("Dataset shape period 5: {}".format(dt5.shape))

  # Period 6

  dt6 = county_db[selected_columns_list[5] + ["Province_State"]]  # total_pop and deathcounts will be removed later
  dt6["total_pop"] = np.log(dt6["total_pop"])  # sample weights
  print2(dt6.shape)
  # drop nan values
  dt6.dropna(inplace=True)
  print2("Dataset shape after nan values removed: {}".format(dt6.shape))
  if type_data != "All Causes":
    dt6 = dt6[dt6["deathCount_period6"] >= 5]
  X6 = dt6[X_selected_columns_list[5]]
  y6 = dt6["deathRate_period6"]
  # weight the regression by the log of a county’s population
  sample_weight6 = dt6["total_pop"].to_numpy()
  # extract groups (province states)
  groups6 = dt6["Province_State"].to_numpy()
  print2("Dataset shape period 6: {}".format(dt6.shape))

  dt_list = [dt1, dt2, dt3, dt4, dt5, dt6]
  X_list = [X1, X2, X3, X4, X5, X6]
  y_list = [y1, y2, y3, y4, y5, y6]
  sample_weight_list = [sample_weight1, sample_weight2, sample_weight3,
                        sample_weight4, sample_weight5, sample_weight6]
  groups_list = [groups1, groups2, groups3, groups4, groups5,
                 groups6]

  return dt_list, X_list, y_list, sample_weight_list, groups_list


def elasticNet_pipeline_period(X_init, y, sample_weight, period_number, *,
                               groups=None, plot=True,
                               model_hyperparameters=None):
  # Split dataset: Train set contains 70% of total counties
  X = X_init.copy()  # COPY THE DATASET (because we modify it)
  
  if model_hyperparameters is None:  # grid search on train set to find the best ones
    X["sample_weight"] = sample_weight
    if groups is None:
      groups_train, groups_test = None, None
    else:
      X["groups"] = groups
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        stratify=groups,
                                                        test_size=0.33,
                                                        random_state=42)
    sample_weight_train, sample_weight_test = X_train["sample_weight"], X_test["sample_weight"]
    X_train.drop(columns=["sample_weight"], inplace=True)
    X_test.drop(columns=["sample_weight"], inplace=True)
    X.drop(columns=["sample_weight"], inplace=True)

    if not(groups is None):
      groups_train, groups_test = X_train["groups"], X_test["groups"]
      X_train.drop(columns=["groups"], inplace=True)
      X_test.drop(columns=["groups"], inplace=True)
      X.drop(columns=["groups"], inplace=True)

  if groups is None:
    cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
    cv_grid_search = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)
  else:
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
    cv = sss.split(X, groups)
    if model_hyperparameters is None:  # grid search cross val parameters
      sss2 = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
      cv_grid_search = sss2.split(X_train, groups_train)

  # Define the model
  model = make_pipeline(MinMaxScaler(), ElasticNet(random_state=42, max_iter=20000))
  
  scoring = "neg_mean_absolute_percentage_error"

  # Define & fit the grid search if no hyperparameters provided
  if model_hyperparameters is None:
    parameters = {"elasticnet__alpha": np.arange(0.05, 1+0.05, 0.05),
                  "elasticnet__l1_ratio": np.arange(0.05, 1+0.05, 0.05)}
    # We perform a grid search using 5-fold cross validation on our training data
    # and pick the hyper-parameters that yield the lowest Mean Absolute Percentage Error (MAPE)
    clf = GridSearchCV(model, parameters,
                      cv=cv_grid_search,
                      n_jobs=-1, scoring=scoring)
    fit_params = {"elasticnet__sample_weight": sample_weight_train}
    clf.fit(X_train, y_train, **fit_params)
    if plot:
      print("Best parameters: {}".format(clf.best_params_))
    best_params = clf.best_params_

  else:
    if plot:
      print("Hyperparameters provided: {}".format(model_hyperparameters))
    best_params = model_hyperparameters

  # Train an elastic net on the entire dataset with the optimal
  # parameters for the given period
  best_params["elasticnet__random_state"] = 42
  best_params["elasticnet__max_iter"] = 20000
  model = make_pipeline(MinMaxScaler(), ElasticNet(random_state=42, max_iter=20000))
  model.set_params(**best_params)
  fit_params = {"elasticnet__sample_weight": sample_weight}
  model.fit(X, y, **fit_params)

  # Plot coefficients
  coefs_p = pd.DataFrame(
   model[1].coef_,
   columns=["Coefficients"], index=X.columns
  )
  
  if plot:
    print(coefs_p)
    coefs_p.plot(kind="barh", figsize=(9, 7))
    plt.title(f"Coefficient: Virus Spread Period {period_number} (Elastic Net)")
    plt.axvline(x=0, color=".5")
    plt.subplots_adjust(left=.3)

  # Confidence intervals (multiple fit after repeated K fold or StratifiedShuffleSplit)
  cv_model = cross_validate(
   model, X, y, cv=cv,
   return_estimator=True, n_jobs=-1, fit_params=fit_params,
   scoring=scoring
  )
  coefs_all_p = pd.DataFrame(
    [model[1].coef_
      for model in cv_model["estimator"]],
      columns=X.columns
  )

  if plot:
    plt.figure(figsize=(9, 7))
    sns.boxplot(data=coefs_all_p, orient="h", color="blue", saturation=0.5)
    plt.axvline(x=0, color='.5')
    plt.xlabel("Coefficient importance")
    plt.title(f"Coefficient importance and its variability - Virus Spread Period {period_number} (Elastic Net)")
    plt.subplots_adjust(left=.3)

  return coefs_p, coefs_all_p, model, cv_model


def randomForest_pipeline_period(X_init, y, sample_weight, period_number, *,
                                 groups=None, plot=True,
                                 model_hyperparameters=None):
  # Split dataset: Train set contains 70% of total counties
  X = X_init.copy()  # COPY THE DATASET (because we modify it)
  
  if model_hyperparameters is None:
    X["sample_weight"] = sample_weight
    if groups is None:
      groups_train, groups_test = None, None
    else:
      X["groups"] = groups
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        stratify=groups,
                                                        test_size=0.33,
                                                        random_state=42)
    sample_weight_train, sample_weight_test = X_train["sample_weight"], X_test["sample_weight"]
    X_train.drop(columns=["sample_weight"], inplace=True)
    X_test.drop(columns=["sample_weight"], inplace=True)
    X.drop(columns=["sample_weight"], inplace=True)
    
    if not(groups is None):
      groups_train, groups_test = X_train["groups"], X_test["groups"]
      X_train.drop(columns=["groups"], inplace=True)
      X_test.drop(columns=["groups"], inplace=True)
      X.drop(columns=["groups"], inplace=True)

    if groups is None:
      cv_grid_search = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)
    else:
      sss2 = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
      cv_grid_search = sss2.split(X_train, groups_train)

  # Define the model
  model = make_pipeline(MinMaxScaler(), RandomForestRegressor(random_state=42, n_jobs=-1))

  scoring = "neg_mean_absolute_percentage_error"

  # Define & fit the grid search if no hyperparameters provided
  if model_hyperparameters is None:
    parameters = {"randomforestregressor__n_estimators": [100, 200, 500],
                  "randomforestregressor__max_features": [len(X.columns),
                                  int(np.sqrt(len(X.columns)))],
                  "randomforestregressor__max_depth": [20, 50],
                  "randomforestregressor__min_samples_split": [2, 5, 10],
                  "randomforestregressor__min_samples_leaf": [10, 20, 50],
                  "randomforestregressor__bootstrap": [True, False]}
    # We perform a grid search using 5-fold cross validation on our training data
    # and pick the hyper-parameters that yield the lowest Mean Absolute Percentage Error (MAPE)

    clf = GridSearchCV(model, parameters,
                       cv=cv_grid_search,
                       n_jobs=-1, scoring=scoring)
    fit_params = {"randomforestregressor__sample_weight": sample_weight_train}
    clf.fit(X_train, y_train, **fit_params)
    if plot:
      print("Best parameters: {}".format(clf.best_params_))
    best_params = clf.best_params_
  else:
    if plot:
      print("Hyperparameters provided: {}".format(model_hyperparameters))
    best_params = model_hyperparameters

  # Train a randomforestregressor on the entire dataset with the optimal
  # parameters for the given period
  best_params["randomforestregressor__random_state"] = 42
  best_params["randomforestregressor__n_jobs"] = -1
  model = make_pipeline(MinMaxScaler(), RandomForestRegressor(random_state=42, n_jobs=-1))
  model.set_params(**best_params)
  fit_params = {"randomforestregressor__sample_weight": sample_weight}
  model.fit(X, y, **fit_params)

  # coefficient
  feature_names = list(X.columns)
  importances = model[1].feature_importances_
  std = np.std([tree.feature_importances_ for tree in model[1].estimators_], axis=0)
  zipped = zip(importances, std, feature_names)
  importancesi, stdi, feature_namesi = [], [], []
  for x, y, z in sorted(zipped, key=lambda x: x[0]):
    importancesi.append(x)
    stdi.append(y)
    feature_namesi.append(z)
  errorsi = [1.96 * s / np.sqrt(len(stdi)) for s in stdi]
  importancesi = pd.DataFrame(importancesi,
                              columns=[f"Feature importances using MDI - Virus Spread Period {period_number} (RandomForestRegressor)\n95% confidence interval"],
                              index=feature_namesi)
  if plot:
    print(importancesi)
    # plot
    importancesi.plot(kind="barh", figsize=(9, 7), xerr=errorsi)  # xerr for horizontal plots
    plt.title(f"Feature importances using MDI - Virus Spread Period {period_number} (RandomForestRegressor)\n95% confidence interval")
    plt.legend(loc="best")
    plt.tight_layout()

  importancesi["errors"] = errorsi

  return importancesi, model

## Introduction Models

For the introduction models (logistic regressions), we take all the counties in our dataset and we try to predict **only the new infected counties** in a given period.

### MinMaxScaler & confidence intervals (Dataset 2)

#### Define datasets

In [ ]:
dt, X, y_list, \
  sample_weight, groups = virus_introduction_periods(county_database2_imputed,
                                                     selected_columns_list,
                                                     X_selected_columns_list,
                                                     plot=False)
y1, y2, y3, yother = y_list

#### Logistic Regression - Period 1 (February 2020 - May 2020)

In [ ]:
coefs_p1, coefs_all_p1, _, _ = lr_pipeline_period(X, y1, sample_weight, "1")

#### Logistic Regression - Period 2 (June 2020 - October 2020)

In [ ]:
coefs_p2, coefs_all_p2, _, _ = lr_pipeline_period(X, y2, sample_weight, "2")

#### Logistic Regression - Period 3 (November 2020 - February 2021)

In [ ]:
coefs_p3, coefs_all_p3, _, _ = lr_pipeline_period(X, y3, sample_weight, "3")

#### Logistic Regression - Period greater than 3 (March 2021 - February 2022)

In [ ]:
coefs_pother, coefs_all_pother, _, _ = lr_pipeline_period(X, yother, sample_weight, ">=4")

#### Plot COVID-19 introduction

In [ ]:
county_database_plot_intro = county_database2_imputed.copy()
covid_intro_l = ["Unknown" for _ in range(len(county_database_plot_intro))]
for i in range(len(county_database_plot_intro)):
  if county_database_plot_intro["deathCount_period1"].iloc[i] >= 5:
    covid_intro_l[i] = "1"
for i in range(len(county_database_plot_intro)):
  if (covid_intro_l[i] == "Unknown") and (county_database_plot_intro["deathCount_period2"].iloc[i] >= 5):
    covid_intro_l[i] = "2"
for i in range(len(county_database_plot_intro)):
  if (covid_intro_l[i] == "Unknown") and (county_database_plot_intro["deathCount_period3"].iloc[i] >= 5):
    covid_intro_l[i] = "3"
for i in range(len(county_database_plot_intro)):
  if (covid_intro_l[i] == "Unknown") and ((county_database_plot_intro["deathCount_period4"].iloc[i] >= 5) or (county_database_plot_intro["deathCount_period5"].iloc[i] >= 5) or (county_database_plot_intro["deathCount_period6"].iloc[i] >= 5)):
    covid_intro_l[i] = ">=4"

county_database_plot_intro["covid_intro_period"] = covid_intro_l

# plot
color_continuous_scale = "rainbown"
label_col = "covid_intro_period"
label_display = "COVID-19 Introduction Period"
create_custom_choropleth(county_database_plot_intro, counties_json, label_col,
                         label_display, color_continuous_scale)

#### Logistic Regression - All periods contributions

In [ ]:
import plotly.graph_objects as go
feature_names = list(X.columns)

intervals1 = (1.96 * coefs_all_p1.std() / np.sqrt(len(coefs_all_p1))).to_frame()
intervals2 = (1.96 * coefs_all_p2.std() / np.sqrt(len(coefs_all_p2))).to_frame()
intervals3 = (1.96 * coefs_all_p3.std() / np.sqrt(len(coefs_all_p3))).to_frame()
intervalsother = (1.96 * coefs_all_pother.std() / np.sqrt(len(coefs_all_pother))).to_frame()
period1_data = []
period2_data = []
period3_data = []
periodother_data = []
error_p1 = []
error_p2 = []
error_p3 = []
error_pother = []
for i, feat in enumerate(feature_names):
  p1 = coefs_p1.loc[feat].iloc[0]
  p2 = coefs_p2.loc[feat].iloc[0]
  p3 = coefs_p3.loc[feat].iloc[0]
  pother = coefs_pother.loc[feat].iloc[0]
  e1 = intervals1.loc[feat].iloc[0]
  e2 = intervals2.loc[feat].iloc[0]
  e3 = intervals3.loc[feat].iloc[0]
  eother = intervalsother.loc[feat].iloc[0]
  period1_data.append(p1)
  period2_data.append(p2)
  period3_data.append(p3)
  periodother_data.append(pother)
  error_p1.append(e1)
  error_p2.append(e2)
  error_p3.append(e3)
  error_pother.append(eother)
fig = go.Figure(data=[
    go.Bar(name="Period 1 (February 2020 - May 2020)", x=feature_names, y=period1_data,
           error_y=dict(type="data", array=error_p1)),
    go.Bar(name="Period 2 (June 2020 - October 2020)", x=feature_names, y=period2_data,
           error_y=dict(type="data", array=error_p2)),
    go.Bar(name="Period 3 (November 2020 - February 2021)", x=feature_names, y=period3_data,
           error_y=dict(type="data", array=error_p3)),
    go.Bar(name="Period >= 4 (March 2021 - February 2022)", x=feature_names, y=periodother_data,
           error_y=dict(type="data", array=error_pother))
])
# Change the bar mode
fig.update_layout(barmode="group",
                  xaxis_title="Features",
                  yaxis_title="Coefficients importance and its variability",
                  legend_title="Periods",
                  title="Coefficients importance and its variability (95% confidence interval)<br> for each period (MinMaxScaler & LogisticRegression)")
fig.show()

In [ ]:
directionality = pd.DataFrame()
directionality["Variable"] = feature_names
directionality["Period 1"] = period1_data
directionality["Period 2"] = period2_data
directionality["Period 3"] = period3_data
directionality["Period >= 4"] = periodother_data
for i in range(1, 4):
  directionality[f"Period {i}"] = directionality[f"Period {i}"].apply(lambda x: "+" if x >= 0 else "-")
directionality["Period >= 4"] = directionality["Period >= 4"].apply(lambda x: "+" if x >= 0 else "-")
directionality

#### Define dataset

In [ ]:
dt, X, y, sample_weight, groups = virus_introduction_all(county_database2_imputed,
                                                         selected_columns_list,
                                                         X_selected_columns_list,
                                                         plot=False)
plt.hist(y)
plt.xlabel("Period")
plt.ylabel("Count")
plt.title("Distribution of COVID-19 introductions for each periods (-1) (3=no introduction)")
plt.tight_layout()

#### Logistic Regression - Multiple output (taking into account all periods)

In [ ]:
coefs_p, coefs_all_p, _, _ = lr_pipeline_period(X, y, sample_weight, "All")

## Virus Spread Models

### MinMaxScaler & confidence intervals (ElasticNet) (Dataset 2)

Minimise MAPE to select best hyperparameters

In [ ]:
# old feature transformations
"""
dt["education"] = dt["education"]**2
dt["income"] = np.sqrt(dt["income"])
dt["StringencyIndex"] = dt["StringencyIndex"]**2
# dt["obesity"] = np.exp(dt["obesity"])  # exp obesity gives too high values!
dt["pct_nursing"] = dt["pct_nursing"]**2
dt["political_leaning"] = dt["political_leaning"]**2
dt["pct_jail"] = np.sqrt(dt["pct_jail"])
dt["pct_hispanic"] = dt["pct_hispanic"]**2
dt["min_distance_top_airport"] = np.sqrt(dt["min_distance_top_airport"])
"""

#### Define datasets

In [ ]:
dt_list, X_list, y_list, \
sample_weight_list, groups_list = virus_spread_dt(county_database2_imputed,
                                                  selected_columns_list,
                                                  X_selected_columns_list,
                                                  type_data="COVID-19",
                                                  plot=False)
dt1, dt2, dt3, dt4, dt5, dt6 = dt_list
X1, X2, X3, X4, X5, X6 = X_list
y1, y2, y3, y4, y5, y6 = y_list
sample_weight1, sample_weight2, sample_weight3, \
  sample_weight4, sample_weight5, sample_weight6 = sample_weight_list
groups1, groups2, groups3, groups4, groups5, \
  groups6 = groups_list

#### ElasticNet - Period 1 (February 2020 - May 2020)

In [ ]:
coefs_p1_enet, coefs_all_p1_enet, _, _ = elasticNet_pipeline_period(X1, y1,
                                                                    sample_weight1,
                                                                    "1")

#### ElasticNet - Period 2 (June 2020 - October 2020)

In [ ]:
coefs_p2_enet, coefs_all_p2_enet, _, _ = elasticNet_pipeline_period(X2, y2,
                                                                    sample_weight2,
                                                                    "2")

#### ElasticNet - Period 3 (November 2020 - February 2021)

In [ ]:
coefs_p3_enet, coefs_all_p3_enet, _, _ = elasticNet_pipeline_period(X3, y3,
                                                                    sample_weight3,
                                                                    "3")

#### ElasticNet - Period 4 (March 2021 - July 2021)

In [ ]:
coefs_p4_enet, coefs_all_p4_enet, _, _ = elasticNet_pipeline_period(X4, y4,
                                                                    sample_weight4,
                                                                    "4")

#### ElasticNet - Period 5 (August 2021 - October 2021)

In [ ]:
coefs_p5_enet, coefs_all_p5_enet, _, _ = elasticNet_pipeline_period(X5, y5,
                                                                    sample_weight5,
                                                                    "5")

#### ElasticNet - Period 6 (November 2021 - February 2022)

In [ ]:
coefs_p6_enet, coefs_all_p6_enet, _, _ = elasticNet_pipeline_period(X6, y6,
                                                                    sample_weight6,
                                                                    "6")

#### ElasticNet - All periods contributions

In [ ]:
import plotly.graph_objects as go
feature_names = list(X1.columns)

intervals1 = (1.96 * coefs_all_p1_enet.std() / np.sqrt(len(coefs_all_p1_enet))).to_frame()
intervals2 = (1.96 * coefs_all_p2_enet.std() / np.sqrt(len(coefs_all_p2_enet))).to_frame()
intervals3 = (1.96 * coefs_all_p3_enet.std() / np.sqrt(len(coefs_all_p3_enet))).to_frame()
intervals4 = (1.96 * coefs_all_p4_enet.std() / np.sqrt(len(coefs_all_p4_enet))).to_frame()
intervals5 = (1.96 * coefs_all_p5_enet.std() / np.sqrt(len(coefs_all_p5_enet))).to_frame()
intervals6 = (1.96 * coefs_all_p6_enet.std() / np.sqrt(len(coefs_all_p6_enet))).to_frame()
period1_data = []
period2_data = []
period3_data = []
period4_data = []
period5_data = []
period6_data = []
error_p1 = []
error_p2 = []
error_p3 = []
error_p4 = []
error_p5 = []
error_p6 = []
for i, feat in enumerate(feature_names):
  p1 = coefs_p1_enet.loc[feat].iloc[0]
  p2 = coefs_p2_enet.loc[feat].iloc[0]
  p3 = coefs_p3_enet.loc[feat].iloc[0]
  p4 = coefs_p4_enet.loc[feat].iloc[0]
  p5 = coefs_p5_enet.loc[feat].iloc[0]
  p6 = coefs_p6_enet.loc[feat].iloc[0]
  e1 = intervals1.loc[feat].iloc[0]
  e2 = intervals2.loc[feat].iloc[0]
  e3 = intervals3.loc[feat].iloc[0]
  e4 = intervals4.loc[feat].iloc[0]
  e5 = intervals5.loc[feat].iloc[0]
  e6 = intervals6.loc[feat].iloc[0]
  period1_data.append(p1)
  period2_data.append(p2)
  period3_data.append(p3)
  period4_data.append(p4)
  period5_data.append(p5)
  period6_data.append(p6)
  error_p1.append(e1)
  error_p2.append(e2)
  error_p3.append(e3)
  error_p4.append(e4)
  error_p5.append(e5)
  error_p6.append(e6)
fig = go.Figure(data=[
    go.Bar(name="Period 1 (February 2020 - May 2020)", x=feature_names, y=period1_data,
           error_y=dict(type="data", array=error_p1)),
    go.Bar(name="Period 2 (June 2020 - October 2020)", x=feature_names, y=period2_data,
           error_y=dict(type="data", array=error_p2)),
    go.Bar(name="Period 3 (November 2020 - February 2021)", x=feature_names, y=period3_data,
           error_y=dict(type="data", array=error_p3)),
    go.Bar(name="Period 4 (March 2021 - July 2021)", x=feature_names, y=period4_data,
           error_y=dict(type="data", array=error_p4)),
    go.Bar(name="Period 5 (August 2021 - October 2021)", x=feature_names, y=period5_data,
           error_y=dict(type="data", array=error_p5)),
    go.Bar(name="Period 6 (November 2021 - February 2022)", x=feature_names, y=period6_data,
           error_y=dict(type="data", array=error_p6))
])
# Change the bar mode
fig.update_layout(barmode="group",
                  xaxis_title="Features",
                  yaxis_title="Coefficients importance and its variability",
                  legend_title="Periods",
                  title="Coefficients importance and its variability (95% confidence interval)<br> for each period (MinMaxScaler & ElasticNet)")
fig.update_xaxes(tickangle=-45)
fig.show()

In [ ]:
directionality = pd.DataFrame()
directionality["Variable"] = feature_names
directionality["Period 1"] = period1_data
directionality["Period 2"] = period2_data
directionality["Period 3"] = period3_data
directionality["Period 4"] = period4_data
directionality["Period 5"] = period5_data
directionality["Period 6"] = period6_data
for i in range(1, 7):
  directionality[f"Period {i}"] = directionality[f"Period {i}"].apply(lambda x: "+" if x >= 0 else "-")
directionality

#### Heatmap

##### Heatmap coefficients

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp

allCoefficients = pd.DataFrame()
allCoefficients["Period 1"] = period1_data
allCoefficients["Period 2"] = period2_data
allCoefficients["Period 3"] = period3_data
allCoefficients["Period 4"] = period4_data
allCoefficients["Period 5"] = period5_data
allCoefficients["Period 6"] = period6_data
allCoefficients.index = feature_names

rows = (len(list(allCoefficients.index)) // 10) + 1
fig = sp.make_subplots(rows=rows, cols=1)

for row in range(rows):
  fig.add_trace(go.Heatmap(z=allCoefficients[10*row:10*(row+1)].T.to_numpy(),
                           x=list(allCoefficients.index)[10*row:10*(row+1)],
                           y=["Period 1", "Period 2", "Period 3", "Period 4",
                              "Period 5", "Period 6"],
                           colorscale="RdBu_r",
                           zmid=0),
                row=(row+1), col=1)

fig.update_xaxes(tickangle=-45)
fig.update_layout(height=rows*600, width=900,
                  title_text=f"Coefficients for each features for each periods",
                  xaxis_title="Features",
                  yaxis_title="Periods")
fig.show()

##### Heatmap rank

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp

allRanks = pd.DataFrame()
allRanks["Period 1"] = 1 + (np.array(period1_data).argsort().argsort())
allRanks["Period 2"] = 1 + (np.array(period2_data).argsort().argsort())
allRanks["Period 3"] = 1 + (np.array(period3_data).argsort().argsort())
allRanks["Period 4"] = 1 + (np.array(period4_data).argsort().argsort())
allRanks["Period 5"] = 1 + (np.array(period5_data).argsort().argsort())
allRanks["Period 6"] = 1 + (np.array(period6_data).argsort().argsort())
allRanks.index = feature_names

rows = (len(list(allRanks.index)) // 10) + 1
fig = sp.make_subplots(rows=rows, cols=1)

for row in range(rows):
  fig.add_trace(go.Heatmap(z=allRanks[10*row:10*(row+1)].T.to_numpy(),
                           x=list(allRanks.index)[10*row:10*(row+1)],
                           y=["Period 1", "Period 2", "Period 3", "Period 4",
                              "Period 5", "Period 6"],
                           colorscale="RdBu",
                           zmid=len(allRanks)//2),
                row=(row+1), col=1)

fig.update_xaxes(tickangle=-45)
fig.update_layout(height=rows*600, width=900,
                  title_text=f"Rank for each features for each periods (high rank=lower value)",
                  xaxis_title="Features",
                  yaxis_title="Periods")
fig.show()

### MinMaxScaler & confidence intervals (RandomForest) (Dataset 2)

#### Define Datasets

In [ ]:
dt_list, X_list, y_list, \
sample_weight_list, groups_list = virus_spread_dt(county_database2_imputed,
                                                  selected_columns_list,
                                                  X_selected_columns_list,
                                                  type_data="COVID-19",
                                                  plot=False)
dt1, dt2, dt3, dt4, dt5, dt6 = dt_list
X1, X2, X3, X4, X5, X6 = X_list
y1, y2, y3, y4, y5, y6 = y_list
sample_weight1, sample_weight2, sample_weight3, \
  sample_weight4, sample_weight5, sample_weight6 = sample_weight_list
groups1, groups2, groups3, groups4, groups5, \
  groups6 = groups_list

#### RandomForest - Period 1 (February 2020 - May 2020)

In [ ]:
importances1, _ = randomForest_pipeline_period(X1, y1, sample_weight1, "1")

#### RandomForest - Period 2 (June 2020 - October 2020)

In [ ]:
importances2, _ = randomForest_pipeline_period(X2, y2, sample_weight2, "2")

#### RandomForest - Period 3 (November 2020 - February 2021)

In [ ]:
importances3, _ = randomForest_pipeline_period(X3, y3, sample_weight3, "3")

#### RandomForest - Period 4 (March 2021 - July 2021)

In [ ]:
importances4, _ = randomForest_pipeline_period(X4, y4, sample_weight4, "4")

#### RandomForest - Period 5 (August 2021 - October 2021)

In [ ]:
importances5, _ = randomForest_pipeline_period(X5, y5, sample_weight5, "5")

#### RandomForest - Period 6 (November 2021 - February 2022)

In [ ]:
importances6, _ = randomForest_pipeline_period(X6, y6, sample_weight6, "6")

#### RandomForest - All periods contributions

In [ ]:
import plotly.graph_objects as go
feature_names = list(X1.columns)

period1_data = []
period2_data = []
period3_data = []
period4_data = []
period5_data = []
period6_data = []
error_p1 = []
error_p2 = []
error_p3 = []
error_p4 = []
error_p5 = []
error_p6 = []
for i, feat in enumerate(feature_names):
  p1 = importances1.loc[feat]["Feature importances using MDI - Virus Spread Period 1 (RandomForestRegressor)\n95% confidence interval"]
  p2 = importances2.loc[feat]["Feature importances using MDI - Virus Spread Period 2 (RandomForestRegressor)\n95% confidence interval"]
  p3 = importances3.loc[feat]["Feature importances using MDI - Virus Spread Period 3 (RandomForestRegressor)\n95% confidence interval"]
  p4 = importances4.loc[feat]["Feature importances using MDI - Virus Spread Period 4 (RandomForestRegressor)\n95% confidence interval"]
  p5 = importances5.loc[feat]["Feature importances using MDI - Virus Spread Period 5 (RandomForestRegressor)\n95% confidence interval"]
  p6 = importances6.loc[feat]["Feature importances using MDI - Virus Spread Period 6 (RandomForestRegressor)\n95% confidence interval"]
  e1 = importances1.loc[feat]["errors"]
  e2 = importances2.loc[feat]["errors"]
  e3 = importances3.loc[feat]["errors"]
  e4 = importances4.loc[feat]["errors"]
  e5 = importances5.loc[feat]["errors"]
  e6 = importances6.loc[feat]["errors"]
  period1_data.append(p1)
  period2_data.append(p2)
  period3_data.append(p3)
  period4_data.append(p4)
  period5_data.append(p5)
  period6_data.append(p6)
  error_p1.append(e1)
  error_p2.append(e2)
  error_p3.append(e3)
  error_p4.append(e4)
  error_p5.append(e5)
  error_p6.append(e6)
fig = go.Figure(data=[
    go.Bar(name="Period 1 (February 2020 - May 2020)", x=feature_names, y=period1_data,
           error_y=dict(type="data", array=error_p1)),
    go.Bar(name="Period 2 (June 2020 - October 2020)", x=feature_names, y=period2_data,
           error_y=dict(type="data", array=error_p2)),
    go.Bar(name="Period 3 (November 2020 - February 2021)", x=feature_names, y=period3_data,
           error_y=dict(type="data", array=error_p3)),
    go.Bar(name="Period 4 (March 2021 - July 2021)", x=feature_names, y=period4_data,
           error_y=dict(type="data", array=error_p4)),
    go.Bar(name="Period 5 (August 2021 - October 2021)", x=feature_names, y=period5_data,
           error_y=dict(type="data", array=error_p5)),
    go.Bar(name="Period 6 (November 2021 - February 2022)", x=feature_names, y=period6_data,
           error_y=dict(type="data", array=error_p6))
])
# Change the bar mode
fig.update_layout(barmode="group",
                  xaxis_title="Features",
                  yaxis_title="Feature importance and its variability (across all trees)",
                  legend_title="Periods",
                  title="Coefficients importance and its variability (across all trees) (95% confidence interval)<br> for each period (MinMaxScaler & RandomForestRegressor)")
fig.show()

## Effect of imputing data, sample weights ...

#### Regression

##### Functions

In [ ]:
import plotly.graph_objects as go


def effect_state_stratify(period=1, type_data="COVID-19", impute=True,
                          use_sample_weight=True,
                          model_hyperparameters=None):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]

  # Stratify shuffle split
  dt_list, X_list, y_list, \
  sample_weight_list, groups_list = virus_spread_dt(county_db,
                                                    selected_columns_list,
                                                    X_selected_columns_list,
                                                    type_data,
                                                    plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_list[period-1]))])

  if use_sample_weight:
    weights = sample_weight_list[period-1]
  else:
    weights = all_equal_weights

  try:
    _, _, _, cv_model_stratify = elasticNet_pipeline_period(X_list[period-1],
                                                            y_list[period-1],
                                                            weights,
                                                            f"{period}",
                                                            groups=groups_list[period-1],
                                                            plot=False,
                                                            model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_stratify = elasticNet_pipeline_period(X_list[period-1],
                                                            y_list[period-1],
                                                            weights,
                                                            f"{period}",
                                                            groups=None,
                                                            plot=False,
                                                            model_hyperparameters=model_hyperparameters)
  # No stratification
  _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                 y_list[period-1],
                                                 weights,
                                                 f"{period}",
                                                 groups=None,
                                                 plot=False,
                                                 model_hyperparameters=model_hyperparameters)
  # Evaluate and compare the performances (MAPE) of the two models

  mape = -np.nanmean(cv_model["test_score"])
  mape_stratify = -np.nanmean(cv_model_stratify["test_score"])
  mape_err = 1.96 * np.nanstd(cv_model["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model["test_score"])))
  mape_stratify_err = 1.96 * np.nanstd(cv_model_stratify["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model_stratify["test_score"])))
  return mape, mape_stratify, mape_err, mape_stratify_err

def effect_imputing_data(period=1, type_data="COVID-19", stratify=True,
                         use_sample_weight=True,
                         model_hyperparameters=None):

  county_db = county_databases[type_data]["county_database2_imputed"]

  # Impute data
  dt_list_imp, X_list_imp, y_list_imp, \
  sample_weight_list_imp, groups_list_imp = virus_spread_dt(county_db,
                                                            selected_columns_list,
                                                            X_selected_columns_list,
                                                            type_data,
                                                            plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_list_imp[period-1]))])

  if use_sample_weight:
    weights_imputed = sample_weight_list_imp[period-1]
  else:
    weights_imputed = all_equal_weights
  
  if stratify:
    groups = groups_list_imp[period-1]
  else:
    groups = None

  try:
    _, _, _, cv_model_imputed = elasticNet_pipeline_period(X_list_imp[period-1],
                                                           y_list_imp[period-1],
                                                           weights_imputed,
                                                           f"{period}",
                                                           groups=groups,
                                                           plot=False,
                                                           model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_imputed = elasticNet_pipeline_period(X_list_imp[period-1],
                                                           y_list_imp[period-1],
                                                           weights_imputed,
                                                           f"{period}",
                                                           groups=None,
                                                           plot=False,
                                                           model_hyperparameters=model_hyperparameters)
  # Regular data

  county_db = county_databases[type_data]["county_database2"]

  dt_list, X_list, y_list, \
  sample_weight_list, groups_list = virus_spread_dt(county_db,
                                                    selected_columns_list,
                                                    X_selected_columns_list,
                                                    type_data,
                                                    plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_list[period-1]))])

  if use_sample_weight:
    weights = sample_weight_list[period-1]
  else:
    weights = all_equal_weights
  
  if stratify:
    groups = groups_list[period-1]
  else:
    groups = None

  try:
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   weights,
                                                   f"{period}",
                                                   groups=groups,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   weights,
                                                   f"{period}",
                                                   groups=None,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)

  # Evaluate and compare the performances (MAPE) of the two models
  mape = -np.nanmean(cv_model["test_score"])
  mape_imputed = -np.nanmean(cv_model_imputed["test_score"])
  mape_err = 1.96 * np.nanstd(cv_model["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model["test_score"])))
  mape_imputed_err = 1.96 * np.nanstd(cv_model_imputed["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model_imputed["test_score"])))
  return mape, mape_imputed, mape_err, mape_imputed_err


def effect_sample_weight(period=1, type_data="COVID-19", impute=True,
                         stratify=True,
                         model_hyperparameters=None):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]

  # Sample weight
  dt_list, X_list, y_list, \
  sample_weight_list, groups_list = virus_spread_dt(county_db,
                                                    selected_columns_list,
                                                    X_selected_columns_list,
                                                    type_data,
                                                    plot=False)
  
  if stratify:
    groups = groups_list[period-1]
  else:
    groups = None

  try:
    _, _, _, cv_model_sample_weight = elasticNet_pipeline_period(X_list[period-1],
                                                                 y_list[period-1],
                                                                 sample_weight_list[period-1],
                                                                 f"{period}",
                                                                 groups=groups,
                                                                 plot=False,
                                                                 model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_sample_weight = elasticNet_pipeline_period(X_list[period-1],
                                                                 y_list[period-1],
                                                                 sample_weight_list[period-1],
                                                                 f"{period}",
                                                                 groups=None,
                                                                 plot=False,
                                                                 model_hyperparameters=model_hyperparameters)

  # No sample weight (only 1)
  dt_list, X_list, y_list, \
  sample_weight_list, groups_list = virus_spread_dt(county_db,
                                                    selected_columns_list,
                                                    X_selected_columns_list,
                                                    type_data,
                                                    plot=False)
  if stratify:
    groups = groups_list[period-1]
  else:
    groups = None

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_list[period-1]))])
  
  try:
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   all_equal_weights,
                                                   f"{period}",
                                                   groups=groups,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   all_equal_weights,
                                                   f"{period}",
                                                   groups=None,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)

  # Evaluate and compare the performances (MAPE) of the two models
  mape = -np.nanmean(cv_model["test_score"])
  mape_sample_weight = -np.nanmean(cv_model_sample_weight["test_score"])
  mape_err = 1.96 * np.nanstd(cv_model["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model["test_score"])))
  mape_sample_weight_err = 1.96 * np.nanstd(cv_model_sample_weight["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model_sample_weight["test_score"])))
  return mape, mape_sample_weight, mape_err, mape_sample_weight_err

def effect_correlated_features(period=1, type_data="COVID-19", impute=True,
                               stratify=True, use_sample_weight=True,
                               model_hyperparameters=None):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]
  
  # Remove correlated features
  nocorr_dt_list, nocorr_X_list, nocorr_y_list, \
  nocorr_sample_weight_list, nocorr_groups_list = virus_spread_dt(county_db,
                                                                  selected_columns_list,
                                                                  X_selected_columns_list,
                                                                  type_data,
                                                                  plot=False)

  if stratify:
    nocorr_groups = nocorr_groups_list[period-1]
  else:
    nocorr_groups = None

  all_equal_weights = np.array([1 for _ in range(len(nocorr_sample_weight_list[period-1]))])

  if use_sample_weight:
    nocorr_weights = nocorr_sample_weight_list[period-1]
  else:
    nocorr_weights = all_equal_weights

  try:
    _, _, _, cv_model_nocorr = elasticNet_pipeline_period(nocorr_X_list[period-1],
                                                          nocorr_y_list[period-1],
                                                          nocorr_weights,
                                                          f"{period}",
                                                          groups=nocorr_groups,
                                                          plot=False,
                                                          model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_nocorr = elasticNet_pipeline_period(nocorr_X_list[period-1],
                                                          nocorr_y_list[period-1],
                                                          nocorr_weights,
                                                          f"{period}",
                                                          groups=None,
                                                          plot=False,
                                                          model_hyperparameters=model_hyperparameters)
  
  # Keep all numerical features (without response variable etc)
  all_columns = list(county_database2_imputed.select_dtypes(["number"]).columns)
  all_columns.remove("FIPS")
  X_all_columns = all_columns.copy()
  for c in ["total_pop", "county_area"]:
    X_all_columns.remove(c)
  for per in range(1, 7):
    X_all_columns.remove(f"deathCount_period{per}")
    X_all_columns.remove(f"deathRate_period{per}")
  
  stringency_columns = [f"StringencyIndex{i}_{s}" for i in range(period+1, 7) for s in ["mean", "std", "median", "min", "max"]]
  for c in stringency_columns:
    X_all_columns.remove(c)
  if period != 1:  # drop distance to airport
    X_all_columns.remove("min_distance_top_airport")

  # convert to a list of all_columns
  all_columns_list = [all_columns for _ in range(7)]
  X_all_columns_list = [X_all_columns for _ in range(7)]

  dt_list, X_list, y_list, \
  sample_weight_list, groups_list = virus_spread_dt(county_db,
                                                    all_columns_list,
                                                    X_all_columns_list,
                                                    type_data,
                                                    plot=False)
  if stratify:
    groups = groups_list[period-1]
  else:
    groups = None

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_list[period-1]))])

  if use_sample_weight:
    weights = sample_weight_list[period-1]
  else:
    weights = all_equal_weights

  try:
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   weights,
                                                   f"{period}",
                                                   groups=groups,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = elasticNet_pipeline_period(X_list[period-1],
                                                   y_list[period-1],
                                                   weights,
                                                   f"{period}",
                                                   groups=None,
                                                   plot=False,
                                                   model_hyperparameters=model_hyperparameters)

  # Evaluate and compare the performances (MAPE) of the two models
  mape = -np.nanmean(cv_model["test_score"])
  mape_nocorr = -np.nanmean(cv_model_nocorr["test_score"])
  mape_err = 1.96 * np.nanstd(cv_model["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model["test_score"])))
  mape_nocorr_err = 1.96 * np.nanstd(cv_model_nocorr["test_score"]) / np.sqrt(np.sum(~np.isnan(cv_model_nocorr["test_score"])))
  return mape, mape_nocorr, mape_err, mape_nocorr_err

##### Select one set of hyperparameters (to compare only one effect across all periods)

In [ ]:
model_hyperparameters = {"elasticnet__alpha": 0.05,
                         "elasticnet__l1_ratio": 1}

##### Effect of stratify (represent all states in each fold) during cross validation

In [ ]:
# Parameters
type_data = "COVID-19"
use_sample_weight = True
impute = True

# Compute the metrics for all the periods
mape_list = []
mape_err_list = []
mape_stratify_list = []
mape_stratify_err_list = []
for period in tqdm(range(1, 7)):
  mape, mape_stratify, \
    mape_err, mape_stratify_err = effect_state_stratify(period=period,
                                                        type_data=type_data,
                                                        use_sample_weight=use_sample_weight,
                                                        impute=impute,
                                                        model_hyperparameters=model_hyperparameters)
  mape_list.append(mape)
  mape_stratify_list.append(mape_stratify)
  mape_err_list.append(mape_err)
  mape_stratify_err_list.append(mape_stratify_err)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="Normal Shuffle Split", x=period_list, y=mape_list,
           error_y=dict(type="data", array=mape_err_list)),
    go.Bar(name="Stratified Shuffle Split on the state variable", x=period_list,
           y=mape_stratify_list,
           error_y=dict(type="data", array=mape_stratify_err_list))
])
# Change the bar mode
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
impute_text = "Impute at state level" if impute else "No imputation"
fig.update_layout(barmode="group",
                  xaxis_title="Periods",
                  yaxis_title="Mean Average Percentage Error (MAPE)",
                  legend_title="No stratification/State stratification",
                  title=f"MAPE difference for each period: dataset with and without stratified shuffle split on the state<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & ElasticNet) (95% confidence interval)<br>{use_sample_weight_text}, {impute_text}, {type_data} crude death rate per 100k")
fig.show()

##### Effect of imputing data at the state level

In [ ]:
# Parameters
type_data = "COVID-19"
use_sample_weight = True
stratify = True

# Compute the metrics for all the periods
mape_list = []
mape_imputed_list = []
mape_err_list = []
mape_imputed_err_list = []
for period in tqdm(range(1, 7)):
  mape, mape_imputed, \
    mape_err, mape_imputed_err = effect_imputing_data(period=period,
                                                      type_data=type_data,
                                                      stratify=stratify,
                                                      use_sample_weight=use_sample_weight,
                                                      model_hyperparameters=model_hyperparameters)
  mape_list.append(mape)
  mape_imputed_list.append(mape_imputed)
  mape_err_list.append(mape_err)
  mape_imputed_err_list.append(mape_imputed_err)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="Normal data", x=period_list, y=mape_list,
           error_y=dict(type="data", array=mape_err_list)),
    go.Bar(name="Imputed data", x=period_list, y=mape_imputed_list,
           error_y=dict(type="data", array=mape_imputed_err_list))
])
# Change the bar mode
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
stratify_text = "State stratification" if impute else "No stratification"
fig.update_layout(barmode="group",
                  xaxis_title="Periods",
                  yaxis_title="Mean Average Percentage Error (MAPE)",
                  legend_title="Not Imputed/Imputed",
                  title=f"MAPE difference for each period: dataset before and after imputation<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & ElasticNet) (95% confidence interval)<br>{use_sample_weight_text}, {stratify_text}, {type_data} crude death rate per 100k")
fig.show()

##### Effect of adding sample weights

In [ ]:
# Parameters
type_data = "COVID-19"
impute = True
stratify = True

# Compute the metrics for all the periods
mape_list = []
mape_sample_weight_list = []
mape_err_list = []
mape_sample_weight_err_list = []
for period in tqdm(range(1, 7)):
  mape, mape_sample_weight, \
    mape_err, mape_sample_weight_err = effect_sample_weight(period=period,
                                                            type_data=type_data,
                                                            impute=impute,
                                                            stratify=stratify,
                                                            model_hyperparameters=model_hyperparameters)
  mape_list.append(mape)
  mape_sample_weight_list.append(mape_sample_weight)
  mape_err_list.append(mape_err)
  mape_sample_weight_err_list.append(mape_sample_weight_err)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="No sample weights", x=period_list, y=mape_list,
           error_y=dict(type="data", array=mape_err_list)),
    go.Bar(name="Sample weights", x=period_list, y=mape_sample_weight_list,
           error_y=dict(type="data", array=mape_sample_weight_err_list))
])
# Change the bar mode
impute_text = "Impute at state level" if impute else "No imputation"
stratify_text = "State stratification" if impute else "No stratification"
fig.update_layout(barmode="group",
                  xaxis_title="Periods",
                  yaxis_title="Mean Average Percentage Error (MAPE)",
                  legend_title="No sample weights/Sample weights",
                  title=f"MAPE difference for each period: model with and without sample weights<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & ElasticNet) (95% confidence interval)<br>{impute_text}, {stratify_text}, {type_data} crude death rate per 100k")
fig.show()

##### Effect of removing correlated features

In [ ]:
# Parameters
type_data = "COVID-19"
impute = True
stratify = True
use_sample_weight = True

# Compute the metrics for all the periods
mape_list = []
mape_nocorr_list = []
mape_err_list = []
mape_nocorr_err_list = []
for period in tqdm(range(1, 7)):
  mape, mape_nocorr, \
    mape_err, mape_nocorr_err = effect_correlated_features(period=period,
                                                           type_data=type_data,
                                                           impute=impute,
                                                           stratify=stratify,
                                                           use_sample_weight=use_sample_weight,
                                                           model_hyperparameters=model_hyperparameters)
  mape_list.append(mape)
  mape_nocorr_list.append(mape_nocorr)
  mape_err_list.append(mape_err)
  mape_nocorr_err_list.append(mape_nocorr_err)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="All numerical features", x=period_list, y=mape_list,
           error_y=dict(type="data", array=mape_err_list)),
    go.Bar(name="Remove correlated features", x=period_list, y=mape_nocorr_list,
           error_y=dict(type="data", array=mape_nocorr_err_list))
])
# Change the bar mode
impute_text = "Impute at state level" if impute else "No imputation"
stratify_text = "State stratification" if impute else "No stratification"
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
fig.update_layout(barmode="group",
                  xaxis_title="Periods",
                  yaxis_title="Mean Average Percentage Error (MAPE)",
                  legend_title="All features/No correlated features",
                  title=f"MAPE difference for each period: model fitted with all features/without correlated features<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & ElasticNet) (95% confidence interval)<br>{use_sample_weight_text}, {impute_text}, {stratify_text}, {type_data} crude death rate per 100k")
fig.show()

#### Classification

##### Functions

In [ ]:
import plotly.graph_objects as go

def effect_state_stratify_clf(type_data="COVID-19", impute=True,
                              use_sample_weight=True):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]

  # Stratify shuffle split
  dt, X, y, sample_weight, groups = virus_introduction_all(county_db,
                                                           selected_columns_list,
                                                           X_selected_columns_list,
                                                           plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight))])

  if use_sample_weight:
    weights = sample_weight
  else:
    weights = all_equal_weights

  try:
    _, _, _, cv_model_stratify = lr_pipeline_period(X, y, weights, "All",
                                                    groups=groups,
                                                    plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_stratify = lr_pipeline_period(X, y, weights, "All",
                                                    groups=None,
                                                    plot=False)
  # No stratification
  _, _, _, cv_model = lr_pipeline_period(X, y, weights, "All",
                                         groups=None,
                                         plot=False)
  # Evaluate and compare the performances of the two models
  metrics = {"normal": {}, "stratify": {},
             "normal_err": {}, "stratify_err": {}}
  for m in ["auroc", "balanced_accuracy", "accuracy", "precision",
            "recall", "f1", "auprc", "avg_precision"]:
    metrics["normal"][m] = np.nanmean(cv_model[f"test_{m}"])
    metrics["stratify"][m] = np.nanmean(cv_model_stratify[f"test_{m}"])
    metrics["normal_err"][m] = 1.96 * np.nanstd(cv_model[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model[f"test_{m}"])))
    metrics["stratify_err"][m] = 1.96 * np.nanstd(cv_model_stratify[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model_stratify[f"test_{m}"])))
  return metrics

def effect_imputing_data_clf(type_data="COVID-19", stratify=True,
                             use_sample_weight=True):

  county_db = county_databases[type_data]["county_database2_imputed"]

  # Impute data
  dt, X_imp, y_imp, sample_weight_imp, groups_imp = virus_introduction_all(county_db,
                                                           selected_columns_list,
                                                           X_selected_columns_list,
                                                           plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight_imp))])

  if use_sample_weight:
    weights_imputed = sample_weight_imp
  else:
    weights_imputed = all_equal_weights
  
  if stratify:
    groups = groups_imp
  else:
    groups = None

  try:
    _, _, _, cv_model_imputed = lr_pipeline_period(X_imp, y_imp,
                                                   weights_imputed, "All",
                                                   groups=groups,
                                                   plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_imputed = lr_pipeline_period(X_imp, y_imp,
                                                   weights_imputed, "All",
                                                   groups=None,
                                                   plot=False)
  # Regular data

  county_db = county_databases[type_data]["county_database2"]

  dt, X, y, sample_weight, groups = virus_introduction_all(county_db,
                                                           selected_columns_list,
                                                           X_selected_columns_list,
                                                           plot=False)

  all_equal_weights = np.array([1 for _ in range(len(sample_weight))])

  if use_sample_weight:
    weights = sample_weight
  else:
    weights = all_equal_weights
  
  if stratify:
    groups = groups
  else:
    groups = None

  try:
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           weights, "All",
                                           groups=groups,
                                           plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           weights, "All",
                                           groups=None,
                                           plot=False)

  # Evaluate and compare the performances of the two models
  metrics = {"normal": {}, "imputed": {},
             "normal_err": {}, "imputed_err": {}}
  for m in ["auroc", "balanced_accuracy", "accuracy", "precision",
            "recall", "f1", "auprc", "avg_precision"]:
    metrics["normal"][m] = np.nanmean(cv_model[f"test_{m}"])
    metrics["imputed"][m] = np.nanmean(cv_model_imputed[f"test_{m}"])
    metrics["normal_err"][m] = 1.96 * np.nanstd(cv_model[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model[f"test_{m}"])))
    metrics["imputed_err"][m] = 1.96 * np.nanstd(cv_model_imputed[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model_imputed[f"test_{m}"])))
  return metrics


def effect_sample_weight_clf(type_data="COVID-19", impute=True,
                             stratify=True):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]

  # Sample weight
  dt, X, y, sample_weight, groups = virus_introduction_all(county_db,
                                                           selected_columns_list,
                                                           X_selected_columns_list,
                                                           plot=False)
  
  if stratify:
    groups = groups
  else:
    groups = None

  try:
    _, _, _, cv_model_sample_weight = lr_pipeline_period(X, y,
                                                         sample_weight, "All",
                                                         groups=groups,
                                                         plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_sample_weight = lr_pipeline_period(X, y,
                                                         sample_weight, "All",
                                                         groups=None,
                                                         plot=False)

  # No sample weight (only 1)
  all_equal_weights = np.array([1 for _ in range(len(sample_weight))])
  
  try:
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           all_equal_weights, "All",
                                           groups=groups,
                                           plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           all_equal_weights, "All",
                                           groups=None,
                                           plot=False)

  # Evaluate and compare the performances of the two models
  metrics = {"normal": {}, "sample_weight": {},
             "normal_err": {}, "sample_weight_err": {}}
  for m in ["auroc", "balanced_accuracy", "accuracy", "precision",
            "recall", "f1", "auprc", "avg_precision"]:
    metrics["normal"][m] = np.nanmean(cv_model[f"test_{m}"])
    metrics["sample_weight"][m] = np.nanmean(cv_model_sample_weight[f"test_{m}"])
    metrics["normal_err"][m] = 1.96 * np.nanstd(cv_model[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model[f"test_{m}"])))
    metrics["sample_weight_err"][m] = 1.96 * np.nanstd(cv_model_sample_weight[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model_sample_weight[f"test_{m}"])))
  return metrics

def effect_correlated_features_clf(type_data="COVID-19", impute=True,
                                   stratify=True, use_sample_weight=True):

  if impute:
    county_db = county_databases[type_data]["county_database2_imputed"]
  else:
    county_db = county_databases[type_data]["county_database2"]
  
  # Remove correlated features
  nocorr_dt, nocorr_X, nocorr_y, nocorr_sample_weight, nocorr_groups = virus_introduction_all(county_db,
                                                                                              selected_columns_list,
                                                                                              X_selected_columns_list,
                                                                                              plot=False)

  if stratify:
    nocorr_groups = nocorr_groups
  else:
    nocorr_groups = None

  all_equal_weights = np.array([1 for _ in range(len(nocorr_sample_weight))])

  if use_sample_weight:
    nocorr_weights = nocorr_sample_weight
  else:
    nocorr_weights = all_equal_weights

  try:
    _, _, _, cv_model_nocorr = lr_pipeline_period(nocorr_X, nocorr_y,
                                                  nocorr_weights, "All",
                                                  groups=nocorr_groups,
                                                  plot=False)

  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model_nocorr = lr_pipeline_period(nocorr_X, nocorr_y,
                                                  nocorr_weights, "All",
                                                  groups=None,
                                                  plot=False)
  
  # Keep all numerical features (without response variable etc)
  all_columns = list(county_database2_imputed.select_dtypes(["number"]).columns)
  all_columns.remove("FIPS")
  X_all_columns = all_columns.copy()
  for c in ["total_pop", "county_area"]:
    X_all_columns.remove(c)
  for per in range(1, 7):
    X_all_columns.remove(f"deathCount_period{per}")
    X_all_columns.remove(f"deathRate_period{per}")
  
  stringency_columns = [f"StringencyIndex{i}_{s}" for i in range(period+1, 7) for s in ["mean", "std", "median", "min", "max"]]
  for c in stringency_columns:
    X_all_columns.remove(c)
  if period != 1:  # drop distance to airport
    X_all_columns.remove("min_distance_top_airport")

  # convert to a list of all_columns
  all_columns_list = [all_columns for _ in range(7)]
  X_all_columns_list = [X_all_columns for _ in range(7)]

  dt, X, y, sample_weight, groups = virus_introduction_all(county_db,
                                                           all_columns_list,
                                                           X_all_columns_list,
                                                           plot=False)

  if stratify:
    groups = groups
  else:
    groups = None

  all_equal_weights = np.array([1 for _ in range(len(sample_weight))])

  if use_sample_weight:
    weights = sample_weight
  else:
    weights = all_equal_weights

  try:
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           weights, "All",
                                           groups=groups,
                                           plot=False)
  except ValueError as e:
    print(e)
    print(f"Can't stratify because some states represented in the dataset don't contains enough counties (period={period})")
    _, _, _, cv_model = lr_pipeline_period(X, y,
                                           weights, "All",
                                           groups=None,
                                           plot=False)

  # Evaluate and compare the performances (MAPE) of the two models
  metrics = {"normal": {}, "nocorr": {},
             "normal_err": {}, "nocorr_err": {}}
  for m in ["auroc", "balanced_accuracy", "accuracy", "precision",
            "recall", "f1", "auprc", "avg_precision"]:
    metrics["normal"][m] = np.nanmean(cv_model[f"test_{m}"])
    metrics["nocorr"][m] = np.nanmean(cv_model_nocorr[f"test_{m}"])
    metrics["normal_err"][m] = 1.96 * np.nanstd(cv_model[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model[f"test_{m}"])))
    metrics["nocorr_err"][m] = 1.96 * np.nanstd(cv_model_nocorr[f"test_{m}"]) / np.sqrt(np.sum(~np.isnan(cv_model_nocorr[f"test_{m}"])))
  return metrics

##### Effect of stratify (represent all states in each fold) during cross validation

In [ ]:
# Parameters
type_data = "COVID-19"
use_sample_weight = True
impute = True

# Compute the metrics
metrics = effect_state_stratify_clf(type_data=type_data,
                                    use_sample_weight=use_sample_weight,
                                    impute=impute)

fig = go.Figure(data=[
    go.Bar(name="Normal Shuffle Split", x=list(metrics["normal"].keys()),
           y=list(metrics["normal"].values()),
           error_y=dict(type="data", array=list(metrics["normal_err"].values()))),
    go.Bar(name="Stratified Shuffle Split on the state variable",
           x=list(metrics["stratify"].keys()),
           y=list(metrics["stratify"].values()),
           error_y=dict(type="data", array=list(metrics["stratify_err"].values())))
])
# Change the bar mode
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
impute_text = "Impute at state level" if impute else "No imputation"
fig.update_layout(barmode="group",
                  xaxis_title="Metric name",
                  yaxis_title="Value",
                  legend_title="No stratification/State stratification",
                  title=f"Classification metrics: dataset with and without stratified shuffle split on the state<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & LogisticRegression) (95% confidence interval)<br>{use_sample_weight_text}, {impute_text}, {type_data} introduction")
fig.show()

##### Effect of imputing data at the state level

In [ ]:
# Parameters
type_data = "COVID-19"
use_sample_weight = True
stratify = True

# Compute the metrics
metrics = effect_imputing_data_clf(type_data=type_data,
                                   stratify=stratify,
                                   use_sample_weight=use_sample_weight)

fig = go.Figure(data=[
    go.Bar(name="Normal data", x=list(metrics["normal"].keys()),
           y=list(metrics["normal"].values()),
           error_y=dict(type="data", array=list(metrics["normal_err"].values()))),
    go.Bar(name="Imputed data", x=list(metrics["imputed"].keys()),
           y=list(metrics["imputed"].values()),
           error_y=dict(type="data", array=list(metrics["stratify_err"].values())))
])
# Change the bar mode
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
stratify_text = "State stratification" if impute else "No stratification"
fig.update_layout(barmode="group",
                  xaxis_title="Metric name",
                  yaxis_title="Value",
                  legend_title="Not Imputed/Imputed",
                  title=f"Classification metrics: dataset before and after imputation<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & LogisticRegression) (95% confidence interval)<br>{use_sample_weight_text}, {stratify_text}, {type_data} introduction")
fig.show()

##### Effect of adding sample weights

In [ ]:
# Parameters
type_data = "COVID-19"
impute = True
stratify = True

# Compute the metrics
metrics = effect_sample_weight_clf(type_data=type_data,
                                   impute=impute,
                                   stratify=stratify)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="No sample weights", x=list(metrics["normal"].keys()),
           y=list(metrics["normal"].values()),
           error_y=dict(type="data", array=list(metrics["normal_err"].values()))),
    go.Bar(name="Sample weights", x=list(metrics["sample_weight"].keys()),
           y=list(metrics["sample_weight"].values()),
           error_y=dict(type="data", array=list(metrics["sample_weight_err"].values())))
])
# Change the bar mode
impute_text = "Impute at state level" if impute else "No imputation"
stratify_text = "State stratification" if impute else "No stratification"
fig.update_layout(barmode="group",
                  xaxis_title="Metric name",
                  yaxis_title="Value",
                  legend_title="No sample weights/Sample weights",
                  title=f"Classification metrics: model with and without sample weights<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & LogisticRegression) (95% confidence interval)<br>{impute_text}, {stratify_text}, {type_data} introduction")
fig.show()

##### Effect of removing correlated features

In [ ]:
# Parameters
type_data = "COVID-19"
impute = True
stratify = True
use_sample_weight = True

# Compute the metrics
metrics = effect_correlated_features_clf(type_data=type_data,
                                         impute=impute,
                                         stratify=stratify,
                                         use_sample_weight=use_sample_weight)

period_list = [f"Period {i}" for i in range(1, 7)]
fig = go.Figure(data=[
    go.Bar(name="All numerical features", x=list(metrics["normal"].keys()),
           y=list(metrics["normal"].values()),
           error_y=dict(type="data", array=list(metrics["normal_err"].values()))),
    go.Bar(name="Remove correlated features", x=list(metrics["nocorr"].keys()),
           y=list(metrics["nocorr"].values()),
           error_y=dict(type="data", array=list(metrics["nocorr_err"].values())))
])
# Change the bar mode
impute_text = "Impute at state level" if impute else "No imputation"
stratify_text = "State stratification" if impute else "No stratification"
use_sample_weight_text = "Use sample weights" if use_sample_weight else "Equal weights"
fig.update_layout(barmode="group",
                  xaxis_title="Metric name",
                  yaxis_title="Value",
                  legend_title="All features/No correlated features",
                  title=f"Classification metrics: model fitted with all features/without correlated features<br>Results of the cross validation after a hyperparameter tuning (MinMaxScaler & LogisticRegression) (95% confidence interval)<br>{use_sample_weight_text}, {impute_text}, {stratify_text}, {type_data} introduction")
fig.show()

## Multiple models comparison (classification & regression)

Let's try more models (for both virus introduction and virus spread). We incorporate the extended dataset and we add the American communities as a feature.

### Intialization

In [ ]:
from sklearn.utils import all_estimators
import xgboost
import lightgbm
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold
from time import perf_counter
from sklearn.metrics import make_scorer, accuracy_score, \
  balanced_accuracy_score, precision_score, recall_score, f1_score, \
  mean_absolute_error, r2_score, mean_squared_error, roc_auc_score, \
  precision_recall_curve, auc, mean_absolute_percentage_error, \
  average_precision_score, pairwise_distances
from sklearn.preprocessing import label_binarize
from sklearn.utils.extmath import softmax


def pr_auc_score(y, y_pred, **kwargs):
  if len(y_pred.shape) == 1:  # binary classification
    precision, recall, _ = precision_recall_curve(y, y_pred, **kwargs)
  else:
    classes = list(range(y_pred.shape[1]))
    if len(classes) == 2:  # binary classification too
        precision, recall, _ = precision_recall_curve(y, y_pred[:,1],
                                                      **kwargs)
    else:  # multiclass
      Y = label_binarize(y, classes=classes)
      precision, recall, _ = precision_recall_curve(Y.ravel(), y_pred.ravel(),
                                                    **kwargs)
  return auc(recall, precision)

def avg_precision(y, y_pred, **kwargs):
  if len(y_pred.shape) == 1:  # binary classification
    return average_precision_score(y, y_pred, average="micro", **kwargs)
  classes = list(range(y_pred.shape[1]))
  if len(classes) == 2:  # binary classification too
    return average_precision_score(y, y_pred[:,1], average="micro", **kwargs)
  # multiclass
  Y = label_binarize(y, classes=classes)
  return average_precision_score(Y, y_pred, average="micro", **kwargs)

custom_scorer_clf = {'accuracy': make_scorer(accuracy_score,
                                             greater_is_better=True),
                     'balanced_accuracy': make_scorer(balanced_accuracy_score,
                                                      greater_is_better=True),
                     'precision': make_scorer(precision_score, average='macro'),
                     'recall': make_scorer(recall_score, average='macro'),
                     'f1': make_scorer(f1_score, average='macro',
                                       greater_is_better=True),
                     'auroc': make_scorer(roc_auc_score, multi_class="ovo",
                                          average="macro",
                                          needs_proba=True,
                                          greater_is_better=True),
                     'auprc': make_scorer(pr_auc_score, needs_proba=True,
                                          greater_is_better=True),
                     'avg_precision': make_scorer(avg_precision,
                                                  needs_proba=True,
                                                  greater_is_better=True)}
def r2_score_adj(y, y_pred, n, p, fit_intercept=True):
    if fit_intercept:
        rsquared = 1 - np.nansum((y - y_pred) ** 2) / np.nansum((y - np.nanmean(y)) ** 2)
        rsquared_adj = 1 - ((n - 1) / (n - p - 1)) * (1 - rsquared)
    else:
        rsquared = 1 - np.nansum((y - y_pred) ** 2) / np.nansum(y ** 2)
        rsquared_adj = 1 - (n / (n - p)) * (1 - rsquared)
    return rsquared_adj

# adj r2 scorer is added in the pipeline (to retrieve the n and p parameters)
# approximation that doesn't take into account the change of n when splitting
# the dataset! But it's ok I guess
custom_scorer_reg = {'r2': make_scorer(r2_score, greater_is_better=True),
                     'rmse': make_scorer(mean_squared_error, squared=True,
                                         greater_is_better=False),
                     'mae': make_scorer(mean_absolute_error,
                                        greater_is_better=False),
                     'mape': make_scorer(mean_absolute_percentage_error,
                                         greater_is_better=False)}

CLASSIFIERS = [est for est in all_estimators(type_filter=["classifier"])]
REGRESSORS = [est for est in all_estimators(type_filter=["regressor"])]

CLASSIFIERS.append(("XGBClassifier", xgboost.XGBClassifier))
CLASSIFIERS.append(("LGBMClassifier", lightgbm.LGBMClassifier))
REGRESSORS.append(("XGBRegressor", xgboost.XGBRegressor))
REGRESSORS.append(("LGBMRegressor", lightgbm.LGBMRegressor))

classifiers_to_remove = ["ClassifierChain", "GaussianProcessClassifier",
                         "MultinomialNB", "MultiOutputClassifier",
                         "NuSVC",
                         "OneVsOneClassifier",
                         "OneVsRestClassifier", "OutputCodeClassifier",
                         "PassiveAggressiveClassifier", "StackingClassifier",
                         "VotingClassifier", "RadiusNeighborsClassifier",
                         "CategoricalNB"]
CLASSIFIERS = [clf for clf in CLASSIFIERS if clf[0] not in classifiers_to_remove]
NO_SAMPLE_WEIGHT_CLASSIFIERS = ["KNeighborsClassifier", "LabelPropagation",
                                "LabelSpreading", "CategoricalNB",
                                "LinearDiscriminantAnalysis", "MLPClassifier",
                                "NearestCentroid",
                                "QuadraticDiscriminantAnalysis",
                                "RadiusNeighborsClassifier"]
NO_PREDICT_PROBA_CLASSIFIERS = ["LinearSVC", "NearestCentroid",
                                "Perceptron", "RidgeClassifier",
                                "RidgeClassifierCV"]
regressors_to_remove = ["CCA", "IsotonicRegression", "GammaRegressor",
                        "MultiOutputRegressor",
                        "MultiTaskElasticNet", "MultiTaskElasticNetCV",
                        "MultiTaskLasso", "MultiTaskLassoCV",
                        "PLSCanonical", "PLSRegression", "QuantileRegressor",
                        "RadiusNeighborsRegressor",
                        "RegressorChain", "StackingRegressor",
                        "VotingRegressor"]
REGRESSORS = [reg for reg in REGRESSORS if reg[0] not in regressors_to_remove]
NO_SAMPLE_WEIGHT_REGRESSORS = ["ARDRegression",
                               "GaussianProcessRegressor",
                               "KNeighborsRegressor", "Lars", "LarsCV",
                               "LassoLars", "LassoLarsCV", "LassoLarsIC",
                               "MLPRegressor", "OrthogonalMatchingPursuit",
                               "OrthogonalMatchingPursuitCV",
                               "PassiveAggressiveRegressor",
                               "TheilSenRegressor"]

In [ ]:
def all_models_clf(models_to_try, no_sample_weight, no_predict_proba,
                   custom_scorer, X, y, sample_weight=None, groups=None):
  Accuracy = []
  B_Accuracy = []
  PRECISION = []
  RECALL = []
  F1 = []
  AUROC = []
  AUPRC = []
  MICRO_AVG_PREC = []
  names = []
  TIME = []

  for name, model in tqdm(models_to_try):
    print(name)
    top = perf_counter()
    if name in no_predict_proba:  # extend the class with a custom predict proba function based on the decision function
      if name == "NearestCentroid":  # no decision function, calculate pairwise distance to clusters instead
        class model2(model):
          def __init__(self):
            super().__init__()
          def predict_proba(self, X):
            distances = pairwise_distances(X, Y=self.centroids_,
                                           metric="euclidean",
                                           n_jobs=-1)
            return softmax(-distances)
      else:
        class model2(model):
          def __init__(self):
            super().__init__()
          def predict_proba(self, X):
            d = self.decision_function(X)
            return softmax(d)
      model = model2
      model_name = "model2"
    else:
      model_name = f"{name.lower()}"

    pipe = make_pipeline(MinMaxScaler(), model())

    model_params = {}
    if "random_state" in model().get_params().keys():
      model_params[f"{model_name}__random_state"] = 42
    if "n_jobs" in model().get_params().keys():
      model_params[f"{model_name}__n_jobs"] = -1
    if "probability" in model().get_params().keys():
      model_params[f"{model_name}__probability"] = True
    if name == "SGDClassifier":  # change the loss to allow multiclass and predict proba (log or modified huber?)
      model_params[f"{model_name}__loss"] = "log"
    if model_params != {}:
      pipe.set_params(**model_params)
    
    if name in no_sample_weight:
      fit_params = {}
    else:
      fit_params = {f"{model_name}__sample_weight": sample_weight}

    if groups is None:
      cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
    else:
      sss = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
      cv = sss.split(X, groups)

    if hasattr(model, "predict_proba"):
      new_custom_scorer = custom_scorer
    else:
      new_custom_scorer = custom_scorer.copy()
      new_custom_scorer.pop("avg_precision")
      new_custom_scorer.pop("auroc")
      new_custom_scorer.pop("auprc")

    try:
      cv_model = cross_validate(
      pipe, X, y, cv=cv,
      return_estimator=False, n_jobs=-1, fit_params=fit_params,
      scoring=new_custom_scorer)
    except ValueError as e:
      print(e)
      print(f"Can't stratify because some states represented in the dataset don't contains enough counties")
      cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
      cv_model = cross_validate(
      pipe, X, y, cv=cv,
      return_estimator=False, n_jobs=-1, fit_params=fit_params,
      scoring=new_custom_scorer)
    except Exception as e:
      print(e)
    
    # add metrics to the lists
    Accuracy.append(np.nanmean(cv_model["test_accuracy"]))
    B_Accuracy.append(np.nanmean(cv_model["test_balanced_accuracy"]))
    PRECISION.append(np.nanmean(cv_model["test_precision"]))
    RECALL.append(np.nanmean(cv_model["test_recall"]))
    F1.append(np.nanmean(cv_model["test_f1"]))
    # nan mean, because when folding, a label can be missing in a fold and cause nan values
    if hasattr(model, "predict_proba"):  # we can predict proba
      auroc_val = np.nanmean(cv_model["test_auroc"])
      auprc_val = np.nanmean(cv_model["test_auprc"])
      micro_avg_prec_val = np.nanmean(cv_model["test_avg_precision"])
    else:
      auroc_val = np.nan
      auprc_val = np.nan
      micro_avg_prec_val = np.nan
    AUROC.append(auroc_val)
    AUPRC.append(auprc_val)
    MICRO_AVG_PREC.append(micro_avg_prec_val)
    names.append(name)
    TIME.append(perf_counter() - top)
  df_models = pd.DataFrame({"Model": names,
                            "Accuracy": Accuracy,
                            "Balanced Accuracy": B_Accuracy,
                            "Recall": RECALL,
                            "Precision": PRECISION,
                            "F1 Score": F1,
                            "AUROC": AUROC,
                            "AUPRC": AUPRC,
                            "Micro avg Precision": MICRO_AVG_PREC,
                            "Time Taken": TIME})
  df_models = df_models.sort_values(by="Balanced Accuracy", ascending=False).set_index("Model")
  return df_models

def all_models_reg(models_to_try, no_sample_weight, custom_scorer, X, y,
                   sample_weight=None, groups=None):
  R2 = []
  Adj_R2 = []
  RMSE = []
  MAE = []
  MAPE = []
  names = []
  TIME = []

  # add adjusted r2 to the custom_scorer
  # we will fix fit_intercept to True
  new_custom_scorer = custom_scorer.copy()
  new_custom_scorer['adj_r2'] = make_scorer(r2_score_adj, n=X.shape[0],
                                            p=X.shape[1], fit_intercept=True,
                                            greater_is_better=True)

  for name, model in tqdm(models_to_try):
    print(name)
    top = perf_counter()
    pipe = make_pipeline(MinMaxScaler(), model())

    model_params = {}
    if "random_state" in model().get_params().keys():
      model_params[f"{name.lower()}__random_state"] = 42
    if "n_jobs" in model().get_params().keys():
      model_params[f"{name.lower()}__n_jobs"] = -1
    if model_params != {}:
      pipe.set_params(**model_params)
    
    if name in no_sample_weight:
      fit_params = {}
    else:
      fit_params = {f"{name.lower()}__sample_weight": sample_weight}

    if groups is None:
      cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
    else:
      sss = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42)
      cv = sss.split(X, groups)

    try:
      cv_model = cross_validate(
      pipe, X, y, cv=cv,
      return_estimator=False, n_jobs=-1, fit_params=fit_params,
      scoring=new_custom_scorer)
    except ValueError as e:
      print(e)
      print(f"Can't stratify because some states represented in the dataset don't contains enough counties")
      cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
      cv_model = cross_validate(
      pipe, X, y, cv=cv,
      return_estimator=False, n_jobs=-1, fit_params=fit_params,
      scoring=new_custom_scorer)
    except Exception as e:
      print(e)
   
    # add metrics to the lists
    R2.append(np.nanmean(cv_model["test_r2"]))
    Adj_R2.append(np.nanmean(cv_model["test_adj_r2"]))
    RMSE.append(np.nanmean(cv_model["test_rmse"]))
    MAE.append(np.nanmean(cv_model["test_mae"]))
    MAPE.append(np.nanmean(cv_model["test_mape"]))
    names.append(name)
    TIME.append(perf_counter() - top)
  # sign flip these metrics (because lower is better)
  RMSE = [-x for x in RMSE]
  MAE = [-x for x in MAE]
  MAPE = [-x for x in MAPE]
  df_models = pd.DataFrame({"Model": names,
                            "R2": R2,
                            "Adjusted R2": Adj_R2,
                            "RMSE": RMSE,
                            "MAE": MAE,
                            "MAPE": MAPE,
                            "Time Taken": TIME})
  df_models = df_models.sort_values(by="MAPE", ascending=True).set_index("Model")
  return df_models

### Virus introduction (all periods)

In [ ]:
dt, X, y, sample_weight, groups = virus_introduction_all(county_database2_imputed,
                                                         selected_columns_list,
                                                         X_selected_columns_list,
                                                         plot=False)
plt.hist(y)
plt.xlabel("Period")
plt.ylabel("Count")
plt.title("Distribution of COVID-19 introductions for each periods (-1) (3=no introduction)")
plt.tight_layout()

In [ ]:
df_clf = all_models_clf(CLASSIFIERS, NO_SAMPLE_WEIGHT_CLASSIFIERS,
                        NO_PREDICT_PROBA_CLASSIFIERS,
                        custom_scorer_clf, X, y, sample_weight,
                        groups)
df_clf

### Virus introduction (each period)

In [ ]:
dt, X, y_list, \
  sample_weight, groups = virus_introduction_periods(county_database2_imputed,
                                                     selected_columns_list,
                                                     X_selected_columns_list,
                                                     plot=False)
y1, y2, y3, yother = y_list

fig, axs = plt.subplots(2, 2)
axs[0, 0].hist(y1)
axs[0, 0].set_title("Introduction Period 1")
axs[0, 1].hist(y2)
axs[0, 1].set_title("Introduction Period 2")
axs[1, 0].hist(y3)
axs[1, 0].set_title("Introduction Period 3")
axs[1, 1].hist(yother)
axs[1, 1].set_title("Introduction Later")

for ax in axs.flat:
    ax.set(xlabel="Period", ylabel="Count")

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
plt.tight_layout()

Period 1

In [ ]:
# Benchmark
print(f"Benchmark: {100 * (max(np.count_nonzero(y1==0), np.count_nonzero(y1==1))) / len(y1)}")

In [ ]:
df_clf = all_models_clf(CLASSIFIERS, NO_SAMPLE_WEIGHT_CLASSIFIERS,
                        NO_PREDICT_PROBA_CLASSIFIERS,
                        custom_scorer_clf, X, y1, sample_weight,
                        groups)
df_clf

Period 2

In [ ]:
# Benchmark
print(f"Benchmark: {100 * (max(np.count_nonzero(y2==0), np.count_nonzero(y2==1))) / len(y2)}")

In [ ]:
df_clf = all_models_clf(CLASSIFIERS, NO_SAMPLE_WEIGHT_CLASSIFIERS,
                        NO_PREDICT_PROBA_CLASSIFIERS,
                        custom_scorer_clf, X, y2, sample_weight,
                        groups)
df_clf

Period 3

In [ ]:
# Benchmark
print(f"Benchmark: {100 * (max(np.count_nonzero(y3==0), np.count_nonzero(y3==1))) / len(y3)}")

In [ ]:
df_clf = all_models_clf(CLASSIFIERS, NO_SAMPLE_WEIGHT_CLASSIFIERS,
                        NO_PREDICT_PROBA_CLASSIFIERS,
                        custom_scorer_clf, X, y3, sample_weight,
                        groups)
df_clf

Period later than 3

In [ ]:
# Benchmark
print(f"Benchmark: {100 * (max(np.count_nonzero(yother==0), np.count_nonzero(yother==1))) / len(yother)}")

In [ ]:
df_clf = all_models_clf(CLASSIFIERS, NO_SAMPLE_WEIGHT_CLASSIFIERS,
                        NO_PREDICT_PROBA_CLASSIFIERS,
                        custom_scorer_clf, X, yother, sample_weight,
                        groups)
df_clf

### Virus spread

In [ ]:
dt_list, X_list, y_list, \
sample_weight_list, groups_list = virus_spread_dt(county_database2_imputed,
                                                  selected_columns_list,
                                                  X_selected_columns_list,
                                                  type_data="COVID-19",
                                                  plot=False)
dt1, dt2, dt3, dt4, dt5, dt6 = dt_list
X1, X2, X3, X4, X5, X6 = X_list
y1, y2, y3, y4, y5, y6 = y_list
sample_weight1, sample_weight2, sample_weight3, \
  sample_weight4, sample_weight5, sample_weight6 = sample_weight_list
groups1, groups2, groups3, groups4, groups5, \
  groups6 = groups_list

In [ ]:
df_reg1 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X1, y1, sample_weight1,
                         groups1)
df_reg1

In [ ]:
df_reg2 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X2, y2, sample_weight2,
                         groups2)
df_reg2

In [ ]:
df_reg3 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X3, y3, sample_weight3,
                         groups3)
df_reg3

In [ ]:
df_reg4 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X4, y4, sample_weight4,
                         groups4)
df_reg4

In [ ]:
df_reg5 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X5, y5, sample_weight5,
                         groups5)
df_reg5

In [ ]:
df_reg6 = all_models_reg(REGRESSORS, NO_SAMPLE_WEIGHT_REGRESSORS,
                         custom_scorer_reg, X6, y6, sample_weight6,
                         groups6)
df_reg6

### Ensemble model - Regression

In [ ]:
chosen_models = ["HistGradientBoostingRegressor", "LGBMR",]
for mod in chosen_models:
  if mod in no_predict_proba:  # extend the class with a custom predict proba function based on the decision function
    if mod == "NearestCentroid":  # no decision function, calculate pairwise distance to clusters instead
      class model2(model):
        def __init__(self):
          super().__init__()
        def predict_proba(self, X):
          distances = pairwise_distances(X, Y=self.centroids_,
                                          metric="euclidean",
                                          n_jobs=-1)
          return softmax(-distances)
    else:
      class model2(model):
        def __init__(self):
          super().__init__()
        def predict_proba(self, X):
          d = self.decision_function(X)
          return softmax(d)
    model = model2
    model_name = "model2"
  else:
    model_name = f"{mod.lower()}"

HistGBMR, LGBMR, SVR, et LinearSVR, XGBRegressor
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

## Calibration plots

### Initialization

In [ ]:
def cal_data_group(prob, true, group, bins, plot=False, wrap_col=3, sns_height=4, save_plot=False):
  cal_dat = pd.DataFrame({"prob": prob,
                          "true": true,
                          "group": group})
  cal_dat["Count"] = 1
  cal_dat["Bin"] = (cal_dat.groupby("group",as_index=False)["prob"]
                      ).transform(lambda x: pd.qcut(x, bins, labels=range(bins))
                      ).astype(int) + 1
  agg_bins = cal_dat.groupby(["group", "Bin"], as_index=False)["Count", "prob",
                                                               "true"].sum()
  agg_bins["Predicted"] = agg_bins["prob"] / agg_bins["Count"]
  agg_bins["Actual"] = agg_bins["true"] / agg_bins["Count"]
  agg_long = pd.melt(agg_bins, id_vars=["Bin", "group"],
                     value_vars=["Predicted", "Actual"], 
                     var_name="Type", value_name="Probability")
  if plot:
    d = {"marker": ["o", "X"]}
    ax = sns.FacetGrid(data=agg_long, col="group", hue="Type", hue_kws=d,
                       col_wrap=wrap_col, despine=False, height=sns_height)
    ax.map(plt.plot, "Bin", "Probability", markeredgecolor="w")
    ax.set_titles("{col_name}")
    ax.set_xlabels("")
    ax.set_xticklabels("")
    ax.axes[0].legend(loc="upper left")
    # Setting xlim in FacetGrid not behaving how I want
    for a in ax.axes:
        a.set_xlim([0,bins+1])
        a.tick_params(length=0)
    if save_plot:
        plt.savefig(save_plot, dpi=500, bbox_inches="tight")
    plt.show()
  return agg_bins

# function to use when determining groups using quantiles
def determine_group(p: float, quantiles: list,
                    names: list) -> str:
  for i in range(len(quantiles)-1, -1, -1):
    if p >= quantiles[i]:
      return names[i+1]
  return names[0]

### Virus Introduction (LogisticRegression)

In [ ]:
county_database2_imputed.index = county_database2_imputed.FIPS
dt = county_database2_imputed[selected_columns]  # total_pop and deathcounts will be removed later

dt["total_pop"] = np.log(dt["total_pop"])  # sample weights
print(dt.shape)

# drop nan values
dt.dropna(inplace=True)
print("Dataset shape after nan values removed: {}".format(dt.shape))

# Dataset: Period 1, 2, 3 or after
X = dt[X_selected_columns]
y = np.array([0 for _ in range(len(dt))])
for i in range(len(dt)):
  p1_count = (dt["deathCount_period1"].iloc[i] >= 5)
  p2_count = (dt["deathCount_period2"].iloc[i] >= 5)
  p3_count = (dt["deathCount_period3"].iloc[i] >= 5)
  if p1_count:
    y[i] = 0

  else:
    y[i] = 1
  """
  elif p2_count and not(p1_count):
    y[i] = 1
  elif p3_count and not(p1_count or p2_count):
    y[i] = 2
  else:
    y[i] = 3
  """
# weight the regression by the log of a county’s population
sample_weight = dt["total_pop"].to_numpy()

In [ ]:
from sklearn.linear_model import LogisticRegression

name, model = "LogisticRegression", LogisticRegression
pipe = make_pipeline(MinMaxScaler(), model())

model_params = {}
if "random_state" in model().get_params().keys():
  model_params[f"{name.lower()}__random_state"] = 42
if "n_jobs" in model().get_params().keys():
  model_params[f"{name.lower()}__n_jobs"] = -1
if model_params != {}:
  pipe.set_params(**model_params)

if name in NO_SAMPLE_WEIGHT_CLASSIFIERS:
  fit_params = {}
else:
  fit_params = {f"{name.lower()}__sample_weight": sample_weight}  # sample_weight_train

pipe.fit(X, y, **fit_params)  # X_train, y_train, **fit_params
y_pred = pipe.predict_proba(X)  # X_test

In [ ]:
from sklearn.calibration import calibration_curve
true_prob, pred_prob = calibration_curve(y, y_pred[:, 1], n_bins=10)  # y_test

In [ ]:
fig, ax = plt.subplots()
# only these two lines are calibration curves
plt.plot(pred_prob, true_prob, marker='o', linewidth=1, label="LogisticRegression")

American Communities

In [ ]:
group = list(map(lambda x: county_info_dic[x], X["acp"].to_numpy()))
cal_data_group(prob=y_pred[:,1], true=y, group=group, bins=20, plot=True,
               save_plot="Plots/Calibration Plots by American Communities.png")

HHS Regions

In [ ]:
group = list(map(lambda x: hhs_region_dic[x], X["HHS Region"].to_numpy()))
cal_data_group(prob=y_pred[:,1], true=y, group=group, bins=20, plot=True,
               save_plot="Plots/Calibration Plots by HHS Regions.png")

Political leaning

In [ ]:
political_quantiles = list(map(lambda q: np.quantile(X["political_leaning"], q),
                          [0.2, 0.4, 0.6, 0.8]))
political_names = ["Dem.", "Weakly dem", "Weakly rep.", "Rep.", "Strongly Rep"]

group = list(map(lambda x: determine_group(x, political_quantiles,
                                           political_names),
                 X["political_leaning"].to_numpy()))
cal_data_group(prob=y_pred[:,1], true=y, group=group, bins=20, plot=True,
               save_plot="Plots/Calibration Plots by Political Leaning.png")

Ethnicity (hispanic)

In [ ]:
ethnicity_quantiles = list(map(lambda q: np.quantile(X["pct_hispanic"], q),
                          [1/3, 2/3]))
ethnicity_names = ["Low hispanic ethnicity", "Medium hispanic ethnicity",
                   "High hispanic ethnicity"]
group = list(map(lambda x: determine_group(x, ethnicity_quantiles,
                                           ethnicity_names),
                 X["pct_hispanic"].to_numpy()))
cal_data_group(prob=y_pred[:,1], true=y, group=group, bins=20, plot=True,
               save_plot="Plots/Calibration Plots by Hispanic Ethnicity.png")

# Analysis through the lens of county community classification

## First analysis

In [ ]:
import plotly.express as px
import math
# rename communities
fig = px.pie(county_database, names="acp_name_with_number", title="Number of counties per communities")
fig.show()

In [ ]:
import plotly.express as px
# rename communities
bypop = county_database.groupby(by="acp_name_with_number")["total_pop"].sum().to_frame()
bypop.reset_index(drop=False, inplace=True)
bypop
fig = px.pie(bypop, names="acp_name_with_number", values="total_pop",
             title="Population per communities")
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.update_layout(legend_title="American Communities (id & name)")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs.layout import Grid

# population threshold: percentage of the american population >= pop_threshold
total_american_pop = county_database["total_pop"].sum()
pop_threshold = 0.05  # 5%
communities_to_select = []
for acp in range(1, 16):
  acp_df = county_database[county_database["acp"] == acp]
  if (acp_df["total_pop"].sum() / total_american_pop) >= pop_threshold:
    communities_to_select.append(acp)

fig = make_subplots(rows=2, cols=3)

# colors of the communities
color_list = ["cyan", "red", "green", "pink",
              "gray", "darkblue", "coral", "maroon"]

for per in range(6):
  acp = communities_to_select[0]
  acp_df = county_database[county_database["acp"] == acp]
  acp_name_with_number = acp_df['acp_name_with_number'].iloc[0]
  if per != 0:
    name = ""
  else:
    name = f"{acp_name_with_number}"
  fig.add_trace(go.Histogram(x=acp_df[f"deathRate_period{per+1}"].to_numpy(),
                             y=(100 * acp_df["total_pop"] / total_american_pop).to_numpy(),
                             histfunc="sum",
                             name=name,
                             marker={"color": color_list[0]}),
                row=(per//3 + 1), col=(per%3 + 1))
  for j, acp in enumerate(communities_to_select[1:]):
    acp_df = county_database[county_database["acp"] == acp]
    acp_name_with_number = acp_df['acp_name_with_number'].iloc[0]
    if per != 0:
      name = ""
    else:
      name = f"{acp_name_with_number}"
    fig.append_trace(go.Histogram(x=acp_df[f"deathRate_period{per+1}"].to_numpy(),
                                  y=(100 * acp_df["total_pop"] / total_american_pop).to_numpy(),
                                  histfunc="sum",
                                  name=name,
                                  marker={"color": color_list[j+1]}),
                     row=(per//3 + 1), col=(per%3 + 1))

fig.update_layout(title="County's Death Rates (COVID-19 related, per 100k residents) per community<br>Only plot communities accounting for more than {thresh}% of the american population".format(thresh=round(100*pop_threshold, 0)),
                  xaxis_title_text="COVID-19 death rate (per 100k residents)",
                  yaxis_title_text="% of the American population",
                  plot_bgcolor ="rgb(255,255,255)")
fig.update_xaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")
fig.update_yaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see all histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs.layout import Grid

# population threshold: percentage of the american population >= pop_threshold
total_american_pop = county_database["total_pop"].sum()
pop_threshold = 0.05  # 5%
communities_to_select = []
for acp in range(1, 16):
  acp_df = county_database[county_database["acp"] == acp]
  if (acp_df["total_pop"].sum() / total_american_pop) >= pop_threshold:
    communities_to_select.append(acp)

fig = go.Figure()
for acp in communities_to_select:
  acp_df = county_database[county_database["acp"] == acp]
  fig.add_trace(go.Histogram(x=acp_df["deathRate_period2"].to_numpy(),
                             y=(100 * acp_df["total_pop"] / total_american_pop).to_numpy(),
                             histfunc="sum",
                             name=acp_df["acp_name_with_number"].iloc[0]))

fig.update_layout(title="Period 2 County's Death Rates (COVID-19 related, per 100k residents) per community<br>Only plot communities accounting for more than {thresh}% of the american population".format(thresh=round(100*pop_threshold, 0)),
                  xaxis_title_text="COVID-19 death rate (per 100k residents)",
                  yaxis_title_text="% of the American population",
                  plot_bgcolor ="rgb(255,255,255)")
fig.update_xaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")
fig.update_yaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see all histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs.layout import Grid

# population threshold: percentage of the american population >= pop_threshold
total_american_pop = county_database["total_pop"].sum()
pop_threshold = 0.05  # 5%
communities_to_select = []
for acp in range(1, 16):
  acp_df = county_database[county_database["acp"] == acp]
  if (acp_df["total_pop"].sum() / total_american_pop) >= pop_threshold:
    communities_to_select.append(acp)

fig = go.Figure()
for acp in communities_to_select:
  acp_df = county_database[county_database["acp"] == acp]
  fig.add_trace(go.Histogram(x=acp_df["deathRate_period3"].to_numpy(),
                             y=(100 * acp_df["total_pop"] / total_american_pop).to_numpy(),
                             histfunc="sum",
                             name=acp_df["acp_name_with_number"].iloc[0]))

fig.update_layout(title="Period 3 County's Death Rates (COVID-19 related, per 100k residents) per community<br>Only plot communities accounting for more than {thresh}% of the american population".format(thresh=round(100*pop_threshold, 0)),
                  xaxis_title_text="COVID-19 death rate (per 100k residents)",
                  yaxis_title_text="% of the American population",
                  plot_bgcolor ="rgb(255,255,255)")
fig.update_xaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")
fig.update_yaxes(showline=True, linewidth=2, linecolor="black", gridcolor="grey")

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see all histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
import plotly.graph_objects as go
acp = np.array(list(county_info_dic.items()))[:,1]  # communities ordered by id (from 1 to 15) (without Unknown)

period1_data = []
period2_data = []
period3_data = []
error_p1 = []
error_p2 = []
error_p3 = []
for c_name in acp:
  subdf = county_database[county_database["acp_name"] == c_name]
  p1 = subdf["deathRate_period1"].mean()
  e1 = 1.96 * subdf["deathRate_period1"].std() / np.sqrt(len(subdf))
  p2 = subdf["deathRate_period2"].mean()
  e2 = 1.96 * subdf["deathRate_period2"].std() / np.sqrt(len(subdf))
  p3 = subdf["deathRate_period3"].mean()
  e3 = 1.96 * subdf["deathRate_period3"].std() / np.sqrt(len(subdf))
  period1_data.append(p1)
  period2_data.append(p2)
  period3_data.append(p3)
  error_p1.append(e1)
  error_p2.append(e2)
  error_p3.append(e3)
fig = go.Figure(data=[
    go.Bar(name="Period 1 (February 2020 - May 2020)", x=acp, y=period1_data,
           error_y=dict(type="data", array=error_p1)),
    go.Bar(name="Period 2 (June 2020 - October 2020)", x=acp, y=period2_data,
           error_y=dict(type="data", array=error_p2)),
    go.Bar(name="Period 3 (November 2020 - February 2021)", x=acp,
           y=period3_data, error_y=dict(type="data", array=error_p3))
])
# Change the bar mode
fig.update_layout(barmode="group",
                  xaxis_title="American Communities",
                  yaxis_title="Counties' average death rate",
                  legend_title="Periods",
                  title="Counties' average death rate (COVID-19 related, per 100k residents)<br> per period per community - 95% confidence interval")
fig.show()

In [ ]:
period1_data_sorted, acp1 = [], []
for x, y in sorted(zip(period1_data, acp), key=lambda x: x[0], reverse=True):
  period1_data_sorted.append(x)
  acp1.append(y)
ranking_period1 = pd.DataFrame(period1_data_sorted, index=acp1)
print("Ranking period 1")
print(ranking_period1)
period2_data_sorted, acp2 = [], []
for x, y in sorted(zip(period2_data, acp), key=lambda x: x[0], reverse=True):
  period2_data_sorted.append(x)
  acp2.append(y)
ranking_period2 = pd.DataFrame(period2_data_sorted, index=acp2)
print("Ranking period 2")
print(ranking_period2)
period3_data_sorted, acp3 = [], []
for x, y in sorted(zip(period3_data, acp), key=lambda x: x[0], reverse=True):
  period3_data_sorted.append(x)
  acp3.append(y)
ranking_period3 = pd.DataFrame(period3_data_sorted, index=acp3)
print("Ranking period 3")
print(ranking_period3)

Remove average per period across all counties to remove a trend

In [ ]:
import plotly.graph_objects as go
acp = np.array(list(county_info_dic.items()))[:,1]  # communities ordered by id (from 1 to 15) (without Unknown)

period1_data = []
period2_data = []
period3_data = []
avg_p1 = county_database["deathRate_period1"].dropna().mean()
avg_p2 = county_database["deathRate_period2"].dropna().mean()
avg_p3 = county_database["deathRate_period3"].dropna().mean()
# error_p1 = []
# error_p2 = []
# error_p3 = []
for c_name in acp:
  subdf = county_database[county_database["acp_name"] == c_name]
  p1 = subdf["deathRate_period1"].mean()
  # e1 = 1.96 * subdf["deathRate_period1"].std() / np.sqrt(len(subdf))
  p2 = subdf["deathRate_period2"].mean()
  # e2 = 1.96 * subdf["deathRate_period2"].std() / np.sqrt(len(subdf))
  p3 = subdf["deathRate_period3"].mean()
  # e3 = 1.96 * subdf["deathRate_period3"].std() / np.sqrt(len(subdf))
  period1_data.append(p1 - avg_p1)
  period2_data.append(p2 - avg_p2)
  period3_data.append(p3 - avg_p3)
fig = go.Figure(data=[
    # go.Bar(name="Period 1 (February 2020 - May 2020)", x=acp, y=period1_data,
    #        error_y=dict(type="data", array=error_p1)),
    # go.Bar(name="Period 2 (June 2020 - October 2020)", x=acp, y=period2_data,
    #        error_y=dict(type="data", array=error_p2)),
    # go.Bar(name="Period 3 (November 2020 - February 2021)", x=acp,
    #        y=period3_data, error_y=dict(type="data", array=error_p3))
    go.Bar(name="Period 1 (February 2020 - May 2020)", x=acp, y=period1_data),
    go.Bar(name="Period 2 (June 2020 - October 2020)", x=acp, y=period2_data),
    go.Bar(name="Period 3 (November 2020 - February 2021)", x=acp,
           y=period3_data)
])
# Change the bar mode
fig.update_layout(barmode="group",
                  xaxis_title="American Communities",
                  yaxis_title="Difference between counties' average death rate and average death rate per period",
                  legend_title="Periods",
                  title="Counties' average death rate (COVID-19 related, per 100k residents)<br> per period per community minus average death per period")
fig.show()

## Scatter plot

Plot for period 1

In [ ]:
import plotly.graph_objects as go

col_to_select = ["acp", "acp_name", "total_pop", "FIPS"]
for i in range(1, 7):
  col_to_select.append(f"deathRate_period{i}")
county_database_covid = county_databases["COVID-19"]["county_database"][col_to_select].dropna()
county_database_excess = county_databases["Excess Mortality"]["county_database"][col_to_select].dropna()
county_database_excess = county_database_excess[county_database_excess["FIPS"].isin(county_database_covid["FIPS"])]

fig = go.Figure()
    
for acp in np.unique(county_database_excess["acp"]):
  subdf = county_database_excess[county_database_excess["acp"] == acp]
  size = np.log(subdf["total_pop"]).to_numpy()
  y = subdf["deathRate_period1"].to_numpy()
  x = []
  for _, fips in enumerate(subdf["FIPS"]):
    x.append(county_database_covid.loc[fips]["deathRate_period1"])
  x = np.array(x)
  fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    marker=dict(size=size),
    name=subdf["acp_name"].iloc[0]))

x_tot = []
for _, fips in enumerate(county_database_excess["FIPS"]):
  x_tot.append(county_database_covid.loc[fips]["deathRate_period1"])
x_tot = np.array(x_tot)

fig.add_trace(go.Scatter(
    x=np.linspace(x_tot.min(), x_tot.max(), 1000),
    y=np.linspace(x_tot.min(), x_tot.max(), 1000),
    line=dict(shape="linear", color="rgb(0, 0, 0)", dash="dash"),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=np.linspace(x_tot.min(), x_tot.max(), 1000),
    y=np.zeros(1000),
    line=dict(shape="linear", color="rgb(0, 0, 0)"),
    showlegend=False
))


fig.update_layout(legend_title_text="American Communities")
fig.update_xaxes(title_text="COVID-19 Crude Deaths Rate (per 100k residents)")
fig.update_yaxes(title_text="Excess All Causes Crude Deaths Rate (per 100k residents)")
fig.show()

Bar for interactive plot

In [ ]:
import plotly.graph_objects as go


col_to_select = ["acp", "acp_name", "total_pop", "FIPS"]
for i in range(1, 7):
  col_to_select.append(f"deathRate_period{i}")
county_database_covid = county_databases["COVID-19"]["county_database"][col_to_select].dropna()
county_database_excess = county_databases["Excess Mortality"]["county_database"][col_to_select].dropna()
county_database_excess = county_database_excess[county_database_excess["FIPS"].isin(county_database_covid["FIPS"])]

fig = go.Figure()

# Add traces for each slider step (period)
for period in range(1, 7):
  for acp in np.unique(county_database_excess["acp"]):
    subdf = county_database_excess[county_database_excess["acp"] == acp]
    size = np.log(subdf["total_pop"]).to_numpy()
    y = subdf[f"deathRate_period{period}"].to_numpy()
    x = []
    for _, fips in enumerate(subdf["FIPS"]):
      x.append(county_database_covid.loc[fips][f"deathRate_period{period}"])
    x = np.array(x)
    fig.add_trace(go.Scatter(
      visible=False,
      x=x,
      y=y,
      mode="markers",
      marker=dict(size=size),
      name=subdf["acp_name"].iloc[0]))

  x_tot = []
  for _, fips in enumerate(county_database_excess["FIPS"]):
    x_tot.append(county_database_covid.loc[fips][f"deathRate_period{period}"])
  x_tot = np.array(x_tot)

  fig.add_trace(go.Scatter(
      visible=False,
      x=np.linspace(x_tot.min(), x_tot.max(), 1000),
      y=np.linspace(x_tot.min(), x_tot.max(), 1000),
      line=dict(shape="linear", color="rgb(0, 0, 0)", dash="dash"),
      showlegend=False
  ))

  fig.add_trace(go.Scatter(
      visible=False,
      x=np.linspace(x_tot.min(), x_tot.max(), 1000),
      y=np.zeros(1000),
      line=dict(shape="linear", color="rgb(0, 0, 0)"),
      showlegend=False
  ))

# Make 1th period visible
for i in range(17):
  fig.data[i].visible = True

# Create and add slider
steps = []
for i in range(1, 7):  # len(fig.data)
    step = dict(
        method="restyle",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Excess Mortality vs COVID-19 mortality per county per period. Period: " + str(i)}],  # layout attribute
        label=f"{i}"
    )
    for j in range(17):
      step["args"][0]["visible"][(i-1)*17 + j] = True  # Toggle i'th period to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Period: "},
    pad={"t": 10},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="Excess Mortality vs COVID-19 mortality per county per period.",
    legend_title_text="American Communities",
    xaxis_title="COVID-19 Crude Deaths Rate (per 100k residents)",
    yaxis_title="Excess All Causes Crude Deaths Rate (per 100k residents)"
)

fig.show()

Animation with play button

In [ ]:
import plotly.graph_objects as go

# Import Data
col_to_select = ["acp", "acp_name", "total_pop", "FIPS"]
for i in range(1, 7):
  col_to_select.append(f"deathRate_period{i}")
county_database_covid = county_databases["COVID-19"]["county_database"][col_to_select].dropna()
county_database_excess = county_databases["Excess Mortality"]["county_database"][col_to_select].dropna()
county_database_excess = county_database_excess[county_database_excess["FIPS"].isin(county_database_covid["FIPS"])]

for period in range(1, 7):  # add covid mortality into excess mortality dataframe
  x_tot = []
  for _, fips in enumerate(county_database_excess["FIPS"]):
    x_tot.append(county_database_covid.loc[fips][f"deathRate_period{period}"])
  county_database_excess[f"deathRate_covid_period{period}"] = x_tot

periods = list(str(i) for i in range(1, 7))

# make list of american communities (acp=american communities project)
acps = list(np.unique(county_database_excess["acp"]))

# Figure
fig_dict  = {
  'data': [],
  'layout': {},
  'frames': [],
  'config': {'scrollzoom': True}
}

# fill in most of layout
max_covid = county_database_excess[[f"deathRate_covid_period{period}" for period in periods]].max().max()
min_excess = county_database_excess[[f"deathRate_period{period}" for period in periods]].min().min()
max_excess = county_database_excess[[f"deathRate_period{period}" for period in periods]].max().max()
# adjust a little bit the min and max
max_covid = 1.05 * max_covid
max_excess = 1.05 * max_excess
min_excess = 1.05 * min_excess if min_excess < 0 else 0.95 * min_excess
fig_dict['layout']['xaxis'] = {'title': 'COVID-19 Crude Deaths Rate (per 100k residents)',
                               'gridcolor': '#FFFFFF',
                               'range': [0, max_covid]}
fig_dict['layout']['yaxis'] = {'title': 'Excess All Causes Crude Deaths Rate (per 100k residents)',
                               'gridcolor': '#FFFFFF',
                               'range': [min_excess, max_excess]}
fig_dict['layout']['legend'] = {'title_text': 'American Communities'}
fig_dict['layout']['title'] = 'Excess Mortality vs COVID-19 mortality per county per period.'
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
fig_dict['layout']['width'] = 1280
fig_dict['layout']['height'] = 720

# Add Slider
sliders_dict = {
  'active': 0,
  'yanchor': 'top',
  'xanchor': 'left',
  'currentvalue': {
    'font': {'size': 20},
    'prefix': 'Period:',
    'visible': True,
    'xanchor': 'right'
  },
  'transition': {'duration': 300, 'easing': 'cubic-in-out'},
  'pad': {'b': 10, 't': 50},
  'len': 0.9,
  'x': 0.1,
  'y': 0,
  'steps': []
}

# Add Play and Pause Buttons
fig_dict['layout']['updatemenus'] = [
  {
    'buttons': [
      {
        'args': [None, {'frame': {'duration': 500, 'redraw': False},
                        'fromcurrent': True,
                        'transition': {'duration': 300,
                                       'easing': 'quadratic-in-out'}}],
        'label': 'Play',
        'method': 'animate'
      },
      {
        'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                          'mode': 'immediate',
                          'transition': {'duration': 0}}],
        'label': 'Pause',
        'method': 'animate'
      }
    ],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
  }
]

# Intitialization: period 1
period = 1
for acp in acps:
  dataset_by_acp = county_database_excess[county_database_excess["acp"] == acp]

  data_dict = {
    "x": list(dataset_by_acp[f"deathRate_covid_period{period}"]),
    "y": list(dataset_by_acp[f"deathRate_period{period}"]),
    "mode": "markers",
    "text": list(dataset_by_acp["FIPS"]),
    "marker": {
      "sizemode": "area",
      # "sizeref": 1e6,
      "size": list(np.log(dataset_by_acp["total_pop"]))
    },
    "name": dataset_by_acp["acp_name"].iloc[0]
  }
  fig_dict["data"].append(data_dict)
# Add diagonal line
data_dict = {
  "x": list(np.linspace(0, max_covid, 1000)),
  "y": list(np.linspace(0, max_covid, 1000)),
  "mode": "lines",
  "line": {
    "shape": "linear",
    "color": "rgb(0, 0, 0)",
    "dash": "dash"
  },
  "showlegend": False
}
fig_dict["data"].append(data_dict)
# Add horizontal line
data_dict = {
  "x": list(np.linspace(min_excess, max_excess, 1000)),
  "y": list(np.zeros(1000)),
  "mode": "lines",
  "line": {
    "shape": "linear",
    "color": "rgb(0, 0, 0)"
  },
  "showlegend": False
}
fig_dict["data"].append(data_dict)

# Create Frames and Animation
for period in periods:
  frame = {"data": [], "name": str(period)}
  for acp in acps:
    dataset_by_acp = county_database_excess[county_database_excess["acp"] == acp]

    data_dict = {
      "x": list(dataset_by_acp[f"deathRate_covid_period{period}"]),
      "y": list(dataset_by_acp[f"deathRate_period{period}"]),
      "mode": "markers",
      "text": list(dataset_by_acp["FIPS"]),
      "marker": {
        "sizemode": "area",
        # "sizeref": 1e6,
        "size": list(np.log(dataset_by_acp["total_pop"]))
      },
      "name": dataset_by_acp["acp_name"].iloc[0]
    }
    frame["data"].append(data_dict)
  
  # Add diagonal line
  data_dict = {
    "x": list(np.linspace(0, max_covid, 1000)),
    "y": list(np.linspace(0, max_covid, 1000)),
    "mode": "lines",
    "line": {
      "shape": "linear",
      "color": "rgb(0, 0, 0)",
      "dash": "dash"
    },
    "showlegend": False
  }
  frame["data"].append(data_dict)
  # Add horizontal line
  data_dict = {
    "x": list(np.linspace(min_excess, max_excess, 1000)),
    "y": list(np.zeros(1000)),
    "mode": "lines",
    "line": {
      "shape": "linear",
      "color": "rgb(0, 0, 0)"
    },
    "showlegend": False
  }
  frame["data"].append(data_dict)

  fig_dict["frames"].append(frame)
  slider_step = {"args": [
      [period],
      {"frame": {"duration": 300, "redraw": False},
        "mode": "immediate",
        "transition": {"duration": 300}}
  ],
      "label": period,
      "method": "animate"}
  sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

Save the animation in gif format

In [ ]:
%pip install -U gif[plotly]

In [ ]:
# Save animation

import gif  # !pip install -U gif[plotly]
@gif.frame
def for_gif(frame):
  fig_dict  = {
    'data': [],
    'layout': {},
    'config': {'scrollzoom': True}
  }

  period = frame["name"]

  fig_dict['layout']['xaxis'] = {'title': 'COVID-19 Crude Deaths Rate (per 100k residents)',
                                'gridcolor': '#FFFFFF',
                                'range': [0, max_covid]}
  fig_dict['layout']['yaxis'] = {'title': 'Excess All Causes Crude Deaths Rate (per 100k residents)',
                                'gridcolor': '#FFFFFF',
                                'range': [min_excess, max_excess]}
  fig_dict['layout']['legend'] = {'title_text': 'American Communities'}
  fig_dict['layout']['title'] = f'Excess Mortality vs COVID-19 mortality per county. Period: {period}'
  fig_dict['layout']['hovermode'] = 'closest'
  fig_dict['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
  fig_dict['layout']['width'] = 1280
  fig_dict['layout']['height'] = 720

  for data_dict in frame["data"]:
    fig_dict["data"].append(data_dict)

  fig = go.Figure(fig_dict)
  return fig

frames = []
for frame in fig.to_dict()["frames"]:
  f = for_gif(frame)
  frames.append(f)
for _ in range(4):  # duplicate the last frame for a smoother animation!
  frames.append(f)
gif.save(frames, "Plot/animation.gif", duration=450)

Animation: mean per american communities

In [ ]:
import plotly.graph_objects as go

# Import Data
col_to_select = ["acp", "acp_name", "total_pop", "FIPS"]
for i in range(1, 7):
  col_to_select.append(f"deathRate_period{i}")
county_database_covid = county_databases["COVID-19"]["county_database"][col_to_select].dropna()
county_database_excess = county_databases["Excess Mortality"]["county_database"][col_to_select].dropna()
county_database_excess = county_database_excess[county_database_excess["FIPS"].isin(county_database_covid["FIPS"])]

for period in range(1, 7):  # add covid mortality into excess mortality dataframe
  x_tot = []
  for _, fips in enumerate(county_database_excess["FIPS"]):
    x_tot.append(county_database_covid.loc[fips][f"deathRate_period{period}"])
  county_database_excess[f"deathRate_covid_period{period}"] = x_tot

periods = list(str(i) for i in range(1, 7))

# make list of american communities (acp=american communities project)
acps = list(np.unique(county_database_excess["acp"]))

# Figure
fig_dict  = {
  'data': [],
  'layout': {},
  'frames': [],
  'config': {'scrollzoom': True}
}

# fill in most of layout
max_covid = county_database_excess[["acp"]+[f"deathRate_covid_period{period}" for period in periods]].groupby(by="acp").mean().max().max()
min_excess = county_database_excess[["acp"]+[f"deathRate_period{period}" for period in periods]].groupby(by="acp").mean().min().min()
max_excess = county_database_excess[["acp"]+[f"deathRate_period{period}" for period in periods]].groupby(by="acp").mean().max().max()
# adjust a little bit the min and max
max_covid = 1.05 * max_covid
max_excess = 1.05 * max_excess
min_excess = 1.05 * min_excess if min_excess < 0 else 0.95 * min_excess
fig_dict['layout']['xaxis'] = {'title': 'Average COVID-19 Crude Deaths Rate (per 100k residents)',
                               'gridcolor': '#FFFFFF',
                               'range': [0, max_covid]}
fig_dict['layout']['yaxis'] = {'title': 'Average Excess All Causes Crude Deaths Rate (per 100k residents)',
                               'gridcolor': '#FFFFFF',
                               'range': [min_excess, max_excess]}
fig_dict['layout']['legend'] = {'title_text': 'American Communities'}
fig_dict['layout']['title'] = 'Excess Mortality vs COVID-19 mortality per american communities (average) per period.'
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
fig_dict['layout']['width'] = 1280
fig_dict['layout']['height'] = 720

# Add Slider
sliders_dict = {
  'active': 0,
  'yanchor': 'top',
  'xanchor': 'left',
  'currentvalue': {
    'font': {'size': 20},
    'prefix': 'Period:',
    'visible': True,
    'xanchor': 'right'
  },
  'transition': {'duration': 300, 'easing': 'cubic-in-out'},
  'pad': {'b': 10, 't': 50},
  'len': 0.9,
  'x': 0.1,
  'y': 0,
  'steps': []
}

# Add Play and Pause Buttons
fig_dict['layout']['updatemenus'] = [
  {
    'buttons': [
      {
        'args': [None, {'frame': {'duration': 500, 'redraw': False},
                        'fromcurrent': True,
                        'transition': {'duration': 300,
                                       'easing': 'quadratic-in-out'}}],
        'label': 'Play',
        'method': 'animate'
      },
      {
        'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                          'mode': 'immediate',
                          'transition': {'duration': 0}}],
        'label': 'Pause',
        'method': 'animate'
      }
    ],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
  }
]

# Intitialization: period 1
period = 1
for acp in acps:
  dataset_by_acp = county_database_excess[county_database_excess["acp"] == acp]

  data_dict = {
    "x": [dataset_by_acp[f"deathRate_covid_period{period}"].mean()],
    "y": [dataset_by_acp[f"deathRate_period{period}"].mean()],
    "mode": "markers",
    "text": [dataset_by_acp["acp_name"].iloc[0]],
    "marker": {
      "sizemode": "area",
      "sizeref": 300,
      "size": [dataset_by_acp["total_pop"].mean()]
    },
    "name": dataset_by_acp["acp_name"].iloc[0]
  }
  fig_dict["data"].append(data_dict)
# Add diagonal line
data_dict = {
  "x": list(np.linspace(0, max_covid, 1000)),
  "y": list(np.linspace(0, max_covid, 1000)),
  "mode": "lines",
  "line": {
    "shape": "linear",
    "color": "rgb(0, 0, 0)",
    "dash": "dash"
  },
  "showlegend": False
}
fig_dict["data"].append(data_dict)
# Add horizontal line
data_dict = {
  "x": list(np.linspace(min_excess, max_excess, 1000)),
  "y": list(np.zeros(1000)),
  "mode": "lines",
  "line": {
    "shape": "linear",
    "color": "rgb(0, 0, 0)"
  },
  "showlegend": False
}
fig_dict["data"].append(data_dict)

# Create Frames and Animation
for period in periods:
  frame = {"data": [], "name": str(period)}
  for acp in acps:
    dataset_by_acp = county_database_excess[county_database_excess["acp"] == acp]

    data_dict = {
      "x": [dataset_by_acp[f"deathRate_covid_period{period}"].mean()],
      "y": [dataset_by_acp[f"deathRate_period{period}"].mean()],
      "mode": "markers",
      "text": [dataset_by_acp["acp_name"].iloc[0]],
      "marker": {
        "sizemode": "area",
        "sizeref": 300,
        "size": [dataset_by_acp["total_pop"].mean()]
      },
      "name": dataset_by_acp["acp_name"].iloc[0]
    }
    frame["data"].append(data_dict)
  
  # Add diagonal line
  data_dict = {
    "x": list(np.linspace(0, max_covid, 1000)),
    "y": list(np.linspace(0, max_covid, 1000)),
    "mode": "lines",
    "line": {
      "shape": "linear",
      "color": "rgb(0, 0, 0)",
      "dash": "dash"
    },
    "showlegend": False
  }
  frame["data"].append(data_dict)
  # Add horizontal line
  data_dict = {
    "x": list(np.linspace(min_excess, max_excess, 1000)),
    "y": list(np.zeros(1000)),
    "mode": "lines",
    "line": {
      "shape": "linear",
      "color": "rgb(0, 0, 0)"
    },
    "showlegend": False
  }
  frame["data"].append(data_dict)

  fig_dict["frames"].append(frame)
  slider_step = {"args": [
      [period],
      {"frame": {"duration": 300, "redraw": False},
        "mode": "immediate",
        "transition": {"duration": 300}}
  ],
      "label": period,
      "method": "animate"}
  sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

Save to gif

In [ ]:
# Save animation

import gif  # !pip install -U gif[plotly]
@gif.frame
def for_gif(frame):
  fig_dict  = {
    'data': [],
    'layout': {},
    'config': {'scrollzoom': True}
  }

  period = frame["name"]

  fig_dict['layout']['xaxis'] = {'title': 'Average COVID-19 Crude Deaths Rate (per 100k residents)',
                                'gridcolor': '#FFFFFF',
                                'range': [0, max_covid]}
  fig_dict['layout']['yaxis'] = {'title': 'Average Excess All Causes Crude Deaths Rate (per 100k residents)',
                                'gridcolor': '#FFFFFF',
                                'range': [min_excess, max_excess]}
  fig_dict['layout']['legend'] = {'title_text': 'American Communities'}
  fig_dict['layout']['title'] = f'Excess Mortality vs COVID-19 mortality per american communities (average). Period: {period}'
  fig_dict['layout']['hovermode'] = 'closest'
  fig_dict['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
  fig_dict['layout']['width'] = 1280
  fig_dict['layout']['height'] = 720

  for data_dict in frame["data"]:
    fig_dict["data"].append(data_dict)

  fig = go.Figure(fig_dict)
  return fig

frames = []
for frame in fig.to_dict()["frames"]:
  f = for_gif(frame)
  frames.append(f)
for _ in range(4):  # duplicate the last frame for a smoother animation!
  frames.append(f)
gif.save(frames, "Plot/animation_mean_acp.gif", duration=450)